<font size="10"> POPCORN TIME </font><br/>
![](https://conversation.which.co.uk/wp-content/uploads/2012/03/popcornshutterstock_61405375.jpg)
This kernel is under development.




Credit: https://www.kaggle.com/ashishpatel26/now-you-see-me

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import datetime
import seaborn as sns
import ast
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.patches as mpatches
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from collections import defaultdict, Counter
from scipy.stats import mode

from sklearn import *
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")
plt.style.use('fivethirtyeight')
import xgboost as xg
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
df = pd.read_csv("../input/train.csv",index_col="id")
df_test = pd.read_csv("../input/test.csv",index_col="id")
df_total = pd.concat([df, df_test])
df_total["train_bool"] = True
df_total.loc[df_test.index,"train_bool"] = False

In [ ]:
df.describe()


In [ ]:
missing = df.isnull().sum()
print("Train Total missing: {:.2f}".format(missing.sum()/np.product(df.shape)))

pd.concat([missing/df.shape[0], missing],axis=1,keys=["Percent","Total"]).sort_values("Total",ascending=False)[:20]

In [ ]:
missing = df_test.isnull().sum()
print("Test Total missing: {:.2f}".format(missing.sum()/np.product(df_test.shape)))

pd.concat([missing/df_test.shape[0], missing],axis=1,keys=["Percent","Total"]).sort_values("Total",ascending=False)[:20]

*  homepage, poster_path, tmbd_id drop.

In [ ]:
df.drop(["imdb_id", "homepage","poster_path"], axis=1, inplace=True)
df_test.drop(["imdb_id", "homepage","poster_path"], axis=1, inplace=True)
df_total.drop(["imdb_id", "homepage","poster_path"], axis=1, inplace=True)

<font size="5">Revenue</font><br/>

In [ ]:
df["revenue"].describe()

* 1 dolar revenue is really unrealistic.Later Im gonna remove these.

In [ ]:
plt.figure(figsize=(15,5))
sns.distplot(df.loc[df["revenue"] < (10**5), "revenue"]);
plt.title("Revenue Less than 100k");

In [ ]:
plt.figure(figsize=(15,5))
sns.distplot(df["revenue"]);
plt.title("Revenue")
print("Kurtosis ", df.revenue.kurtosis())
print("Skewness:", df.revenue.skew())

* Highy positive skewed data

In [ ]:
df["revenue_log"] = np.log(df["revenue"])

In [ ]:
plt.figure(figsize=(15,5))
sns.distplot(df["revenue_log"]);
plt.title("After log transformation Revenue")
print("After log transformation Kurtosis ", df.revenue_log.kurtosis())
print("After log transformation Skewness:", df.revenue_log.skew())

<font size="5"> Relationship with revenue, budget and popularity</font><br/>

In [ ]:
df_total["budget"].describe()

* %25 of the budget is 0.This is absurd. Unknown maybe?Lets check

In [ ]:
df_total[df_total["budget"] == 0][["original_title"]].head()

![](https://i.imgur.com/UrVQXD9.png)

* These are unknown lets fill these with mean value.
* I dont think movie budget can be less than 50k$ unless it is amatuer.So replace these too.

In [ ]:
df.loc[df["budget"] <= (5*10**4), "budget"] = df_total.loc[df_total["budget"] > (5*10**4), "budget"].mean()
df_test.loc[df_test["budget"] <= (5*10**4), "budget"] = df_total.loc[df_total["budget"] >(5*10**4), "budget"].mean()
df_total.loc[df_total["budget"] <= (5*10**4), "budget"] = df_total.loc[df_total["budget"] > (5*10**4), "budget"].mean()

In [ ]:
var = "budget"
plt.figure(figsize=(20,10))
sns.scatterplot(x=df[var], y=df["revenue"], marker="x", color="black", label="revenue");

model = linear_model.HuberRegressor()
model.fit(np.array(df[var]).reshape(-1,1), df["revenue"])
rev_pred = model.predict(np.array(df[var]).reshape(-1, 1))

plt.plot(df[var], rev_pred, color = 'orange', label = 'linear')


params = {"eval_metric": "rmse", "silent":1}
dtrain = xg.DMatrix(np.array(df[var]).reshape(-1,1), np.array(df['revenue']))
xg_model = xg.train(params, dtrain)
dtest = xg.DMatrix(np.array(df[var]).reshape(-1,1))
rev_pred = xg_model.predict(dtest)

plt.scatter(df[var], rev_pred, color = 'blue', label = 'xgboost',marker="o")
plt.title("{} Impact".format((var[0].upper()+var[1:])))
plt.xlabel(var)
plt.ylabel('revenue')
plt.legend();


 <font size="5">Popularity</font><br/>

In [ ]:
plt.figure(figsize=(15,5))
sns.distplot(df_total["popularity"])
plt.title("Popularity");

In [ ]:
var = "popularity"
plt.figure(figsize=(20,10))
sns.scatterplot(x=df[var], y=df["revenue"], marker="x", color="black", label="revenue");

model = linear_model.HuberRegressor()
model.fit(np.array(df[var]).reshape(-1,1), df["revenue"])
rev_pred = model.predict(np.array(df[var]).reshape(-1, 1))

plt.plot(df[var], rev_pred, color = 'orange', label = 'linear')


params = {"eval_metric": "rmse", "silent":1}
dtrain = xg.DMatrix(np.array(df[var]).reshape(-1,1), np.array(df['revenue']))
xg_model = xg.train(params, dtrain)
dtest = xg.DMatrix(np.array(df[var]).reshape(-1,1))
rev_pred = xg_model.predict(dtest)

plt.scatter(df[var], rev_pred, color = 'blue', label = 'xgboost',marker="o")
plt.title("{} Impact".format((var[0].upper()+var[1:])))
plt.xlabel(var)
plt.ylabel('revenue')
plt.legend();


its almost like a vertical line idk how popularity calculated but certainly need to find out

 <font size="5">Runtime</font><br/>

In [ ]:
df.loc[2303, "runtime"] = 81 #Source imdb
df.runtime.fillna(0, inplace=True)
df_test.loc[3244, "runtime"] = 93 #Source imdb
df_test.loc[4490, "runtime"] = 95 #Source imdb
df_test.loc[4633, "runtime"] = 100 #Source filmaffinity
df_test.loc[4633, "runtime"] = 90 #Source imdb
df_test.loc[6818, "runtime"] = 90 #Source imdb
df_total.runtime.fillna(0, inplace=True)
df_total.loc[2303, "runtime"] = 81 #Source imdb
df_total.loc[3244, "runtime"] = 93 #Source imdb
df_total.loc[4490, "runtime"] = 95 #Source imdb
df_total.loc[4633, "runtime"] = 100 #Source filmaffinity
df_total.loc[4633, "runtime"] = 90 #Source imdb
df_total.loc[6818, "runtime"] = 90 #Source imdb

In [ ]:
plt.figure(figsize=(20,10))
sns.countplot(df_total["runtime"].apply(lambda x: (x//5)*5).dropna().astype(int))
plt.xticks(fontsize=15)
plt.title("Runtime",fontsize=20);
plt.tight_layout();

* 0 runtime is not possible.There must be a mistake.

In [ ]:
print("0 Runtime number of movies: ", df_total[df_total["runtime"] == 0]["runtime"].count())

In [ ]:
df_total[df_total["runtime"] <= 10][["original_title","runtime"]].head()

![](https://i.imgur.com/hyoVp39.png)

For now i will fill with mean values.But later i will fill these from internet.

In [ ]:
df.loc[df["runtime"] == 0, "runtime"] = df_total.loc[df_total["runtime"] > 0, "runtime"].mean()
df_test.loc[df_test["runtime"] == 0, "runtime"] = df_total.loc[df_total["runtime"] > 0, "runtime"].mean()
df_total.loc[df_total["runtime"] == 0, "runtime"] = df_total.loc[df_total["runtime"] > 0, "runtime"].mean()

In [ ]:
var = "runtime"
idx = np.array(df[~df[var].isnull()].index)
plt.figure(figsize=(20,10))
sns.scatterplot(x=df.loc[idx,var], y=df.loc[idx,"revenue"], marker="x", color="black", label="revenue");

model = linear_model.HuberRegressor()
model.fit(np.array(df.loc[idx,var]).reshape(-1,1), df.loc[idx,"revenue"])
rev_pred = model.predict(np.array(df.loc[idx,var]).reshape(-1, 1))

plt.plot(df.loc[idx,var], rev_pred, color = 'orange', label = 'linear')


params = {"eval_metric": "rmse", "silent":1}
dtrain = xg.DMatrix(np.array(df.loc[idx,var]).reshape(-1,1), np.array(df.loc[idx,"revenue"]))
xg_model = xg.train(params, dtrain)
dtest = xg.DMatrix(np.array(df.loc[idx,var]).reshape(-1,1))
rev_pred = xg_model.predict(dtest)

plt.scatter(df.loc[idx,var], rev_pred, color = 'blue', label = 'xgboost',marker="o")
plt.title("{} Impact".format((var[0].upper()+var[1:])))
plt.xlabel(var)
plt.ylabel('revenue')
plt.legend();


In [ ]:
df_total["budget_runtime_ratio"] = df_total["budget"]/df_total["runtime"]
df["budget_runtime_ratio"] = df["budget"]/df_total["runtime"]
df_test["budget_runtime_ratio"] = df_test["budget"]/df_total["runtime"]

- Relationship with categorical variables

 <font size="5">Belongs to Collection </font><br/>

In [ ]:
belongs_df = df[["belongs_to_collection","revenue"]]
belongs_df["belongs_to_collection_bool"] = True
belongs_df.loc[belongs_df["belongs_to_collection"].isnull(), "belongs_to_collection_bool"] = False
belongs_df['in_saga'] = np.nan
belongs_df.loc[belongs_df.belongs_to_collection_bool == True, "in_saga"] = belongs_df[belongs_df.belongs_to_collection_bool == True].revenue
belongs_df['no_saga'] = np.nan
belongs_df.loc[belongs_df.belongs_to_collection_bool == False, "no_saga"] = belongs_df[belongs_df.belongs_to_collection_bool == False].revenue
# belongs_df.belongs_to_collection[~df.belongs_to_collection.isnull()] = df.belongs_to_collection[~df.belongs_to_collection.isnull()].str.split("'name': '").str[1].str.split("', 'poster").str[0]
belongs_df.drop(columns=['revenue', 'belongs_to_collection', 'belongs_to_collection_bool'], inplace=True)
sns.boxplot(data=belongs_df, orient='h')
plt.xlabel('value')
plt.ylabel('Data')
plt.title('Box plot of Belongig to Collection')
plt.show()

In [ ]:
df["belongs_to_collection_bool"] = True
df_test["belongs_to_collection_bool"] = True
df_total["belongs_to_collection_bool"] = True
df.loc[df["belongs_to_collection"].isnull(), "belongs_to_collection_bool"] = False
df_test.loc[df_test["belongs_to_collection"].isnull(), "belongs_to_collection_bool"] = False
df_total.loc[df_total["belongs_to_collection"].isnull(), "belongs_to_collection_bool"] = False
df.loc[df["belongs_to_collection_bool"], "belongs_to_collection_len"] = df.loc[df["belongs_to_collection_bool"], "belongs_to_collection"].str.len().astype(int)
df_test.loc[df_test["belongs_to_collection_bool"], "belongs_to_collection_len"] = df_test.loc[df_test["belongs_to_collection_bool"], "belongs_to_collection"].str.len()
df_total.loc[df_total["belongs_to_collection_bool"], "belongs_to_collection_len"] = df_total.loc[df_total["belongs_to_collection_bool"], "belongs_to_collection"].str.len()


In [ ]:
plt.figure(figsize=(20,10))
plt.title("Collection Name Length", fontsize=25)
plt.yticks(fontsize=15)
plt.xticks(rotation=90, fontsize=12)
sns.countplot(df_total["belongs_to_collection_len"].dropna().astype(int));
plt.xlabel("Name Length",fontsize=15);

In [ ]:
df[["belongs_to_collection_len", "revenue"]].dropna().astype(int).groupby("belongs_to_collection_len").mean().plot(kind="bar", figsize=(20,10), label="mean revenue")
plt.title("Collection Name Length by Average Revenue", fontsize=25)
plt.yticks(fontsize=15)
plt.xticks(rotation=90, fontsize=12)
plt.xlabel("Collection Name Length",fontsize=15);

 <font size="5">Genres </font><br/>

In [ ]:
df.genres.fillna("", inplace=True)
df_test.genres.fillna("", inplace=True)
df_total.genres.fillna("", inplace=True)

In [ ]:
genres_df = df[['genres', 'revenue']]
genres_df['genres_test'] = genres_df['genres']
# genres_df['number_genres'] = genres_df.genres.str.count("'id'").fillna(0.0).astype(int)
genres_df.genres_test = genres_df.genres_test.str.strip('[]')
genres_df.genres_test[genres_df.genres_test.isnull()] = ''
genres_list = pd.Series(list(set(", ".join(genres_df.genres_test.unique().tolist()).split('}, ')))).str.split("'name': '").str[1].str.split("'").str[0].tolist()
for i, genre in enumerate(genres_list):
    genres_df[genre] = np.nan
    genres_df[genre][genres_df.genres_test.str.contains(genre)] = genres_df.revenue[genres_df.genres_test.str.contains(genre)]
    df[genre] = 0
    df.loc[df.genres.str.contains(genre),genre] = 1
    df_test[genre] = 0
    df_test.loc[df_test.genres.str.contains(genre),genre] = 1
    df_total[genre] = 0
    df_total.loc[df_total.genres.str.contains(genre),genre] = 1
genres_df.drop(columns=['revenue', 'genres', 'genres_test'], inplace=True)
# genres_boxplot = genres_df.boxplot(figsize=(15, 10), rot=90)
plt.figure(figsize=(20,12))
sns.boxplot(data=genres_df)
plt.xlabel('Data')
plt.ylabel('Value')
plt.title('Box plot of genres')
plt.tight_layout()
plt.show()


In [ ]:
df["genre_count"] = df.loc[:, genres_list].sum(axis=1)
df_test["genre_count"] = df_test.loc[:, genres_list].sum(axis=1)
df_total["genre_count"] = df_total.loc[:, genres_list].sum(axis=1)

 <font size="5">Original Language </font><br/> 

In [ ]:
var = "original_language"
plt.figure(figsize=(15,5))
plt.title("Original Language")
sns.countplot(df[var])
plt.xticks(rotation=45);

In [ ]:
df["org_lan_en"] = False
df.loc[df["original_language"] == "en", "org_lan_en"] = True

In [ ]:
var = "org_lan_en"
plt.figure(figsize=(15,7))
plt.title("Original Language")
g = sns.countplot(df[var])
ax = g.axes
for p in ax.patches:
    ax.annotate(f"{p.get_height() * 100 / df.shape[0]:.2f}%", (p.get_x() + p.get_width() / 2., p.get_height()),
         ha='center', va='center', fontsize=11, color='gray', rotation=0, xytext=(0, 10),
         textcoords='offset points');
plt.xticks(range(2),["Other","English"], fontsize=15);
plt.xlabel("Original Language English");

 <font size="5">Original Title </font><br/> 

In [ ]:
print("Unique Original Title",df["original_title"].nunique())
print(f"Average Title Length: {df['original_title'].str.len().mean():.0f}")
print(f"Average Word per Title: {df['original_title'].str.split().str.len().mean():.0f}")
print(f"Mode Word per Title: {df['original_title'].str.split().str.len().mode()[0]:.0f}")
stopwords = set(STOPWORDS)
wordcloud = WordCloud(max_font_size=50, max_words=100, background_color="white", stopwords=stopwords).generate(" ".join(list(df.original_title.values)))
plt.figure(figsize=(15,5))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off");

* No suprise "**American**"  is used a lot.Most of the production countries is US.
* "**Movie**"  is also in word cloud.I wonder why lets check it.

In [ ]:
print("Some of the Movie titles which contains 'American' \n","\n".join(list(df[df["original_title"].str.contains("American")].original_title.sample(3))))
print("Some of the Movie titles which contains 'Movie' \n","\n".join(list(df[df["original_title"].str.contains("Movie")].original_title.sample(8))))

* Rather than using as title.They sometimes used **Movie** word to distinguish from other products.(Such as tv series, games etc.)

In [ ]:
print("Unique Title",df["title"].nunique())
print(f"Average Title Length: {df['title'].str.len().mean():.0f}")
print(f"Average Word per Title: {df['title'].str.split().str.len().mean():.0f}")
print(f"Mode Word per Title: {df['title'].str.split().str.len().mode()[0]:.0f}")
stopwords = set(STOPWORDS)
wordcloud = WordCloud(max_font_size=50, max_words=100, background_color="white", stopwords=stopwords).generate(" ".join(list(df.title.values)))
plt.figure(figsize=(15,5))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off");

In [ ]:
df_test[df_test.title.isnull()][["original_title", "original_language"]]

Source: TMDB 

In [ ]:
df_test.loc[6629, "title"] = "Barefoot"
df_test.loc[5426, "title"] = "Would I Lie to You? 3"
df_test.loc[5399, "title"] = "The Life of Guskou Budori"
df_total.loc[6629, "title"] = "Barefoot"
df_total.loc[5426, "title"] = "Would I Lie to You? 3"
df_total.loc[5399, "title"] = "The Life of Guskou Budori"

In [ ]:
df["original_title_length"] = df["original_title"].str.len()
df_test["original_title_length"] = df_test["original_title"].str.len()
df_total["original_title_length"] = df_total["original_title"].str.len()
df["original_title_word_count"] = df["original_title"].str.split().str.len()
df_test["original_title_word_count"] = df_test["original_title"].str.split().str.len()
df_total["original_title_word_count"] = df_total["original_title"].str.split().str.len()

df["title_length"] = df["title"].str.len()
df_test["title_length"] = df_test["title"].str.len()
df_total["title_length"] = df_total["title"].str.len()
df["title_word_count"] = df["title"].str.split().str.len()
df_test["title_word_count"] = df_test["title"].str.split().str.len()
df_total["title_word_count"] = df_total["title"].str.split().str.len()

df_total["original_title_runtime_ratio"] = df_total["original_title_length"]/df_total["runtime"]
df["original_title_runtime_ratio"] = df["original_title_length"]/df_total["runtime"]
df_test["original_title_runtime_ratio"] = df_test["original_title_length"]/df_total["runtime"]

df_total["title_runtime_ratio"] = df_total["title_length"]/df_total["runtime"]
df["title_runtime_ratio"] = df["title_length"]/df_total["runtime"]
df_test["title_runtime_ratio"] = df_test["title_length"]/df_total["runtime"]

In [ ]:
title_df = df.loc[:,["original_title","original_title_length","original_title_word_count","revenue"]]
fig, ax = plt.subplots(1,2, figsize=(20,10))

title_df.groupby("original_title_length").mean().revenue.sort_values(ascending=False)[:20].plot(kind="bar", ax =ax[0])
ax[0].set_title("Title Length by Mean Revenue", fontsize=25);
ax[0].xaxis.set_tick_params(labelsize=15, rotation = 0)
ax[0].yaxis.set_tick_params(labelsize=15)
ax[0].set_xlabel("Title Length", fontsize=20);
title_df.groupby("original_title_length").count().revenue.sort_values(ascending=False)[:20].plot(kind="bar", ax=ax[1])
ax[1].set_title("Title Length Count", fontsize=25);
ax[1].xaxis.set_tick_params(labelsize=15, rotation = 0);
ax[1].yaxis.set_tick_params(labelsize=15);
ax[1].set_xlabel("Title Length", fontsize=20);

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(20,10))

title_df.groupby("original_title_word_count").mean().revenue.sort_values(ascending=False)[:20].plot(kind="bar", ax =ax[0])
ax[0].set_title("Number of Words in Title by Mean Revenue", fontsize=25);
ax[0].xaxis.set_tick_params(labelsize=15,rotation=0)
ax[0].yaxis.set_tick_params(labelsize=15)
ax[0].set_xlabel("Number of Words", fontsize=20);
title_df.groupby("original_title_word_count").count().revenue.sort_values(ascending=False)[:20].plot(kind="bar", ax=ax[1])
ax[1].set_title("Number of Words Count", fontsize=25);
ax[1].xaxis.set_tick_params(labelsize=15,rotation=0);
ax[1].yaxis.set_tick_params(labelsize=15);
ax[1].set_xlabel("Number of Words", fontsize=20);

 <font size="5">Overview </font><br/> 

In [ ]:
df_total[df_total.overview.isnull()][["original_title","overview","train_bool"]] 

In [ ]:
overviews_missing = ["Paolo needs to reach the castle of Alberto Caccia, where he was invited to spend Christmas with his family and Margherita, the ninth month of pregnancy. With them there will also be blessed, daughter of Alberto and childhood friend of Margherita, who was also pregnant. Among various mishaps and blunders, Paolo will combine another of his, coming to make everyone believe, because of a misunderstanding, that Alberto's died for his fault.",
            "Eight men, eight stories about the morning after party...",
            "Seven days before his wedding, a man must contend with a series of never-ending disasters.",
            "",
            "",
            "A young boy's dreams of glory and war turn into a bitter nightmare as his father's kingdom is overrun by an invading army. Lost and alone in the woods, he finds an ancient sword that promises him the ability to claim his vengeance.",
            "",
            "Cetto and its city council were arrested, but in prison the conspiratorial former mayor makes no earning name as the gratitude of the powerful secretary. These decide to replace some MPs mysteriously killed precisely with Cetto and two other characters: Rodolfo Favaretto and Frengo Stop",
             "",
            "Father Alexander is trying to maintain peaceful life for his church amidst the Nazi occupation during WWII.",
            "",
            "",
            "",
            "Drona (Nitin) is the only son of Mukhesh Rushi and Sita. Drona’s father Mukesh Rushi is an honest and dedicated Police officer. Drona likes his mother Meenakshi (Sita) so much and love her as his own life. When he was a child Drona steals his father’s revolver to scare his friends. On knowing this his father scolds him, consequently Drona leaves his home and gets lost. After 10 years, he comes back to his parents with the name Chandu. A Neighboring girl Indu (Priyamani) likes Drona/Chandu a lot. On a fateful day, she finds out that Chandu is none other than the long lost boy, Drona",
            "",
            "An elderly, yet young-at-heart man, moves in with his grandson, and both their lives turn upside-down.",
            "Because of their children's bad grades, some parents want the teacher Mrs. Müller gone. They meet with Mrs. Müller and try to convince her to leave the class.",
            "",
            "",
            "",
            "",
             ""]

In [ ]:
df_total.loc[df_total.overview.isnull(),"overview"] = np.array(overviews_missing)
df.loc[df.overview.isnull(),"overview"] = np.array(overviews_missing[:df.overview.isnull().sum()])
df_test.loc[df_test.overview.isnull(),"overview"] = np.array(overviews_missing[len(overviews_missing)-df_test.overview.isnull().sum():])

In [ ]:
df_total[["original_title","overview","train_bool"]].dropna().loc[df_total.overview.dropna().str.contains("No overview")]

In [ ]:
df_total.loc[5598, "overview"] = "Tells the story about Karan, a young, spirited, sporty boy who lives life off the rules and Sana, a young petite, shy girl who lives life by the rules. This develops into a magical love story. Karan's uncle, Dr. Yatinder Khanna's much developed time machine finally works. Sana expresses her wish to travel ahead in time to Mumbai in the future. We land in Mumbai 2050 in the time machine... along with Uncle Yatinder, Rahul, and Thea (Sana's younger brother and sister). Flying cars, 200-story buildings, robots and sky rails have changed the very face of Mumbai. Through a series of twists and turns, the time traveler Karan finds himself separating from his love. QT, a friendly femme robot, and Boo, a small robotic teddy bear play their distinct roles in bringing their love story together but soon find themselves under the threat of the demi-god of the future, Dr. Hoshi. Will Karan be able to overcome the threat of Dr. Hoshi and win his love and travel back in time?"
df_total.loc[5675, "overview"] = "A realtor fires one of his co-workers and out of spite, the co-worker and his wife does black magic on his daughter. Being an atheist, he shuns his mother and wife's theories on the supernatural and relies on a doctor for his daughter's treatment."
df_test.loc[5598, "overview"] = "Tells the story about Karan, a young, spirited, sporty boy who lives life off the rules and Sana, a young petite, shy girl who lives life by the rules. This develops into a magical love story. Karan's uncle, Dr. Yatinder Khanna's much developed time machine finally works. Sana expresses her wish to travel ahead in time to Mumbai in the future. We land in Mumbai 2050 in the time machine... along with Uncle Yatinder, Rahul, and Thea (Sana's younger brother and sister). Flying cars, 200-story buildings, robots and sky rails have changed the very face of Mumbai. Through a series of twists and turns, the time traveler Karan finds himself separating from his love. QT, a friendly femme robot, and Boo, a small robotic teddy bear play their distinct roles in bringing their love story together but soon find themselves under the threat of the demi-god of the future, Dr. Hoshi. Will Karan be able to overcome the threat of Dr. Hoshi and win his love and travel back in time?"
df_test.loc[5675, "overview"] = "A realtor fires one of his co-workers and out of spite, the co-worker and his wife does black magic on his daughter. Being an atheist, he shuns his mother and wife's theories on the supernatural and relies on a doctor for his daughter's treatment."

In [ ]:
df_total["overview_length"] = df_total["overview"].str.len()
df["overview_length"] = df["overview"].str.len()
df_test["overview_length"] = df_test["overview"].str.len()
df_total["overview_runtime_ratio"] = df_total["overview_length"]/df_total["runtime"]
df["overview_runtime_ratio"] = df["overview_length"]/df_total["runtime"]
df_test["overview_runtime_ratio"] = df_test["overview_length"]/df_total["runtime"]

df_total["overview_genre_ratio"] = df_total["overview_length"]/df_total["genre_count"]
df["overview_genre_ratio"] = df["overview_length"]/df_total["genre_count"]
df_test["overview_genre_ratio"] = df_test["overview_length"]/df_total["genre_count"]

In [ ]:
tfidf = TfidfVectorizer(stop_words="english", analyzer="word", max_features=100,max_df=30, min_df=10)
train_tfidf = pd.DataFrame(tfidf.fit_transform(df.overview).toarray(), columns=["tfidf_"+str(i) for i in range(100)], index=df.index)
test_tfidf = pd.DataFrame(tfidf.transform(df_test.overview).toarray(), columns=["tfidf_"+str(i) for i in range(100)], index=df_test.index)
total_tfidf = pd.DataFrame(tfidf.transform(df_total.overview).toarray(), columns=["tfidf_"+str(i) for i in range(100)], index=df_total.index)
print("overview:")
print(tfidf.get_feature_names())

In [ ]:
df = pd.concat([df, train_tfidf], axis=1)
df_test = pd.concat([df_test, test_tfidf], axis=1)
df_total = pd.concat([df_total, total_tfidf], axis=1)

<font size="5">Production Companies  </font><br/> 

In [ ]:
df.production_companies.fillna("[]", inplace=True)
df_test.production_companies.fillna("[]", inplace=True)
df_total.production_companies.fillna("[]", inplace=True)


In [ ]:
df.loc[:,"production_companies"] = df.production_companies.apply(lambda x: [dicts["name"].strip() for dicts in ast.literal_eval(x)])
df_test.loc[:,"production_companies"] = df_test.production_companies.apply(lambda x: [dicts["name"].strip() for dicts in ast.literal_eval(x)])
df_total.loc[:,"production_companies"] = df_total.production_companies.apply(lambda x: [dicts["name"].strip() for dicts in ast.literal_eval(x)])

In [ ]:
def sum_companies(company):
    if company == "":
        return
    companies[company] += 1

companies = Counter()
df_total.production_companies.apply(lambda x: [sum_companies(company.strip()) for company in x]);

![](https://i.imgur.com/FBKmSAY.png)
![](https://i.imgur.com/KPBxEuO.png)

In [ ]:
pd.DataFrame(companies.most_common(20)).set_index(0).sort_values(1,ascending=True).plot(kind="barh",figsize=(15,10))
plt.title("Production Companies")
plt.ylabel("Company")
plt.xlabel("Value");
plt.legend().remove();

In [ ]:
for company, value in companies.most_common(20):
    df[company] = df.production_companies.apply(lambda x: 1 if company in x else 0)
    df_test[company] = df_test.production_companies.apply(lambda x: 1 if company in x else 0)
    df_total[company] = df_total.production_companies.apply(lambda x: 1 if company in x else 0)

In [ ]:
def sum_companies_revenue(companies, revenue):
    for company in companies:
        company = company.strip()
        if company in companies_revenue:
            companies_revenue[company].append(revenue)
        else:

            companies_revenue[company] = [revenue]

companies_revenue = Counter()
df[["production_companies", "revenue"]].apply(lambda x: sum_companies_revenue(x["production_companies"], x["revenue"]), axis=1)
for company, revenue in companies_revenue.items():
    companies_revenue[company] = np.max(np.array(revenue))

In [ ]:
pd.DataFrame(companies_revenue.most_common(20)).set_index(0).sort_values(1,ascending=True).plot(kind="barh",figsize=(15,10))
plt.title("Production Companies")
plt.ylabel("Company")
plt.xlabel("Max Revenue");
plt.legend().remove();

In [ ]:
df["production_companies_length"] = df.production_companies.apply(lambda x : len(x))
df_test["production_companies_length"] = df_test.production_companies.apply(lambda x : len(x))
df_total["production_companies_length"] = df_total.production_companies.apply(lambda x : len(x))

<font size="5">Production Countries  </font><br/> 

In [ ]:
df.production_countries.fillna("[]", inplace=True)
df_test.production_countries.fillna("[]", inplace=True)
df_total.production_countries.fillna("[]", inplace=True)

In [ ]:
df.loc[:,"production_countries"] = df.production_countries.apply(lambda x: [dicts["iso_3166_1"].strip() for dicts in ast.literal_eval(x)])
df_test.loc[:,"production_countries"] = df_test.production_countries.apply(lambda x: [dicts["iso_3166_1"].strip() for dicts in ast.literal_eval(x)])
df_total.loc[:,"production_countries"] = df_total.production_countries.apply(lambda x: [dicts["iso_3166_1"].strip() for dicts in ast.literal_eval(x)])

In [ ]:
df["production_countries_length"] = df.production_countries.apply(lambda x : len(x))
df_test["production_countries_length"] = df_test.production_countries.apply(lambda x : len(x))
df_total["production_countries_length"] = df_total.production_countries.apply(lambda x : len(x))

In [ ]:
def sum_countries(country):
    if country == "":
        return
    countries[country] += 1

countries = Counter()
df_total.production_countries.apply(lambda x: [sum_countries(country.strip()) for country in x]);


In [ ]:
pd.DataFrame(countries.most_common(20)).set_index(0).sort_values(1,ascending=True).plot(kind="barh",figsize=(20,10))
plt.title("Production Countries")
plt.ylabel("Country")
plt.xlabel("Value");
plt.legend().remove();

In [ ]:
for country, value in countries.most_common(10):
    df[country] = df.production_countries.apply(lambda x: 1 if country in x else 0)
    df_test[country] = df_test.production_countries.apply(lambda x: 1 if country in x else 0)
    df_total[country] = df_total.production_countries.apply(lambda x: 1 if country in x else 0)

In [ ]:
def sum_countries_revenue(countries, revenue):
    for country in countries:
        country = country.strip()
        if country in countries_revenue:
            countries_revenue[country].append(revenue)
        else:

            countries_revenue[country] = [revenue]

countries_revenue = Counter()
df[["production_countries", "revenue"]].apply(lambda x: sum_countries_revenue(x["production_countries"], x["revenue"]), axis=1)
for country, revenue in countries_revenue.items():
    countries_revenue[country] = np.max(np.array(revenue))
    

In [ ]:
pd.DataFrame(countries_revenue.most_common(20)).set_index(0).sort_values(1,ascending=True).plot(kind="barh",figsize=(20,10))
plt.title("Production Countries")
plt.ylabel("Country")
plt.xlabel("Max Revenue");
plt.legend().remove();

<font size="5">Release Date  </font><br/> 

In [ ]:
df_test.loc[3829, "release_date"] = "05/01/00"
df_total.loc[3829, "release_date"] = "05/01/00"
#Jails, Hospitals & Hip-Hop:  Source imdb

In [ ]:
df.loc[~df.release_date.isnull(),"date"] = df[~df.release_date.isnull()].release_date.apply(lambda x: np.datetime64("20"+x.split("/")[2]+"-"+x.split("/")[0].zfill(2)+"-"+x.split("/")[1].zfill(2)) if int(x.split("/")[2]) <= 18 else np.datetime64("19"+x.split("/")[2]+"-"+x.split("/")[0].zfill(2)+"-"+x.split("/")[1].zfill(2)))
df_test.loc[~df_test.release_date.isnull(),"date"] = df_test[~df_test.release_date.isnull()].release_date.apply(lambda x: np.datetime64("20"+x.split("/")[2]+"-"+x.split("/")[0].zfill(2)+"-"+x.split("/")[1].zfill(2)) if int(x.split("/")[2]) <= 18 else np.datetime64("19"+x.split("/")[2]+"-"+x.split("/")[0].zfill(2)+"-"+x.split("/")[1].zfill(2)))
df_total.loc[~df_total.release_date.isnull(),"date"] = df_total[~df_total.release_date.isnull()].release_date.apply(lambda x: np.datetime64("20"+x.split("/")[2]+"-"+x.split("/")[0].zfill(2)+"-"+x.split("/")[1].zfill(2)) if int(x.split("/")[2]) <= 18 else np.datetime64("19"+x.split("/")[2]+"-"+x.split("/")[0].zfill(2)+"-"+x.split("/")[1].zfill(2)))

In [ ]:
df["year"] = df.date.dt.year
df["month"] = df.date.dt.month
df["week"] = df.date.dt.week
df["day"] = df.date.dt.day
df_test["year"] = df_test.date.dt.year
df_test["month"] = df_test.date.dt.month
df_test["week"] = df_test.date.dt.week
df_test["day"] = df_test.date.dt.day
df_total["year"] = df_total.date.dt.year
df_total["month"] = df_total.date.dt.month
df_total["week"] = df_total.date.dt.week
df_total["day"] = df_total.date.dt.day


In [ ]:
plt.figure(figsize=(20,10))
plt.title("Films")
g = sns.countplot(df_total.year)
plt.xticks(rotation=90);
plt.ylabel("Number of Films");
ax = g.axes
last = 0
for p in ax.patches:
    p.set_color("green" if p.get_height() > last else "red")
    last = p.get_height()
green_patch = mpatches.Patch(label="increase compared to last year",color="green")
red_patch = mpatches.Patch(label="decrease compared to last year",color="red")
plt.legend(handles=[green_patch, red_patch]);

In [ ]:
plt.figure(figsize=(20,5))
plt.title("Movie Budgets & Revenue Maximum")
df_total.groupby("year").revenue.max().plot(color="green", marker="o", logy=True)
df_total.groupby("year").budget.max().plot(color="blue", marker="o", logy=True)
plt.legend()
plt.xticks(rotation=90);

In [ ]:
plt.figure(figsize=(20,5))
plt.title("Movie Budgets & Revenue Mean")
df_total.groupby("year").revenue.mean().plot(color="green", marker="o", logy=True)
df_total.groupby("year").budget.mean().plot(color="blue", marker="o", logy=True)
plt.legend()
plt.xticks(rotation=90);

<font size="5">Spoken languages </font><br/> 

In [ ]:
df.spoken_languages.fillna("[]", inplace=True)
df_test.spoken_languages.fillna("[]", inplace=True)
df_total.spoken_languages.fillna("[]", inplace=True)

In [ ]:
df.loc[:,"spoken_languages"] = df.spoken_languages.apply(lambda x: [dicts["iso_639_1"].strip() for dicts in ast.literal_eval(x)])
df_test.loc[:,"spoken_languages"] = df_test.spoken_languages.apply(lambda x: [dicts["iso_639_1"].strip() for dicts in ast.literal_eval(x)])
df_total.loc[:,"spoken_languages"] = df_total.spoken_languages.apply(lambda x: [dicts["iso_639_1"].strip() for dicts in ast.literal_eval(x)])

In [ ]:
df["spoken_languages_length"] = df.spoken_languages.apply(lambda x : len(x))
df_test["spoken_languages_length"] = df_test.spoken_languages.apply(lambda x : len(x))
df_total["spoken_languages_length"] = df_total.spoken_languages.apply(lambda x : len(x))

In [ ]:
def sum_languages(language):
    if language == "":
        return
    languages[language] += 1

languages = Counter()
df_total.spoken_languages.apply(lambda x: [sum_languages(language.strip()) for language in x]);

In [ ]:
pd.DataFrame(languages.most_common(20)).set_index(0).sort_values(1,ascending=True).plot(kind="barh",figsize=(20,10))
plt.title("Spoken Languages")
plt.ylabel("Language short")
plt.xlabel("Value");
plt.legend().remove();

In [ ]:
for language, value in languages.most_common(10):
    df[language] = df.spoken_languages.apply(lambda x: 1 if language in x else 0)
    df_test[language] = df_test.spoken_languages.apply(lambda x: 1 if language in x else 0)
    df_total[language] = df_total.spoken_languages.apply(lambda x: 1 if language in x else 0)

<font size="5">Status</font><br/> 

In [ ]:
df_test[df_test.status.isnull()].original_title

In [ ]:
df_test.loc[df_test.status.isnull(), "status"] = "Released"
df_total.loc[df_total.status.isnull(), "status"] = "Released"

In [ ]:
g = sns.countplot(df_total.status)
ax = g.axes
for p in ax.patches:
    ax.annotate(f"{(p.get_height()*100/df_total.shape[0]):.2f}", (p.get_x() + p.get_width() /2., p.get_height()),
               ha ="center", va="center", fontsize=11, color="gray", rotation=0, xytext=(0, 10), textcoords="offset points")


In [ ]:
df["status"] = df["status"].apply(lambda x: 1 if x == "Released" else 0)
df_test["status"] = df_test["status"].apply(lambda x: 1 if x == "Released" else 0)
df_total["status"] = df_total["status"].apply(lambda x: 1 if x == "Released" else 0)

Almost all of the status data is released

<font size="5">Tagline</font><br/> 

In [ ]:
df.tagline.fillna("", inplace=True)
df_test.tagline.fillna("", inplace=True)
df_total.tagline.fillna("", inplace=True)

In [ ]:
wordcloud = WordCloud(max_font_size=50,max_words=100, stopwords=stopwords, background_color ="white").generate(" ".join(list(df_total.tagline.values)))
plt.figure(figsize=(20,5))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off");


In [ ]:
tfidf = TfidfVectorizer(stop_words="english", analyzer="word", max_features=100,max_df=30, min_df=10)
train_tfidf = pd.DataFrame(tfidf.fit_transform(df.tagline).toarray(), columns=["tfidf_tagline_"+str(i) for i in range(100)], index=df.index)
test_tfidf = pd.DataFrame(tfidf.transform(df_test.tagline).toarray(), columns=["tfidf_tagline_"+str(i) for i in range(100)], index=df_test.index)
total_tfidf = pd.DataFrame(tfidf.transform(df_total.tagline).toarray(), columns=["tfidf_tagline_"+str(i) for i in range(100)], index=df_total.index)
print("Tagline:")
print(tfidf.get_feature_names())

In [ ]:
df = pd.concat([df, train_tfidf], axis=1)
df_test = pd.concat([df_test, test_tfidf], axis=1)
df_total = pd.concat([df_total, total_tfidf], axis=1)

In [ ]:
df["tagline_length"] = df.tagline.str.len()
df_test["tagline_length"] = df_test.tagline.str.len()
df_total["tagline_length"] = df_total.tagline.str.len()
df["tagline_no_of_words"] = df.tagline.str.split().apply(lambda x: len(x))
df_test["tagline_no_of_words"] = df_test.tagline.str.split().apply(lambda x: len(x))
df_total["tagline_no_of_words"] = df_total.tagline.str.split().apply(lambda x: len(x))

In [ ]:
plt.figure(figsize=(20,10))
sns.distplot(df_total[df_total.tagline_length > 0].tagline_length, color="blue")
sns.distplot(df_total[df_total.tagline_no_of_words > 0].tagline_no_of_words, color="red")
plt.title("Tagline Length and Number of Words")
plt.xlabel("Length");

For now i just filled with nothing.I will try another thing later.

 <font size="5">Keywords</font><br/> 

In [ ]:
df.Keywords.fillna("[]", inplace=True)
df_test.Keywords.fillna("[]", inplace=True)
df_total.Keywords.fillna("[]", inplace=True)

df.loc[:,"Keywords"] = df.Keywords.apply(lambda x: [dicts["name"].strip() for dicts in ast.literal_eval(x)])
df_test.loc[:,"Keywords"] = df_test.Keywords.apply(lambda x: [dicts["name"].strip() for dicts in ast.literal_eval(x)])
df_total.loc[:,"Keywords"] = df_total.Keywords.apply(lambda x: [dicts["name"].strip() for dicts in ast.literal_eval(x)])

df.loc[:,"Keywords_length"] = df.Keywords.apply(lambda x: len(x))
df_test.loc[:,"Keywords_length"] = df_test.Keywords.apply(lambda x: len(x))
df_total.loc[:,"Keywords_length"] = df_total.Keywords.apply(lambda x: len(x))

In [ ]:
def sum_keywords(keyword):
    if keyword == "":
        return
    keywords[keyword] += 1

keywords = Counter()

df_total.Keywords.apply(lambda x: [sum_keywords(keyword.strip()) for keyword in x]);
pd.DataFrame(keywords.most_common(30)).set_index(0).sort_values(1,ascending=True).plot(kind="barh",figsize=(20,10))
plt.title("Keywords")
plt.ylabel("keyword")
plt.xlabel("Value");
plt.legend().remove();

In [ ]:
for keyword, value in keywords.most_common(30):
    df[keyword.replace(" ", "_")] = df.Keywords.apply(lambda x: 1 if keyword in x else 0)
    df_test[keyword.replace(" ", "_")] = df_test.Keywords.apply(lambda x: 1 if keyword in x else 0)
    df_total[keyword.replace(" ", "_")] = df_total.Keywords.apply(lambda x: 1 if keyword in x else 0)

 <font size="5">Cast</font><br/> 

* Fill missing values from tmdb webpage.

In [ ]:
missing_cast = [[{'name': 'Gal Gadot', 'character': 'Diana Prince / Wonder Woman', 'gender': 0}, {'name': 'Chris Pine', 'character': 'Steve Trevor', 'gender': 1}, {'name': 'Connie Nielsen', 'character': 'Queen Hippolyta', 'gender': 0}, {'name': 'Robin Wright', 'character': 'Antiope', 'gender': 0}, {'name': 'Danny Huston', 'character': 'Erich Ludendorff', 'gender': 1}, {'name': 'David Thewlis', 'character': 'Sir Patrick Morgan / Ares', 'gender': 1}, {'name': 'Saïd Taghmaoui', 'character': 'Sameer', 'gender': 2}, {'name': 'Ewen Bremner', 'character': 'Charlie', 'gender': 1}, {'name': 'Eugene Brave Rock', 'character': 'Chief Napi', 'gender': 1}, {'name': 'Lucy Davis', 'character': 'Etta Candy', 'gender': 0}, {'name': 'Elena Anaya', 'character': 'Isabel Maru / Doctor Poison', 'gender': 0}, {'name': 'Lilly Aspell', 'character': 'Young Diana (8)', 'gender': 0}, {'name': 'Lisa Loven Kongsli', 'character': 'Menalippe', 'gender': 0}, {'name': 'Ann Wolfe', 'character': 'Artemis', 'gender': 2}, {'name': 'Ann Ogbomo', 'character': 'Phillipus', 'gender': 2}, {'name': 'Emily Carey', 'character': 'Diana (12)', 'gender': 0}, {'name': 'James Cosmo', 'character': 'FIeld Marshall Haig', 'gender': 1}, {'name': 'Wolf Kahler', 'character': 'German Commander', 'gender': 1}, {'name': 'Alexander Mercury', 'character': 'German Lieutenant', 'gender': 1}, {'name': 'Martin Bishop', 'character': 'Kaiser Wilhelm II', 'gender': 2}, {'name': 'Flora Nicholson', 'character': 'Marie', 'gender': 2}, {'name': 'Pat Abernethy', 'character': 'Photographer', 'gender': 2}, {'name': 'Freddy Elletson', 'character': 'Trench Sentry', 'gender': 2}, {'name': 'Sammy Hayman', 'character': 'Trench Sentry', 'gender': 1}, {'name': 'Michael Tantrum', 'character': 'Trench Sentry', 'gender': 2}, {'name': 'Philippe Spall', 'character': 'Trench Officer - Belgian', 'gender': 2}, {'name': 'Edward Wolstenholme', 'character': 'Trench Officer - British', 'gender': 2}, {'name': 'Ian Hughes', 'character': 'German Soldier', 'gender': 1}, {'name': 'Marko Leht', 'character': 'German Soldier in Boat', 'gender': 1}, {'name': 'Steffan Rhodri', 'character': 'Colonel Darnell', 'gender': 1}, {'name': 'Andrew Byron', 'character': 'Weary Captain (Shot)', 'gender': 1}, {'name': 'Dominic Kinnaird', 'character': 'Suited Man', 'gender': 2}, {'name': 'Rachel Pickup', 'character': 'Fausta Grables', 'gender': 2}, {'name': 'Ulli Ackermann', 'character': 'Guard at Invite Table', 'gender': 2}, {'name': 'Frank Allen Forbes', 'character': 'Turkish Slave', 'gender': 2}, {'name': 'Peter Stark', 'character': 'Soldier at Gala', 'gender': 2}, {'name': 'Rainer Bock', 'character': 'Von Hindenberg', 'gender': 1}, {'name': 'Josh Bromley', 'character': 'Young Soldier (Station)', 'gender': 2}, {'name': 'Jennie Eggleton', 'character': 'Young Wife (Station)', 'gender': 2}, {'name': 'Eva Dabrowski', 'character': 'Nina', 'gender': 2}, {'name': 'Harvey James', 'character': 'German Pilot', 'gender': 2}, {'name': 'George Johnston', 'character': 'German Pilot', 'gender': 2}, {'name': 'Danielle Lewis', 'character': 'Senator Timandra', 'gender': 0}, {'name': 'Florence Kasumba', 'character': 'Senator Acantha', 'gender': 0}, {'name': 'Eleanor Matsuura', 'character': 'Epione', 'gender': 0}, {'name': 'Josette Simon', 'character': 'Mnemosyne', 'gender': 0}, {'name': 'Doutzen Kroes', 'character': 'Venelia', 'gender': 0}, {'name': 'Hayley Warnes', 'character': 'Aella', 'gender': 2}, {'name': 'Caitlin Burles', 'character': "Queen's Guard", 'gender': 2}, {'name': 'Jemma Moore', 'character': 'Queens Guard', 'gender': 0}, {'name': 'Samantha Jo', 'character': 'Euboea', 'gender': 0}, {'name': 'Brooke Ence', 'character': 'Penthiselea', 'gender': 2}, {'name': 'Madeleine Vall', 'character': 'Egeria (as Madeleine Vall Beijner)', 'gender': 2}, {'name': 'Hari James', 'character': 'Trigona', 'gender': 2}, {'name': 'Jacqui-Lee Pryce', 'character': 'Niobe', 'gender': 2}, {'name': 'Betty Adewole', 'character': 'Amazon Army', 'gender': 2}, {'name': 'Caroline Maria Winberg', 'character': 'Amazon Army (as Caroline Winberg)', 'gender': 2}, {'name': 'Lizzie Bowden', 'character': 'Amazon Army', 'gender': 2}, {'name': 'Kattreya Scheurer-Smith', 'character': 'Amazon Army', 'gender': 2}, {'name': 'Rekha Luther', 'character': 'Amazon Townsfolk', 'gender': 0}, {'name': 'Thaina Oliveira', 'character': 'Amazon Townfolk', 'gender': 2}, {'name': 'Ooooota Adepo', 'character': 'Amazon Townsfolk', 'gender': 2}, {'name': 'Zinnia Kumar', 'character': 'Amazon Townsfolk', 'gender': 2}, {'name': 'Toma McDonagh', 'character': 'Amazon Townfolk', 'gender': 2}, {'name': 'Amber Doyle', 'character': 'Mother', 'gender': 2}, {'name': 'Freddy Carter', 'character': 'Soldier', 'gender': 2}, {'name': 'Fred Fergus', 'character': 'Soldier', 'gender': 2}, {'name': 'Tim Pritchett', 'character': 'Wounded Soldier', 'gender': 2}, {'name': 'Gana Bayarsaikhan', 'character': 'Throne Room Amazon', 'gender': 2}, {'name': 'Camilla Roholm', 'character': 'Throne Room Amazon', 'gender': 2}, {'name': 'Stephanie Haymes-Roven', 'character': 'Amazon Cavalry General', 'gender': 2}, {'name': 'Nia Burke', 'character': 'Veld Singer', 'gender': 2}, {'name': 'Dee Lewis Clay', 'character': 'Singer', 'gender': 0}, {'name': 'Tori Letzler', 'character': 'Singer', 'gender': 2}], [{'name': 'Dennis Quaid', 'character': 'Jack Hall', 'gender': 1}, {'name': 'Jake Gyllenhaal', 'character': 'Sam Hall', 'gender': 1}, {'name': 'Emmy Rossum', 'character': 'Laura Chapman', 'gender': 0}, {'name': 'Dash Mihok', 'character': 'Jason Evans', 'gender': 1}, {'name': 'Jay O. Sanders', 'character': 'Frank Harris', 'gender': 1}, {'name': 'Sela Ward', 'character': 'Dr. Lucy Hall', 'gender': 0}, {'name': 'Austin Nichols', 'character': 'J.D.', 'gender': 1}, {'name': 'Arjay Smith', 'character': 'Brian Parks', 'gender': 1}, {'name': 'Tamlyn Tomita', 'character': 'Janet Tokada', 'gender': 0}, {'name': 'Sasha Roiz', 'character': 'Parker', 'gender': 1}, {'name': 'Ian Holm', 'character': 'Terry Rapson', 'gender': 1}, {'name': 'Richard McMillan', 'character': 'Dennis', 'gender': 1}, {'name': 'Nassim Sharara', 'character': 'Saudi Delegate', 'gender': 2}, {'name': 'Carl Alacchi', 'character': 'Venezuelan Delegate', 'gender': 1}, {'name': 'Kenneth Welsh', 'character': 'Vice President Becker', 'gender': 1}, {'name': "Michel 'Gish' Abou-Samah", 'character': 'Saudi Translator', 'gender': 2}, {'name': 'Kenneth Moskow', 'character': 'Bob', 'gender': 2}, {'name': 'Glenn Plummer', 'character': 'Luther', 'gender': 1}, {'name': 'Adrian Lester', 'character': 'Simon', 'gender': 1}, {'name': 'Nestor Serrano', 'character': 'Gomez', 'gender': 1}, {'name': 'John Maclaren', 'character': 'Veteran Scientist', 'gender': 1}, {'name': 'Richard Zeman', 'character': 'Flight Director', 'gender': 2}, {'name': 'Perry King', 'character': 'President Blake', 'gender': 2}, {'name': 'Mimi Kuzyk', 'character': 'Secretary of State', 'gender': 0}, {'name': 'Vitali Makarov', 'character': 'Yuri, Russian Astronaut', 'gender': 2}, {'name': 'Russell Yuen', 'character': 'Hideki, Japanese Astronaut', 'gender': 1}, {'name': 'Christopher Britton', 'character': 'Vorsteen', 'gender': 1}, {'name': 'Christian Tessier', 'character': 'Aaron', 'gender': 1}, {'name': 'Rick Hoffman', 'character': 'NY Businessman on Bus', 'gender': 1}, {'name': 'Alan Fawcett', 'character': 'Commander Daniels', 'gender': 1}, {'name': 'Sheila McCarthy', 'character': 'Judith', 'gender': 0}, {'name': 'Amy Sloan', 'character': 'Elsa', 'gender': 0}, {'name': 'Karen Glave', 'character': 'Maria', 'gender': 0}, {'name': 'Joe Cobden', 'character': 'Zack', 'gender': 1}, {'name': 'Caroline Keenan', 'character': 'Tina', 'gender': 0}], [{'name': 'Christian Bale', 'character': 'Bruce Wayne / Batman', 'gender': 1}, {'name': 'Michael Caine', 'character': 'Alfred Pennyworth', 'gender': 1}, {'name': 'Gary Oldman', 'character': 'James Gordon', 'gender': 1}, {'name': 'Anne Hathaway', 'character': 'Selina Kyle / Catwoman', 'gender': 0}, {'name': 'Tom Hardy', 'character': 'Bane', 'gender': 1}, {'name': 'Marion Cotillard', 'character': 'Miranda Tate / Talia al Ghul', 'gender': 0}, {'name': 'Joseph Gordon-Levitt', 'character': 'John Blake', 'gender': 1}, {'name': 'Morgan Freeman', 'character': 'Lucius Fox', 'gender': 1}, {'name': 'Matthew Modine', 'character': 'Peter Foley', 'gender': 1}, {'name': 'Alon Aboutboul', 'character': 'Leonid Pavel', 'gender': 1}, {'name': 'Ben Mendelsohn', 'character': 'John Daggett', 'gender': 1}, {'name': 'Burn Gorman', 'character': 'Phillip Stryver', 'gender': 2}, {'name': 'Daniel Sunjata', 'character': 'Captain Jones', 'gender': 1}, {'name': 'Cillian Murphy', 'character': 'Jonathan Crane / Scarecrow', 'gender': 1}, {'name': 'Liam Neeson', 'character': "Ra's al Ghul", 'gender': 1}, {'name': 'Juno Temple', 'character': 'Jen', 'gender': 0}, {'name': 'Nestor Carbonell', 'character': 'Anthony Garcia', 'gender': 1}, {'name': 'Josh Stewart', 'character': 'Barsad', 'gender': 1}, {'name': 'Aidan Gillen', 'character': 'CIA Op', 'gender': 1}, {'name': 'Tom Conti', 'character': 'Prisoner', 'gender': 2}, {'name': 'Joey King', 'character': 'Young Talia al Ghul', 'gender': 0}, {'name': 'Josh Pence', 'character': "Young Ra's al Ghul", 'gender': 1}, {'name': 'John Nolan', 'character': 'Douglas Fredericks', 'gender': 1}, {'name': 'Warren Brown', 'character': 'Mercenary Security #1', 'gender': 1}, {'name': 'Sam Kennard', 'character': 'Special Ops Sergeant', 'gender': 2}, {'name': 'Aliash Tepina', 'character': 'Hooded Man', 'gender': 2}, {'name': 'Nick Julian', 'character': 'Caterer', 'gender': 2}, {'name': 'Miranda Nolan', 'character': 'Maid #2', 'gender': 0}, {'name': 'Claire Julien', 'character': 'Maid #3', 'gender': 2}, {'name': 'Brett Cullen', 'character': 'Congressman', 'gender': 1}, {'name': 'Reggie Lee', 'character': 'Ross', 'gender': 1}, {'name': 'Joseph Lyle Taylor', 'character': 'DWP Man', 'gender': 1}, {'name': 'Chris Ellis', 'character': 'Fr. Reilly', 'gender': 1}, {'name': 'Duane Henry', 'character': 'SWAT in Dive Bar', 'gender': 1}, {'name': 'James Harvey Ward', 'character': 'SWAT in Alley', 'gender': 2}, {'name': 'Gonzalo Menendez', 'character': 'Cop in Manhole', 'gender': 1}, {'name': 'Cameron Jack', 'character': 'Sewer Thug #1', 'gender': 1}, {'name': 'Lex Daniel', 'character': 'Sewer Thug #2', 'gender': 1}, {'name': 'Tyler Dean Flores', 'character': 'Mark', 'gender': 1}, {'name': 'Thomas Lennon', 'character': 'Doctor', 'gender': 1}, {'name': 'Trevor White', 'character': 'Yuppie', 'gender': 1}, {'name': 'Rob Brown', 'character': 'Allen', 'gender': 1}, {'name': 'Daina Griffith', 'character': "Foley's Wife", 'gender': 2}, {'name': 'Fredric Lehne', 'character': 'Exchange Security Chief', 'gender': 1}, {'name': 'Courtney Munch', 'character': 'Female Security Guard', 'gender': 2}, {'name': 'Chris Hill', 'character': 'Paparazzi #1', 'gender': 1}, {'name': 'Travis Guba', 'character': 'Paparazzi #2', 'gender': 1}, {'name': 'Jay Benedict', 'character': 'Rich Twit', 'gender': 1}, {'name': 'Will Estes', 'character': 'Officer Simon Jansen', 'gender': 1}, {'name': 'David Dayan Fisher', 'character': 'Shoe Shine Man at GSE', 'gender': 1}, {'name': 'Glen Powell', 'character': 'Trader #1', 'gender': 1}, {'name': 'Ben Cornish', 'character': 'Trader #2', 'gender': 2}, {'name': 'Russ Fega', 'character': 'Trader #3', 'gender': 1}, {'name': 'Andres Perez-Molina', 'character': 'Valet at Museum', 'gender': 1}, {'name': 'Brent Briscoe', 'character': 'Veteran Cop', 'gender': 1}, {'name': 'Oliver Cotton', 'character': '2 Star Air Force General', 'gender': 1}, {'name': 'Mark Killeen', 'character': 'Airport Cop', 'gender': 2}, {'name': 'Sarah Goldberg', 'character': 'Analyst #1', 'gender': 0}, {'name': 'John MacMillan', 'character': 'Analyst #2', 'gender': 1}, {'name': 'Robert Wisdom', 'character': 'Army Captain at Bridge', 'gender': 1}, {'name': 'Ronnie Gene Blevins', 'character': 'Cement Truck Driver', 'gender': 1}, {'name': 'John Hollingworth', 'character': 'CIA Analyst', 'gender': 2}, {'name': 'Ian Bohen', 'character': 'Cop with Gordon', 'gender': 1}, {'name': 'Uri Gavriel', 'character': 'Blind Prisoner', 'gender': 1}, {'name': 'Noel Gugliemi', 'character': 'Ex-Prisoner at River', 'gender': 1}, {'name': 'Hector Atreyu Ruiz', 'character': 'Gangbanger', 'gender': 1}, {'name': 'Patrick Cox', 'character': 'Huge Inmate', 'gender': 2}, {'name': 'Aramis Knight', 'character': 'Kid with Apple', 'gender': 1}, {'name': 'William Devane', 'character': 'President', 'gender': 1}, {'name': 'David Gyasi', 'character': 'Skinny Prisoner', 'gender': 1}, {'name': 'Patrick Jordan', 'character': 'Special Forces #2', 'gender': 1}, {'name': 'Joshua Elijah Reese', 'character': 'Mercenary at City Hall', 'gender': 2}, {'name': 'Desmond Harrington', 'character': 'Uniform', 'gender': 1}, {'name': 'Mychael Bates', 'character': 'Bomb Truck Driver', 'gender': 1}, {'name': 'Tomas Arana', 'character': "Wayne's Lawyer", 'gender': 1}, {'name': 'Peter Holden', 'character': 'Applied Sciences Tech #1', 'gender': 2}, {'name': 'David Monahan', 'character': 'Applied Sciences Tech #2', 'gender': 1}, {'name': 'Jillian Armenante', 'character': "Lawyer's Clerk", 'gender': 0}, {'name': 'Aja Evans', 'character': 'Greeter at Museum', 'gender': 0}, {'name': 'Aldous Davidson', 'character': 'Valet at Wayne Enterprises', 'gender': 2}, {'name': 'Wade Williams', 'character': 'Warden at Blackgate', 'gender': 1}, {'name': 'Jake Canuso', 'character': 'Waiter in Florence Café', 'gender': 2}, {'name': 'Julie Mun', 'character': 'Reporter at Stadium', 'gender': 2}, {'name': 'Rory Nolan', 'character': 'Little Boy at Bridge', 'gender': 2}, {'name': 'Harry Coles', 'character': 'Younger Prison Child', 'gender': 2}, {'name': 'Massi Furlan', 'character': 'Janitor at GSE', 'gender': 2}, {'name': 'Christopher Judge', 'character': 'Mercenary Security #4', 'gender': 1}, {'name': 'Patrick Leahy', 'character': 'Board Member #2', 'gender': 1}, {'name': 'Todd Gearhart', 'character': 'Uniform #2', 'gender': 2}, {'name': 'Kevin Kiely', 'character': 'Thug #1 in Basement', 'gender': 2}, {'name': 'Daniel Newman', 'character': 'Thug #2 in Basement', 'gender': 1}, {'name': 'Marc Abbink', 'character': 'Gotham City Police Officer (uncredited)', 'gender': 1}, {'name': 'Isiah Adams', 'character': 'Young Stock Broker (uncredited)', 'gender': 2}, {'name': 'Charlie Alejandro', 'character': 'Gotham City Police Officer (uncredited)', 'gender': 0}, {'name': 'Robert Arensen', 'character': 'Former Upscaler (uncredited)', 'gender': 1}, {'name': 'Grant Babbitt', 'character': 'Angry Mobster / Prisoner (uncredited)', 'gender': 2}, {'name': 'Fileena Bahris', 'character': 'Gotham Rogues Fan (uncredited)', 'gender': 2}, {'name': 'Rick Bolander', 'character': 'Mercenary (uncredited)', 'gender': 2}, {'name': 'Kyle Patrick Brennan', 'character': 'Police Officer (uncredited)', 'gender': 2}, {'name': 'Scott Churchson', 'character': 'Gotham City Police Officer (uncredited)', 'gender': 2}, {'name': 'Bill Cowher', 'character': 'Gotham Rogues Coach (uncredited)', 'gender': 2}, {'name': 'Graham Curry', 'character': 'Pentagon Navy Petty (uncredited)', 'gender': 1}, {'name': 'Stephanie Domini Ehlert', 'character': 'Reporter (uncredited)', 'gender': 0}, {'name': 'John Farrer', 'character': 'Older Upscale Person (uncredited)', 'gender': 1}, {'name': 'Frank Fata', 'character': 'Prisoner / Thug (uncredited)', 'gender': 2}, {'name': 'Christopher Bryan Gomez', 'character': 'Stock Broker (uncredited)', 'gender': 2}, {'name': 'Vito Grassi', 'character': 'Thug (uncredited)', 'gender': 2}, {'name': 'Michael Wren Gucciardo', 'character': 'Stockbroker in Snipers Scoop - Stock Exchange (uncredited)', 'gender': 1}, {'name': 'Sam Harris', 'character': 'Prisoner (uncredited)', 'gender': 1}, {'name': 'Ed Heavey', 'character': 'Police Officer (uncredited)', 'gender': 2}, {'name': 'John W. Iwanonkiw', 'character': 'Gotham Police Officer / Fan (uncredited)', 'gender': 1}, {'name': 'Cindy Jackson', 'character': "Gordon's Cop #4 (uncredited)", 'gender': 2}, {'name': 'Daniel Jordano', 'character': 'Boom Operator News Crew (uncredited)', 'gender': 2}, {'name': 'William Kania', 'character': 'Gothamite (uncredited)', 'gender': 2}, {'name': 'Tiffany Kemp', 'character': 'Gotham Rogues Fan (uncredited)', 'gender': 2}, {'name': 'Sun Jae Kim', 'character': 'Stock Broker (uncredited)', 'gender': 2}, {'name': 'Hrvoje Klecz', 'character': 'Thug (uncredited)', 'gender': 2}, {'name': 'Alex Kruz', 'character': 'Wall Street Broker (uncredited)', 'gender': 2}, {'name': 'Tyler La Marr', 'character': 'Gotham City Police Officer (uncredited)', 'gender': 2}, {'name': 'LeJon', 'character': 'Thug (uncredited)', 'gender': 1}, {'name': 'Paul Jude Letersky', 'character': 'Gotham City Police Officer (uncredited)', 'gender': 2}, {'name': 'Joe Lipari', 'character': 'Prisoner (uncredited)', 'gender': 2}, {'name': 'Silvia Lombardo', 'character': 'Tourist (uncredited)', 'gender': 2}, {'name': 'Jorge Mardel', 'character': 'Gotham Rogues Fan (uncredited)', 'gender': 2}, {'name': 'Cale McConnell', 'character': 'Gotham Rogues Fan (uncredited)', 'gender': 2}, {'name': 'David Dale McCue', 'character': 'Gotham City Police Officer (uncredited)', 'gender': 2}, {'name': 'Tiffany Sander McKenzie', 'character': 'Gotham Rogues Fan (uncredited)', 'gender': 0}, {'name': 'Allen Merritt', 'character': 'Angry Mobster (uncredited)', 'gender': 2}, {'name': 'Olan Montgomery', 'character': 'Wayne Enterprises Security (uncredited)', 'gender': 2}, {'name': 'Alex Moore', 'character': 'Mercenary (uncredited)', 'gender': 2}, {'name': 'Shane Nolan', 'character': 'Gotham City Police Officer (uncredited)', 'gender': 2}, {'name': 'Jackson Nunn', 'character': 'Prisoner / Thug (uncredited)', 'gender': 2}, {'name': "Joseph O'Brien", 'character': 'Gotham City News Cameraman (uncredited)', 'gender': 2}, {'name': 'Michael Papajohn', 'character': 'Prison Guard (uncredited)', 'gender': 1}, {'name': 'Salomon Passariello', 'character': 'Prisoner / Mob (uncredited)', 'gender': 2}, {'name': 'Kyle David Pierce', 'character': 'Gotham Stock Trader (uncredited)', 'gender': 2}, {'name': 'Troy Polamalu', 'character': 'Gotham Rogues Player (uncredited)', 'gender': 2}, {'name': 'Michael Power', 'character': 'Gotham Police Officer (uncredited)', 'gender': 2}, {'name': 'James Rawlings', 'character': 'Thug (uncredited)', 'gender': 2}, {'name': 'Kirsten Roeters', 'character': 'Doris in Black (uncredited)', 'gender': 2}, {'name': 'Ben Roethlisberger', 'character': 'Gotham Rogues Player (uncredited)', 'gender': 2}, {'name': 'Mark Roman', 'character': 'Gotham Police Officer (uncredited)', 'gender': 1}, {'name': 'Anthony J. Sacco', 'character': 'Thug (uncredited)', 'gender': 1}, {'name': 'Eric Salazar', 'character': 'Angry Thug (uncredited)', 'gender': 2}, {'name': 'Emily Schooley', 'character': 'Hot Football Fan (uncredited)', 'gender': 2}, {'name': 'Thomas Tull', 'character': 'Gotham Rogues Owner (uncredited)', 'gender': 2}, {'name': 'Chris Vaina', 'character': 'Thug (uncredited)', 'gender': 1}, {'name': 'Vincent van Ommen', 'character': 'Prisoner (voice) (uncredited)', 'gender': 2}, {'name': 'Michelle Vezzani', 'character': 'Gotham City Press (uncredited)', 'gender': 2}, {'name': 'Barbara Vincent', 'character': 'Gotham Rogues Fan (uncredited)', 'gender': 2}, {'name': 'Justin Michael Woods', 'character': 'Gotham City Police Officer (uncredited)', 'gender': 2}, {'name': 'Jason Yee', 'character': 'Mercenary Soldier (uncredited)', 'gender': 2}, {'name': 'Maria Zambrana', 'character': 'Gotham Citizen (uncredited)', 'gender': 0}, {'name': 'John Zion', 'character': 'Gotham City Police Officer (uncredited)', 'gender': 2}, {'name': 'Alex Ziwak', 'character': 'Thug (uncredited)', 'gender': 1}, {'name': 'Tommy Bayiokos', 'character': 'Gotham City Police Officer (uncredited)', 'gender': 1}, {'name': 'Matthew Gooley', 'character': 'Football Fan (uncredited)', 'gender': 2}, {'name': 'Jeff Moffitt', 'character': 'Cop (uncredited)', 'gender': 2}, {'name': 'Mark Falvo', 'character': 'Football Fan (uncredited)', 'gender': 2}, {'name': 'Diogo Hausen', 'character': 'Thug (uncredited)', 'gender': 2}, {'name': 'Gary Sievers', 'character': 'Prisoner (uncredited)', 'gender': 1}, {'name': 'Orion McCabe', 'character': 'Mercenary (uncredited)', 'gender': 2}, {'name': 'London May', 'character': 'Court Room Thug (uncredited)', 'gender': 1}, {'name': 'James Quinn', 'character': 'Gothamite (uncredited)', 'gender': 1}, {'name': 'Joe Fishel', 'character': 'Police Officer (uncredited)', 'gender': 1}], [{'name': 'Keanu Reeves', 'character': 'John Wick', 'gender': 1}, {'name': 'Riccardo Scamarcio', 'character': "Santino D'Antonio", 'gender': 1}, {'name': 'Ian McShane', 'character': 'Winston', 'gender': 1}, {'name': 'Laurence Fishburne', 'character': 'Bowery King', 'gender': 1}, {'name': 'Common', 'character': 'Cassian', 'gender': 1}, {'name': 'Ruby Rose', 'character': 'Ares', 'gender': 0}, {'name': 'Claudia Gerini', 'character': "Gianna D'Antonio", 'gender': 0}, {'name': 'Lance Reddick', 'character': 'Charon', 'gender': 1}, {'name': 'John Leguizamo', 'character': 'Aurelio', 'gender': 1}, {'name': 'Bridget Moynahan', 'character': 'Helen Wick', 'gender': 0}, {'name': 'Thomas Sadoski', 'character': 'Jimmy', 'gender': 1}, {'name': 'David Patrick Kelly', 'character': 'Charlie', 'gender': 1}, {'name': 'Peter Stormare', 'character': 'Abram', 'gender': 1}, {'name': 'Franco Nero', 'character': 'Julius', 'gender': 1}, {'name': 'Peter Serafinowicz', 'character': 'Sommelier', 'gender': 1}, {'name': 'Perry Yung', 'character': 'Dr. Randall', 'gender': 2}, {'name': 'Tobias Segal', 'character': 'Earl', 'gender': 2}, {'name': 'Chukwudi Iwuji', 'character': 'Akoni', 'gender': 2}, {'name': 'Nico Toffoli', 'character': 'Brian Blackman', 'gender': 2}, {'name': 'Marko Caka', 'character': 'Mario Club Businessman', 'gender': 1}, {'name': 'Aaron Cohen', 'character': 'Hitman', 'gender': 2}, {'name': 'Aly Mang', 'character': "Winston's Personal Assistant", 'gender': 0}, {'name': 'Nora Sommerkamp', 'character': 'Hotel Guest - Assassin', 'gender': 0}, {'name': 'Marmee Cosico', 'character': 'ND Chinatown Pedestrian', 'gender': 0}, {'name': 'Nancy Cejari', 'character': 'Business Woman', 'gender': 2}, {'name': 'Crystal Lonneberg', 'character': 'Switch Board Operator', 'gender': 0}, {'name': 'Frank Modica', 'character': 'Park Goer - Assasin', 'gender': 2}, {'name': 'Jennifer Dong', 'character': 'Cocktail Waitress', 'gender': 2}, {'name': 'Elli', 'character': 'Pawnbroker', 'gender': 1}, {'name': 'Midori Nakamura', 'character': 'Seamstress', 'gender': 2}, {'name': 'Sidney Beitz', 'character': 'Bartender', 'gender': 2}, {'name': 'Mark Vincent', 'character': "Winston's Guard", 'gender': 2}, {'name': 'Justin L. Wilson', 'character': 'Special Ops Assassin', 'gender': 2}, {'name': 'Chris LaPanta', 'character': "Earl's Guard #1", 'gender': 1}, {'name': 'Guyviaud Joseph', 'character': "Earl's Guard #2", 'gender': 1}, {'name': 'Alisa Ermolaev', 'character': 'Switchboard Operator #1', 'gender': 2}, {'name': 'Kelly Rae LeGault', 'character': 'Switchboard Operator #2', 'gender': 2}, {'name': 'Vadim Kroll', 'character': 'Cheslav', 'gender': 2}, {'name': 'Shade Rupe', 'character': 'Opera Attendee (uncredited)', 'gender': 1}, {'name': 'Thaddeus Daniels', 'character': 'Nigerian Man', 'gender': 1}, {'name': 'Margaret Daly', 'character': 'The Operator', 'gender': 2}, {'name': 'Wass Stevens', 'character': 'Consiglieri', 'gender': 1}, {'name': 'Erik Frandsen', 'character': 'Numismatic', 'gender': 2}, {'name': 'Youma Diakite', 'character': 'Lucia', 'gender': 2}, {'name': 'Luca Mosca', 'character': 'Italian Tailor', 'gender': 1}, {'name': 'Mario Donatone', 'character': 'Cardinal', 'gender': 2}, {'name': 'Kenny Sheard', 'character': 'Ares Team (uncredited)', 'gender': 1}, {'name': 'Tim Connolly', 'character': 'Ares Team #2 (uncredited)', 'gender': 2}, {'name': 'Heidi Moneymaker', 'character': 'Assassin (uncredited)', 'gender': 0}, {'name': 'Angel Pai', 'character': 'Assassin (uncredited)', 'gender': 2}, {'name': 'Johnny Otto', 'character': 'Assassin (uncredited)', 'gender': 2}, {'name': 'Nobuya Shimamoto', 'character': 'Assassin (uncredited)', 'gender': 2}, {'name': 'Oleg Prudius', 'character': 'Russian Assassin (uncredited)', 'gender': 1}, {'name': 'Alex Ziwak', 'character': 'Russian Gunman (uncredited)', 'gender': 1}, {'name': 'Ciscandra Nostalghia', 'character': 'Concert Singer', 'gender': 0}], [{'name': 'Jean Arthur', 'character': 'Clarissa Saunders', 'gender': 0}, {'name': 'James Stewart', 'character': 'Jefferson Smith', 'gender': 1}, {'name': 'Claude Rains', 'character': 'Sen. Joseph Harrison Paine', 'gender': 1}, {'name': 'Edward Arnold', 'character': 'Jim Taylor', 'gender': 1}, {'name': 'Guy Kibbee', 'character': "Gov. Hubert 'Happy' Hopper", 'gender': 1}, {'name': 'Thomas Mitchell', 'character': 'Diz Moore', 'gender': 1}, {'name': 'Eugene Pallette', 'character': 'Chick McGann', 'gender': 1}, {'name': 'Beulah Bondi', 'character': 'Ma Smith', 'gender': 0}, {'name': 'H.B. Warner', 'character': 'Senate Majority Leader', 'gender': 1}, {'name': 'Harry Carey', 'character': 'President of the Senate', 'gender': 1}, {'name': 'Astrid Allwyn', 'character': 'Susan Paine', 'gender': 0}, {'name': 'Ruth Donnelly', 'character': 'Emma Hopper', 'gender': 0}, {'name': 'Grant Mitchell', 'character': 'Sen. MacPherson', 'gender': 1}, {'name': 'Porter Hall', 'character': 'Sen. Monroe', 'gender': 1}, {'name': 'Larry Simms', 'character': 'Hopper Boy', 'gender': 1}, {'name': 'H.V. Kaltenborn', 'character': 'Himself, a Radio Broadcaster / Trailer Narrator', 'gender': 2}, {'name': 'Pierre Watkin', 'character': 'Sen. Barnes', 'gender': 1}, {'name': 'Charles Lane', 'character': 'Nosey Newsman', 'gender': 1}, {'name': 'William Demarest', 'character': 'Bill Griffith', 'gender': 1}, {'name': 'Dick Elliott', 'character': 'Carl Cook', 'gender': 1}, {'name': 'Billy Watson', 'character': 'Peter Hopper', 'gender': 1}, {'name': 'Delmar Watson', 'character': 'Jimmie Hopper', 'gender': 1}, {'name': 'John Russell', 'character': 'Hopper Boy', 'gender': 1}, {'name': 'Harry Watson', 'character': 'Hopper Boy', 'gender': 1}, {'name': 'Garry Watson', 'character': 'Hopper Boy', 'gender': 2}, {'name': 'Russell Simpson', 'character': 'Kenneth Allen (uncredited)', 'gender': 1}, {'name': 'Frances Gifford', 'character': 'Hopper Girl (uncredited)', 'gender': 0}, {'name': 'Colin James Mackey', 'character': 'Narrator (uncredited)', 'gender': 2}, {'name': 'Wilson Benge', 'character': "Hopper's Butler (uncredited)", 'gender': 1}, {'name': 'Dickie Jones', 'character': 'Pageboy Richard Jones (uncredited)', 'gender': 1}, {'name': 'Erville Alderson', 'character': 'Handwriting Expert (uncredited)', 'gender': 1}, {'name': 'Frank Puglia', 'character': 'Handwriting Expert (uncredited)', 'gender': 1}, {'name': 'Maurice Cass', 'character': 'Handwriting Expert (uncredited)', 'gender': 1}, {'name': 'Stanley Andrews', 'character': 'Senator Hodges (uncredited)', 'gender': 1}, {'name': 'Al Bridge', 'character': 'Senator Dwight (uncredited)', 'gender': 1}, {'name': 'Edwin August', 'character': 'Senator (uncredited)', 'gender': 2}, {'name': 'Edmund Cobb', 'character': 'Senator Gower (uncredited)', 'gender': 1}, {'name': 'Sam Ash', 'character': 'Senator Lancaster (uncredited)', 'gender': 1}, {'name': 'Harry A. Bailey', 'character': 'Senator Hammett (uncredited)', 'gender': 2}, {'name': 'Stanley Mack', 'character': 'Senator (uncredited)', 'gender': 2}, {'name': 'Brooks Benedict', 'character': 'Senate Clerk (uncredited)', 'gender': 1}, {'name': 'Wade Boteler', 'character': 'Family Man (uncredited)', 'gender': 1}, {'name': 'Dorothy Comingore', 'character': 'Woman at Station (uncredited)', 'gender': 0}, {'name': 'Chester Conklin', 'character': 'Man in Press Section of Senate Gallery (uncredited)', 'gender': 1}, {'name': 'Gino Corrado', 'character': 'Barber (uncredited)', 'gender': 1}, {'name': 'Ann Doran', 'character': "Paine's Secretary (uncredited)", 'gender': 0}, {'name': 'Robert Emmett Keane', 'character': 'Editor (uncredited)', 'gender': 1}, {'name': 'Matt McHugh', 'character': 'Reporter (uncredited)', 'gender': 1}, {'name': 'Kathryn Bates', 'character': 'Committeewoman (uncredited)', 'gender': 2}, {'name': 'Frank Austin', 'character': "Inventor at Smith's Office Door (uncredited)", 'gender': 1}, {'name': 'William Arnold', 'character': 'Reporter (uncredited)', 'gender': 1}, {'name': 'Tommy Baker', 'character': 'Boy Ranger (uncredited)', 'gender': 2}, {'name': 'Robert Sterling', 'character': 'Senate Reporter (uncredited)', 'gender': 1}, {'name': 'Dub Taylor', 'character': 'Reporter (uncredited)', 'gender': 2}, {'name': 'George Chandler', 'character': 'Reporter (uncredited)', 'gender': 1}, {'name': "Fred 'Snowflake' Toones", 'character': 'Porter (uncredited)', 'gender': 1}, {'name': 'Jack Carson', 'character': 'Sweeney Farrell a Newsman (uncredited)', 'gender': 1}, {'name': 'Tommy Bupp', 'character': 'Boy Cheering for Smith in Meeting', 'gender': 1}, {'name': 'Dora Clement', 'character': 'Mrs. McGann', 'gender': 0}, {'name': 'Bert Moorhouse', 'character': 'Man in Senate Building (uncredited)', 'gender': 1}, {'name': 'Harry C. Bradley', 'character': 'Arthur Kim', 'gender': 1}, {'name': 'Lynton Brent', 'character': 'Photographer', 'gender': 1}, {'name': 'Ed Brewer', 'character': 'Senate Reporter', 'gender': 2}, {'name': 'Harlan Briggs', 'character': 'Mr. Edwards a Howling Citizen', 'gender': 1}, {'name': 'Harry Burkhardt', 'character': 'Senate Reporter', 'gender': 1}, {'name': 'Frederick Burton', 'character': 'Senator Dearhorn', 'gender': 1}, {'name': 'Georgia Caine', 'character': 'Third Radio Speaker (uncredited)', 'gender': 0}, {'name': 'Ken Carpenter', 'character': 'Announcer', 'gender': 2}, {'name': 'Burr Caruth', 'character': 'Townsend', 'gender': 2}, {'name': 'Allan Cavan', 'character': 'Ragner a Newsman', 'gender': 2}, {'name': 'Eddy Chandler', 'character': 'Reporter', 'gender': 1}, {'name': 'Davison Clark', 'character': 'Committeeman', 'gender': 1}, {'name': 'Richard Clucas', 'character': 'Minor Role', 'gender': 2}, {'name': 'Shirley Coates', 'character': 'Assistant Bartender', 'gender': 2}, {'name': 'Eddie Coke', 'character': 'Photographer', 'gender': 2}, {'name': 'Hal Cooke', 'character': 'Reporter', 'gender': 2}, {'name': 'Alec Craig', 'character': 'Speaker', 'gender': 1}, {'name': 'Anne Cornwall', 'character': 'Senate Reporter', 'gender': 0}, {'name': 'George Cooper', 'character': 'Waiter (uncredited)', 'gender': 1}, {'name': 'Georgie Cooper', 'character': 'Committeewoman', 'gender': 2}, {'name': 'Jack Cooper', 'character': 'Photographer', 'gender': 2}, {'name': 'Nick Copeland', 'character': 'Senate Reporter', 'gender': 2}, {'name': 'Vernon Dent', 'character': 'Senate Reporter', 'gender': 1}, {'name': 'Maurice Costello', 'character': 'Diggs a Newsman (uncredited)', 'gender': 2}, {'name': 'Beatrice Curtis', 'character': "Paine's Secretary", 'gender': 2}, {'name': 'Lew Davis', 'character': 'Senate Clerk', 'gender': 2}, {'name': 'Dulcie Day', 'character': 'Senate Reporter', 'gender': 2}, {'name': 'Wally Dean', 'character': "Paine's Friend", 'gender': 2}, {'name': 'Harry Depp', 'character': 'Hat Salesman / Secretary', 'gender': 1}, {'name': 'Byron Foulger', 'character': "Hopper's Secretary (uncredited)", 'gender': 1}, {'name': 'Helen Jerome Eddy', 'character': "Paine's Secretary (uncredited)", 'gender': 0}, {'name': 'Douglas Evans', 'character': 'Francis Scott Key', 'gender': 1}, {'name': 'Joe Devlin', 'character': 'Waiter', 'gender': 1}, {'name': 'Clyde Dilson', 'character': 'Reporter', 'gender': 2}, {'name': 'John Dilson', 'character': 'Secretary', 'gender': 2}, {'name': 'Neal Dodd', 'character': 'Senate Chaplain', 'gender': 1}, {'name': 'Lester Dorr', 'character': "Taylor's Stooge", 'gender': 1}, {'name': 'Robert Dudley', 'character': 'Reporter', 'gender': 1}, {'name': 'Edward Earle', 'character': 'Reporter', 'gender': 1}, {'name': 'Jack Egan', 'character': 'Reporter', 'gender': 1}, {'name': 'Eddie Fetherston', 'character': 'Senate Reporter', 'gender': 1}, {'name': 'Mabel Forrest', 'character': 'Senate Reporter', 'gender': 2}, {'name': 'Gladys Gale', 'character': 'Committeewoman (uncredited)', 'gender': 0}, {'name': 'Jack Gardner', 'character': 'Reporter', 'gender': 1}, {'name': 'Mary Gordon', 'character': 'Woman (uncredited)', 'gender': 0}, {'name': 'June Gittelson', 'character': 'Woman at Station', 'gender': 0}, {'name': 'Lorna Gray', 'character': 'Woman at Station', 'gender': 0}, {'name': 'Harry Hayden', 'character': 'First Radio Announcer (uncredited)', 'gender': 1}, {'name': 'Louis Jean Heydt', 'character': 'Soapbox Speaker', 'gender': 1}, {'name': 'John Ince', 'character': 'Senator Fernwick (uncredited)', 'gender': 1}, {'name': 'Frank Jaquet', 'character': 'Senator Byron', 'gender': 1}, {'name': 'Philo McCullough', 'character': 'Senator Albert', 'gender': 1}, {'name': 'Wright Kramer', 'character': 'Senator Carlton', 'gender': 2}, {'name': 'Henry Hebert', 'character': 'Senator', 'gender': 1}, {'name': 'Fred Hoose', 'character': 'Senator', 'gender': 2}, {'name': 'Richard Kipling', 'character': 'Senator', 'gender': 2}, {'name': 'Roger Haliday', 'character': 'Senate Guard', 'gender': 2}, {'name': 'Wilfred Hari', 'character': 'House Boy', 'gender': 2}, {'name': 'Philip Hurlic', 'character': 'Boy Ranger', 'gender': 1}, {'name': 'Olaf Hytten', 'character': 'Butler (uncredited)', 'gender': 1}, {'name': 'Lloyd Ingraham', 'character': 'Committeeman (uncredited)', 'gender': 1}, {'name': 'Mitchell Ingraham', 'character': 'Minor Role', 'gender': 2}, {'name': 'Dick Jensen', 'character': 'Hoodlum', 'gender': 2}, {'name': 'John Lester Johnson', 'character': 'Butler', 'gender': 1}, {'name': 'Eddie Kane', 'character': 'Reporter', 'gender': 1}, {'name': 'Donald Kerr', 'character': 'Reporter', 'gender': 1}, {'name': 'Milton Kibbee', 'character': 'Senate Reporter (uncredited)', 'gender': 1}, {'name': 'Joe King', 'character': 'Summers a Newsman', 'gender': 1}, {'name': 'Evalyn Knapp', 'character': "Reporter Asking 'What Do You Think of the Girls in This Town?'", 'gender': 0}, {'name': 'Paul Kruger', 'character': 'Bodyguard', 'gender': 1}, {'name': 'Bobby Larson', 'character': 'Boy Ranger', 'gender': 2}, {'name': 'Billy Lechner', 'character': 'Boy Ranger', 'gender': 2}, {'name': 'Jackie Lowe', 'character': 'Boy Ranger', 'gender': 2}, {'name': 'P.H. Levy', 'character': 'Rabbi', 'gender': 2}, {'name': 'Vera Lewis', 'character': 'Mrs. Edwards (uncredited)', 'gender': 0}, {'name': 'Jack Lindquist', 'character': 'Chorus Member', 'gender': 2}, {'name': 'George Lloyd', 'character': 'Hoodlum', 'gender': 1}, {'name': 'Arthur Loft', 'character': 'Chief Clerk', 'gender': 1}, {'name': 'Jack Low', 'character': 'Hoodlum', 'gender': 2}, {'name': 'Jimmie Lucas', 'character': 'Photographer', 'gender': 2}, {'name': 'Mary MacLaren', 'character': 'Head Sister', 'gender': 0}, {'name': 'Hank Mann', 'character': 'Photographer (uncredited)', 'gender': 1}, {'name': 'Margaret Mann', 'character': 'Nun with Cheering Orphan Boys', 'gender': 0}, {'name': 'Eric Mayne', 'character': 'Man in Senate Building', 'gender': 1}, {'name': 'Ralph McCullough', 'character': 'Assistant Bartender', 'gender': 2}, {'name': 'George McKay', 'character': 'Reporter', 'gender': 2}, {'name': 'Lafe McKee', 'character': 'Civil War Veteran at Lincoln Memorial', 'gender': 1}, {'name': 'Ben Taggart', 'character': 'Pompous Man (uncredited)', 'gender': 1}, {'name': 'Field Norton', 'character': 'Pompous Man', 'gender': 2}, {'name': 'Wilfred Lucas', 'character': 'Pompous Man', 'gender': 1}, {'name': 'Sammy McKim', 'character': 'Boy Ranger', 'gender': 2}, {'name': 'James McNamara', 'character': 'Reporter', 'gender': 2}, {'name': 'William Newell', 'character': 'Reporter', 'gender': 1}, {'name': 'Robert Middlemass', 'character': 'Second Radio Announcer', 'gender': 1}, {'name': 'James Millican', 'character': 'Senate Reporter', 'gender': 1}, {'name': 'Howard M. Mitchell', 'character': 'Shoe Salesman', 'gender': 1}, {'name': 'Charles R. Moore', 'character': 'Porter', 'gender': 2}, {'name': 'Gene Morgan', 'character': 'Reporter', 'gender': 1}, {'name': 'Robert Morgan', 'character': 'Senate Clerk', 'gender': 1}, {'name': 'Ray Nichols', 'character': 'Boy Ranger', 'gender': 2}, {'name': 'Alex Novinsky', 'character': 'Foreign Diplomat', 'gender': 2}, {'name': "Frank O'Connor", 'character': 'Senator Alfred', 'gender': 1}, {'name': 'Frank Otto', 'character': 'Fisk', 'gender': 2}, {'name': 'Joe Palma', 'character': 'Hoodlum', 'gender': 1}, {'name': 'Spencer Quinn', 'character': 'Minor Role', 'gender': 2}, {'name': 'Tom Quinn', 'character': 'Senate Reporter', 'gender': 1}, {'name': 'Ed Randolph', 'character': 'Senate Reporter', 'gender': 2}, {'name': 'Charles Regan', 'character': 'Hoodlum', 'gender': 1}, {'name': 'Jack Rice', 'character': 'Lang', 'gender': 1}, {'name': 'Wyndham Standing', 'character': 'Senator Ashman', 'gender': 1}, {'name': 'Ferris Taylor', 'character': 'Senator Carlisle', 'gender': 1}, {'name': 'Jack Richardson', 'character': 'Senator Manchester', 'gender': 1}, {'name': 'Walter Soderling', 'character': 'Senator Pickett', 'gender': 1}, {'name': 'Harry Stafford', 'character': 'Senator Atwater', 'gender': 2}, {'name': 'Victor Travis', 'character': 'Senator Grainger', 'gender': 1}, {'name': 'Carl Stockdale', 'character': 'Senator Burdette', 'gender': 1}, {'name': 'Robert Walker', 'character': 'Senator Holland', 'gender': 1}, {'name': 'Henry Roquemore', 'character': 'Senator', 'gender': 1}, {'name': 'Craig Stevens', 'character': 'Senate Reporter', 'gender': 1}, {'name': 'Paul Stanton', 'character': 'Flood a Newsman', 'gender': 1}, {'name': 'Johnny Russell', 'character': 'Otis Hopper', 'gender': 2}, {'name': 'Walter Sande', 'character': 'Newspaperman with Pipe', 'gender': 1}, {'name': 'Frank M. Thomas', 'character': 'Hendricks', 'gender': 1}, {'name': 'Harry Strang', 'character': 'Bodyguard', 'gender': 1}, {'name': 'Harry Tenbrook', 'character': 'Bodyguard', 'gender': 1}, {'name': 'Dave Willock', 'character': 'Senate Guard', 'gender': 1}, {'name': 'Lloyd Whitlock', 'character': 'Schultz a Newsman', 'gender': 1}, {'name': 'John Tyrrell', 'character': 'Undetermined Role', 'gender': 1}, {'name': 'Frederick Vroom', 'character': "Paine's Friend (uncredited)", 'gender': 1}, {'name': 'Charles Sullivan', 'character': 'Cab Driver', 'gender': 1}, {'name': 'Arthur Thalasso', 'character': 'Doorman', 'gender': 1}, {'name': 'Edward Thomas', 'character': 'Butler', 'gender': 2}, {'name': 'Laura Treadwell', 'character': 'Mrs. Taylor', 'gender': 2}, {'name': 'Layne Tom Jr.', 'character': 'Boy Ranger', 'gender': 2}, {'name': 'Myonne Walsh', 'character': 'Jane Hopper', 'gender': 2}, {'name': 'Billy Wayne', 'character': 'Reporter', 'gender': 1}, {'name': 'Max Waizmann', 'character': 'Photographer', 'gender': 2}, {'name': 'John Ward', 'character': 'Photographer', 'gender': 1}, {'name': 'Jane Loofbourrow', 'character': 'Committeewoman', 'gender': 2}, {'name': 'Blanche Payson', 'character': 'Committeewoman', 'gender': 0}, {'name': 'Emma Tansey', 'character': 'Committeewoman', 'gender': 2}, {'name': 'Bess Wade', 'character': 'Committeewoman', 'gender': 2}, {'name': 'Florence Wix', 'character': 'Committeewoman', 'gender': 2}, {'name': 'Eleanor Wood', 'character': 'Committeewoman', 'gender': 2}, {'name': 'Jesse Graves', 'character': 'Black Committeeman', 'gender': 1}, {'name': 'Gus Glassmire', 'character': 'Angry Committee Member', 'gender': 2}, {'name': 'Larry Steers', 'character': 'Committeeman', 'gender': 1}, {'name': 'Landers Stevens', 'character': 'Committeeman', 'gender': 1}, {'name': 'William Worthington', 'character': 'Committeeman', 'gender': 1}, {'name': 'Edward Biby', 'character': 'Foreign Diplomat', 'gender': 2}, {'name': 'Count Stefenelli', 'character': 'Foreign Diplomat', 'gender': 2}, {'name': 'David Wade', 'character': 'Minor Role', 'gender': 2}, {'name': 'Frank McLure', 'character': 'Senator (uncredited)', 'gender': 1}], [{'name': 'Sean Penn', 'character': 'Samuel Bicke', 'gender': 1}, {'name': 'Naomi Watts', 'character': 'Marie Bicke', 'gender': 0}, {'name': 'Don Cheadle', 'character': 'Bonny Simmons', 'gender': 1}, {'name': 'Jack Thompson', 'character': 'Jack Jones', 'gender': 1}, {'name': 'Nick Searcy', 'character': 'Tom Ford', 'gender': 1}, {'name': 'Michael Wincott', 'character': 'Julius Bicke', 'gender': 1}, {'name': 'Mykelti Williamson', 'character': 'Harold Mann', 'gender': 1}, {'name': 'April Grace', 'character': 'Mae Simmons', 'gender': 0}, {'name': 'Lily Knight', 'character': 'Receptionist', 'gender': 0}, {'name': 'Jared Dorrance', 'character': 'Sammy Jr.', 'gender': 2}, {'name': 'Jenna Milton', 'character': 'Ellen', 'gender': 2}, {'name': 'Mariah Massa', 'character': 'Julie', 'gender': 2}, {'name': 'Eileen Ryan', 'character': "Marie's Mother", 'gender': 0}, {'name': 'Derek Greene', 'character': 'Joey Simmons', 'gender': 2}, {'name': 'Joe Marinelli', 'character': 'Mel Samuels', 'gender': 1}, {'name': 'Robert Kenneth Cooper', 'character': 'Irate Driver', 'gender': 1}, {'name': 'Tracy Middendorf', 'character': 'Businesswoman', 'gender': 0}, {'name': 'Kenneth White', 'character': 'Customer #1', 'gender': 1}, {'name': 'Brett Rickaby', 'character': 'Customer #2', 'gender': 1}, {'name': 'James Morrison Reese', 'character': 'Customer #3', 'gender': 2}, {'name': 'Frank Brown Jr.', 'character': 'Customer #4', 'gender': 2}, {'name': 'James Carraway', 'character': 'Customer #5', 'gender': 2}, {'name': 'Jay Jacobus', 'character': 'Customer #6', 'gender': 2}, {'name': 'Kathryn Howell', 'character': 'Hostage', 'gender': 0}, {'name': 'Denise Balthrop Cassidy', 'character': 'Bank Teller', 'gender': 2}, {'name': 'Gary L. Mack', 'character': 'Security Man', 'gender': 2}, {'name': 'Melissa Saltzman', 'character': 'Boarding Agent', 'gender': 2}, {'name': 'Brad William Henke', 'character': 'Martin Jones', 'gender': 1}, {'name': 'J.C. MacKenzie', 'character': 'Co-Pilot', 'gender': 1}], [{'name': 'Hugh Jackman', 'character': 'Logan Howlett / Wolverine / X-24', 'gender': 1}, {'name': 'Patrick Stewart', 'character': 'Charles Xavier / Professor X', 'gender': 1}, {'name': 'Dafne Keen', 'character': 'Laura Kinney / X-23', 'gender': 0}, {'name': 'Boyd Holbrook', 'character': 'Donald Pierce', 'gender': 1}, {'name': 'Stephen Merchant', 'character': 'Caliban', 'gender': 1}, {'name': 'Richard E. Grant', 'character': 'Dr. Zander Rice', 'gender': 1}, {'name': 'Sienna Novikov', 'character': 'Laura (double)', 'gender': 0}, {'name': 'Eriq La Salle', 'character': 'Will Munson', 'gender': 1}, {'name': 'Elise Neal', 'character': 'Kathryn Munson', 'gender': 0}, {'name': 'Elizabeth Rodriguez', 'character': 'Gabriela Lopez', 'gender': 0}, {'name': 'Doris Morgado', 'character': 'Maria', 'gender': 0}, {'name': 'David Kallaway', 'character': 'Rhodes', 'gender': 1}, {'name': 'Han Soto', 'character': 'Valet', 'gender': 2}, {'name': 'Jayson Genao', 'character': 'Rictor', 'gender': 2}, {'name': 'Krzysztof Soszynski', 'character': 'Mohawk', 'gender': 1}, {'name': 'Alison Fernandez', 'character': 'Delilah', 'gender': 0}, {'name': 'Quincy Fouse', 'character': 'Nate Munson', 'gender': 1}, {'name': 'Al Coronel', 'character': 'Federale Commander', 'gender': 1}, {'name': 'Frank Gallegos', 'character': 'Federale Lieutenant', 'gender': 2}, {'name': 'Anthony Escobar', 'character': 'Federale', 'gender': 2}, {'name': 'Reynaldo Gallegos', 'character': 'Rey (as Rey Gallegos)', 'gender': 1}, {'name': 'Stephen Dunlevy', 'character': 'Macon', 'gender': 1}, {'name': 'Daniel Bernhardt', 'character': 'Bone Breaker', 'gender': 2}, {'name': 'Ryan Sturz', 'character': 'Pretty Boy', 'gender': 2}, {'name': 'Brandon Melendy', 'character': 'Reaver', 'gender': 2}, {'name': 'Luke Hawx', 'character': 'Reaver', 'gender': 1}, {'name': "Paul Andrew O'Connor", 'character': 'Reaver', 'gender': 1}, {'name': 'Rocky Abou-Sakher', 'character': 'Reaver', 'gender': 2}, {'name': 'Jean Claude Leuyer', 'character': 'Reaver', 'gender': 2}, {'name': 'Jef Groff', 'character': 'Reaver', 'gender': 2}, {'name': 'Jeremy Fitzgerald', 'character': 'Reaver', 'gender': 1}, {'name': 'Chris Palermo', 'character': 'Reaver', 'gender': 1}, {'name': 'Clinton Roberts', 'character': 'Reaver', 'gender': 2}, {'name': 'Keith Jardine', 'character': 'Reaver', 'gender': 2}, {'name': 'Andrew Arrabito', 'character': 'Reaver', 'gender': 2}, {'name': 'Sebastian James', 'character': 'Reaver', 'gender': 2}, {'name': 'Aaron Matthews', 'character': 'Reaver', 'gender': 2}, {'name': 'Garrett Hammond', 'character': 'Reaver', 'gender': 2}, {'name': 'Matt McClain', 'character': 'Reaver', 'gender': 2}, {'name': 'Maureen Brennan', 'character': 'Motel Manager', 'gender': 0}, {'name': 'Hannah Westerfield', 'character': 'Rebecca', 'gender': 2}, {'name': 'Bryant Tardy', 'character': 'Bobby', 'gender': 2}, {'name': 'Ashlyn Casalegno', 'character': 'Charlotte', 'gender': 2}, {'name': 'Parker Lovein', 'character': 'Lizard Boy', 'gender': 2}, {'name': 'Jimmy Gonzales', 'character': 'Mexican Staff Member', 'gender': 1}, {'name': 'Lennie Loftin', 'character': 'Jackson', 'gender': 1}, {'name': 'Mark Ashworth', 'character': 'Bartender', 'gender': 1}, {'name': 'James Handy', 'character': 'Old Doctor', 'gender': 1}, {'name': 'Bryce Romero', 'character': 'Prom Teen #1', 'gender': 1}, {'name': 'Phi Vu', 'character': 'Prom Teen #2', 'gender': 2}, {'name': 'Chester Rushing', 'character': 'Prom Teen #3', 'gender': 1}, {'name': 'David Simpson', 'character': 'Prom Teen #4', 'gender': 2}, {'name': 'Lauren Gros', 'character': 'Bridesmaid', 'gender': 2}, {'name': 'John Raymond', 'character': 'Pastor', 'gender': 1}, {'name': 'Vanessa Cloke', 'character': 'Mourning Widow', 'gender': 0}, {'name': 'Katie Anne Mitchell', 'character': 'Infirmary Nurse', 'gender': 0}, {'name': 'Lara Grice', 'character': 'Car Dealer', 'gender': 0}, {'name': 'James Moses Black', 'character': 'Major', 'gender': 1}, {'name': 'Ned Yousef', 'character': 'Banger', 'gender': 1}, {'name': 'Baxter Humby', 'character': 'Banger', 'gender': 1}, {'name': 'Daniel Hernández', 'character': 'Banger', 'gender': 2}, {'name': 'Michael Lehr', 'character': 'Banger', 'gender': 2}, {'name': 'Bryan Sloyer', 'character': 'Banger', 'gender': 2}, {'name': 'John Bernecker', 'character': 'Federale', 'gender': 1}, {'name': 'Joe Williams', 'character': 'Federale', 'gender': 2}, {'name': 'Robert Wu', 'character': 'Federale', 'gender': 1}, {'name': 'Victor Winters-Junco', 'character': 'Federale', 'gender': 2}, {'name': 'Craig Henningsen', 'character': 'Federale', 'gender': 2}, {'name': 'Evan Dane Taylor', 'character': 'Federale', 'gender': 1}, {'name': 'Toby Holguin', 'character': 'Federale', 'gender': 1}, {'name': 'Panuvat Anthony Nanakornpanom', 'character': 'Federale', 'gender': 2}, {'name': 'Eyad Elbitar', 'character': 'Federale', 'gender': 1}, {'name': 'Rissa Rose Kilar', 'character': 'Pine Cone Girl', 'gender': 2}, {'name': 'Salef Celiz', 'character': 'Jonah', 'gender': 2}, {'name': 'Aidan Kennedy', 'character': 'Mutant Child', 'gender': 2}, {'name': 'Chase Cubia', 'character': 'Mutant Child', 'gender': 2}, {'name': 'Vincenzo Lucatorto', 'character': 'Mutant Child', 'gender': 2}, {'name': 'Haley Glass', 'character': 'Mutant Child', 'gender': 2}, {'name': 'Nayah Murphy', 'character': 'Mutant Child', 'gender': 2}, {'name': 'Emma Teo', 'character': 'Mutant Child', 'gender': 2}, {'name': 'Noell Jellison', 'character': 'Mutant Child', 'gender': 2}, {'name': 'Ella Rowbotham', 'character': 'Mutant Child', 'gender': 2}, {'name': 'Hudson Wright', 'character': 'Mutant Child', 'gender': 2}, {'name': 'Sebeon Jackson', 'character': 'Mutant Child', 'gender': 2}, {'name': 'Kelton DuMont', 'character': 'Glowing Eyes Boy', 'gender': 1}, {'name': 'Damon Carney', 'character': "Jackson's Henchman", 'gender': 1}, {'name': 'Cynthia Woods', 'character': 'Screaming Casino Woman', 'gender': 2}, {'name': "Mali O'Connor", 'character': 'Bumpy Skinned Kid', 'gender': 2}, {'name': 'Robert Vargas', 'character': 'Border Guard', 'gender': 2}, {'name': 'David Paris', 'character': 'Helicopter Pilot', 'gender': 1}, {'name': 'Ted Ferguson', 'character': "Laura's Grandpa (uncredited)", 'gender': 1}, {'name': 'Juan Gaspard', 'character': 'Homeless Guy (uncredited)', 'gender': 2}, {'name': 'Christopher Heskey', 'character': 'Reaver (uncredited)', 'gender': 2}, {'name': 'Julia Holt', 'character': 'Casino Patron (uncredited)', 'gender': 2}, {'name': 'Natosha Humphrey', 'character': 'Club Goer (uncredited)', 'gender': 2}, {'name': 'Lizeth Hutchings', 'character': 'Nurse (uncredited)', 'gender': 0}, {'name': 'Donald M. Krause', 'character': 'Clinic Visitor (uncredited)', 'gender': 2}, {'name': 'Justin Lebrun', 'character': 'Reaver (uncredited)', 'gender': 2}, {'name': 'Gentry Lee', 'character': 'Military Contractor (uncredited)', 'gender': 2}, {'name': 'Gonzalo Robles', 'character': 'Special Forces (uncredited)', 'gender': 2}, {'name': 'Daymond C. Roman', 'character': 'Special Forces (uncredited)', 'gender': 1}, {'name': 'Mary Peyton Stewart', 'character': 'Bridesmaid (uncredited)', 'gender': 2}, {'name': 'Michael Love Toliver', 'character': 'FBI Agent (uncredited)', 'gender': 2}, {'name': 'Gregory Paul Valdez', 'character': 'FBI Agent (uncredited)', 'gender': 2}, {'name': 'Dave Davis', 'character': 'Convenience Store Clerk', 'gender': 1}, {'name': 'Stephen Apostolina', 'character': 'ADR Voice Cast (voice)', 'gender': 2}, {'name': 'David Berón', 'character': 'ADR Voice Cast (voice)', 'gender': 1}, {'name': 'Lucien Dodge', 'character': 'ADR Voice Cast (voice)', 'gender': 2}, {'name': 'Taliesin Jaffe', 'character': 'ADR Voice Cast (voice)', 'gender': 1}, {'name': 'David Michie', 'character': 'ADR Voice Cast (voice)', 'gender': 1}, {'name': 'Ben Pronsky', 'character': 'ADR Voice Cast (voice)', 'gender': 2}, {'name': 'Al Rodrigo', 'character': 'ADR Voice Cast (voice)', 'gender': 2}], [{'name': 'Leonardo DiCaprio', 'character': 'Jordan Belfort', 'gender': 1}, {'name': 'Jonah Hill', 'character': 'Donnie Azoff', 'gender': 1}, {'name': 'Margot Robbie', 'character': 'Naomi Lapaglia', 'gender': 0}, {'name': 'Kyle Chandler', 'character': 'Agent Patrick Denham', 'gender': 1}, {'name': 'Rob Reiner', 'character': 'Max Belfort', 'gender': 1}, {'name': 'Matthew McConaughey', 'character': 'Mark Hanna', 'gender': 1}, {'name': 'Jon Bernthal', 'character': 'Brad Bodnick', 'gender': 1}, {'name': 'Kenneth Choi', 'character': 'Chester Ming', 'gender': 1}, {'name': 'P. J. Byrne', 'character': "Nicky Koskoff ('Rugrat')", 'gender': 1}, {'name': 'Brian Sacca', 'character': "Robbie Feinberg ('Pinhead')", 'gender': 1}, {'name': 'Henry Zebrowski', 'character': "Alden Kupferberg ('Sea Otter')", 'gender': 2}, {'name': 'Jon Favreau', 'character': 'Manny Riskin', 'gender': 1}, {'name': 'Cristin Milioti', 'character': 'Teresa Petrillo', 'gender': 0}, {'name': 'Joanna Lumley', 'character': 'Aunt Emma', 'gender': 0}, {'name': 'Jake Hoffman', 'character': 'Steve Madden', 'gender': 1}, {'name': 'Ethan Suplee', 'character': 'Toby Welch', 'gender': 1}, {'name': 'Jean Dujardin', 'character': 'Jean-Jacques Handali', 'gender': 1}, {'name': 'Christine Ebersole', 'character': 'Leah Belfort', 'gender': 0}, {'name': 'Shea Whigham', 'character': 'Captain Ted Beecham', 'gender': 1}, {'name': 'Katarina Čas', 'character': 'Chantalle', 'gender': 0}, {'name': 'P. J. Byrne', 'character': 'Rugrat', 'gender': 1}, {'name': 'Bo Dietl', 'character': 'Bo Dietl', 'gender': 1}, {'name': 'Jon Spinogatti', 'character': 'Nicholas the Butler', 'gender': 1}, {'name': 'Aya Cash', 'character': 'Janet', 'gender': 0}, {'name': 'Rizwan Manji', 'character': 'Kalil', 'gender': 1}, {'name': 'Stephanie Kurtzuba', 'character': 'Kimmie Belzer', 'gender': 0}, {'name': 'J.C. MacKenzie', 'character': 'Lucas Solomon', 'gender': 1}, {'name': 'Ashlie Atkinson', 'character': 'Rochelle Applebaum', 'gender': 0}, {'name': 'Stephen Kunken', 'character': 'Jerry Fogel', 'gender': 1}, {'name': 'Ted Griffin', 'character': 'Agent Hughes', 'gender': 1}, {'name': 'Sandra Nelson', 'character': 'Aliyah Farran', 'gender': 0}, {'name': 'Dan Bittner', 'character': 'Rothschild Broker #1', 'gender': 2}, {'name': 'Spike Jonze', 'character': 'Dwayne', 'gender': 1}, {'name': 'Mackenzie Meehan', 'character': 'Hildy Azoff', 'gender': 2}, {'name': 'Thomas Middleditch', 'character': 'Stratton Broker in a Bowtie', 'gender': 1}, {'name': 'Edward Herrmann', 'character': 'Stratton Oakmont Commercial (voice)', 'gender': 1}, {'name': 'Jordan Belfort', 'character': 'Auckland Straight Line Host', 'gender': 2}, {'name': 'Fran Lebowitz', 'character': 'Honorary Samantha Stogel', 'gender': 2}, {'name': 'Robert Clohessy', 'character': "Nolan Drager (Jordan's Lawyer)", 'gender': 1}, {'name': 'Welker White', 'character': 'Waitress', 'gender': 0}, {'name': 'Danny Flaherty', 'character': 'Zip (Lude Buying Teenager #1)', 'gender': 1}, {'name': 'Carla Corvo', 'character': 'Pam', 'gender': 2}, {'name': 'Madison McKinley', 'character': 'Heidi', 'gender': 2}, {'name': 'Aaron Lazar', 'character': 'Blair Hollingsworth', 'gender': 1}, {'name': 'Steve Routman', 'character': 'SEC Attorney #1', 'gender': 1}, {'name': 'Steve Witting', 'character': 'SEC Attorney #2', 'gender': 2}, {'name': 'Charley Morgan', 'character': 'SEC Attorney #3', 'gender': 2}, {'name': 'Michael Nathanson', 'character': 'Barry Kleinman', 'gender': 1}, {'name': 'Kathleen Fellegara', 'character': 'Straight Line Testimonial #1', 'gender': 2}, {'name': 'John Behlmann', 'character': 'Rothschild Broker #2', 'gender': 2}, {'name': 'Ward Horton', 'character': 'Rothschild Broker #3', 'gender': 1}, {'name': 'Bret Shuford', 'character': 'Rothschild Broker #4', 'gender': 2}, {'name': "J.T. O'Connor", 'character': 'Rothschild Broker #8', 'gender': 2}, {'name': 'Steven Boyer', 'character': "Investor's Center Broker #1", 'gender': 1}, {'name': 'Danny A. Abeckaser', 'character': "Investor's Center Broker #2", 'gender': 1}, {'name': 'Matthew Rauch', 'character': 'Stratton Broker #1', 'gender': 1}, {'name': 'Michael Izquierdo', 'character': 'Stratton Broker #2', 'gender': 2}, {'name': 'Donnie Keshawarz', 'character': 'Stratton Broker #3', 'gender': 1}, {'name': 'Johnathan Tchaikovsky', 'character': 'Stratton Broker #4', 'gender': 1}, {'name': 'Aaron Glaser', 'character': 'Stratton Broker #5', 'gender': 2}, {'name': 'Ben Rameaka', 'character': 'Stratton Broker #6', 'gender': 2}, {'name': 'Brian Charles Johnson', 'character': 'Young Broker', 'gender': 2}, {'name': 'Sebastian Tillinger', 'character': 'Another Broker', 'gender': 1}, {'name': 'Chris Riggi', 'character': 'Party Broker #1', 'gender': 1}, {'name': 'Meghan Rafferty', 'character': "Donnie's Assistant", 'gender': 0}, {'name': 'José Ramón Rosario', 'character': "Maitre d' Hector", 'gender': 2}, {'name': 'Davram Stiefler', 'character': "Broker in Men's Room", 'gender': 1}, {'name': 'Dan Daily', 'character': 'Honorary Raymond Samitz', 'gender': 2}, {'name': 'Ben Van Bergen', 'character': 'Swiss Customs Officer #1', 'gender': 1}, {'name': 'Matte Osian', 'character': 'Swiss Customs Officer #2', 'gender': 2}, {'name': 'Michael Devine', 'character': 'Cop #1', 'gender': 1}, {'name': 'Jason Furlani', 'character': 'Cop #2', 'gender': 2}, {'name': 'Jeremy Bobb', 'character': 'Police Officer #2', 'gender': 2}, {'name': 'Tom Greer', 'character': 'Brookville Police Officer #2', 'gender': 1}, {'name': 'Sharon Jones', 'character': 'Wedding Singer #1', 'gender': 2}, {'name': 'Emily Tremaine', 'character': 'Cristy', 'gender': 0}, {'name': 'Zineb Oukach', 'character': 'Hostess on The Naomi', 'gender': 0}, {'name': 'Giselle Eisenberg', 'character': 'Skylar Belfort (4 Years Old)', 'gender': 0}, {'name': 'Deema Aitken', 'character': 'Lude Buying Teenager #2', 'gender': 2}, {'name': 'Ashley Springer', 'character': 'Job Applicant #1', 'gender': 1}, {'name': 'Rémy Bennett', 'character': 'Abby', 'gender': 2}, {'name': 'Catherine Curtin', 'character': 'FBI Agent #1', 'gender': 0}, {'name': 'Paul Urcioli', 'character': 'FBI Agent #2', 'gender': 1}, {'name': "Michael O'Hara", 'character': 'Prison Guard', 'gender': 2}, {'name': 'Michael Bryan French', 'character': 'Inmate Playing Tennis #2', 'gender': 2}, {'name': 'Armen Garo', 'character': "Rao's Patron #1", 'gender': 1}, {'name': 'Garry Pastore', 'character': "Rao's Patron #2", 'gender': 2}, {'name': 'Louis Vanaria', 'character': "Rao's Patron #3", 'gender': 2}, {'name': 'Peter Youngblood Hills', 'character': 'Audience Member #1', 'gender': 1}, {'name': 'Brendan Griffin', 'character': 'Audience Member #3', 'gender': 1}, {'name': 'Derek Milman', 'character': 'Audience Member #4', 'gender': 2}, {'name': 'Victor Verhaeghe', 'character': 'Audience Member #5', 'gender': 1}, {'name': 'Chris Caldovino', 'character': 'Rocco #1', 'gender': 2}, {'name': 'Fileena Bahris', 'character': 'Wedding Guest (uncredited)', 'gender': 2}, {'name': 'Silvia Kal', 'character': 'Wedding Guest (uncredited)', 'gender': 2}, {'name': 'Kamron Leal', 'character': 'Wedding Guest (uncredited)', 'gender': 2}, {'name': 'Tommy Bayiokos', 'character': 'Kacandes Diner Man (uncredited)', 'gender': 1}, {'name': 'Gianni Biasetti Jr.', 'character': 'Italian Coastguard (uncredited)', 'gender': 2}, {'name': 'Rick Bolander', 'character': 'Broker (uncredited)', 'gender': 2}, {'name': 'Spenser Granese', 'character': 'Broker (uncredited)', 'gender': 2}, {'name': 'Julian Brand', 'character': 'Stratton Broker (uncredited)', 'gender': 2}, {'name': 'Kenneth Carrella', 'character': 'Stratton Broker (uncredited)', 'gender': 1}, {'name': 'Austin Farwell', 'character': 'Stratton Broker (uncredited)', 'gender': 2}, {'name': 'Zach Miko', 'character': 'Stratton Broker (uncredited)', 'gender': 2}, {'name': 'Tyler Evan Rowe', 'character': 'Stratton Broker (uncredited)', 'gender': 2}, {'name': 'Stefano Villabona', 'character': 'Stratton Broker (uncredited)', 'gender': 1}, {'name': 'Gregory Brown', 'character': 'Upper Eastside Pedestrian (uncredited)', 'gender': 1}, {'name': 'Tucker Bryan', 'character': 'Wolfpack #4 (uncredited)', 'gender': 2}, {'name': 'Michael Jefferson', 'character': 'Wolfpack #1 (uncredited)', 'gender': 1}, {'name': 'Bryan Burton', 'character': 'Young Stratton Inc. Broker (uncredited)', 'gender': 2}, {'name': 'Mike Catapano', 'character': 'Broker Applicant (uncredited)', 'gender': 2}, {'name': 'Steven Conroy', 'character': 'Audience Member (uncredited)', 'gender': 2}, {'name': 'Chris J. Cullen', 'character': 'Court Officer / Broker (uncredited)', 'gender': 2}, {'name': 'Kelsey Deanne', 'character': 'Pool Party Guest (uncredited)', 'gender': 0}, {'name': 'Maria Di Angelis', 'character': "Nicky Koskoff's Escort (uncredited)", 'gender': 2}, {'name': 'Matthew Gooley', 'character': 'Tai Chi Inmate (uncredited)', 'gender': 2}, {'name': 'London Hall', 'character': 'Swiss Airline Passenger (uncredited)', 'gender': 0}, {'name': 'Rosemary Howard', 'character': "Rao's Pedestrian (uncredited)", 'gender': 0}, {'name': 'Chris Kerson', 'character': "Kimmie Belzer's Desk Mate (uncredited)", 'gender': 1}, {'name': 'Natasha Key', 'character': 'Singer', 'gender': 0}, {'name': 'Ben Leasure', 'character': 'Brantley (uncredited)', 'gender': 2}, {'name': 'Paul Jude Letersky', 'character': 'Marina Sightseer (uncredited)', 'gender': 2}, {'name': 'Will MacAdam', 'character': 'Irritated Plane Passinger (uncredited)', 'gender': 1}, {'name': 'Jeff Moffitt', 'character': 'Party Goer (uncredited)', 'gender': 2}, {'name': 'Chris Nunez', 'character': 'Miami Busboy (uncredited)', 'gender': 1}, {'name': "Seregon O'Dassey", 'character': 'Masseuse to Donnie Azoff (uncredited)', 'gender': 0}, {'name': 'Joseph Oliveira', 'character': 'Court Room Attendant (uncredited)', 'gender': 1}, {'name': 'And Palladino', 'character': 'Stratton Applicant (uncredited)', 'gender': 2}, {'name': 'Michael Power', 'character': 'Seminar Guest (uncredited)', 'gender': 2}, {'name': 'Jon Douglas Rainey', 'character': 'Strattonite (uncredited)', 'gender': 2}, {'name': 'Nicole Rutigliano', 'character': 'Stripper (uncredited)', 'gender': 2}, {'name': 'Sibyl Santiago', 'character': 'Convention Attendee (uncredited)', 'gender': 2}, {'name': 'Kathryn Shasha', 'character': 'Courtroom Observer (uncredited)', 'gender': 2}, {'name': 'Vitaliy Shtabnoy', 'character': 'Kent - Broker (uncredited)', 'gender': 2}, {'name': 'Blago Simon', 'character': 'Stock Broker (uncredited)', 'gender': 2}, {'name': 'Paul Thornton', 'character': '1st Class Swiss Airline Passenger (uncredited)', 'gender': 2}, {'name': 'Madeleine Wade', 'character': 'Call Girl (uncredited)', 'gender': 0}, {'name': 'David Wenzel', 'character': 'Banker (uncredited)', 'gender': 2}, {'name': 'Joe Zaso', 'character': 'Rudy (uncredited)', 'gender': 2}, {'name': 'Francis Brooke', 'character': 'Conference Attendee (uncredited)', 'gender': 1}], [{'name': 'Robert De Niro', 'character': 'Detective Eddie Flemming', 'gender': 1}, {'name': 'Charlize Theron', 'character': 'Rose Hearn', 'gender': 0}, {'name': 'Edward Burns', 'character': 'Jordy Warsaw', 'gender': 1}, {'name': 'Kelsey Grammer', 'character': 'Robert Hawkins', 'gender': 1}, {'name': 'Avery Brooks', 'character': 'Detective Leon Jackson', 'gender': 1}, {'name': 'Vera Farmiga', 'character': 'Dahpne Handlova', 'gender': 0}, {'name': 'Kim Cattrall', 'character': 'Cassandra', 'gender': 0}, {'name': 'David Alan Grier', 'character': 'Mugger in Central Park', 'gender': 1}, {'name': 'Roseanne Barr', 'character': 'Herself', 'gender': 0}, {'name': 'Anton Yelchin', 'character': 'Boy in Burning Building', 'gender': 1}, {'name': 'Paul Herman', 'character': 'Detective Paulie', 'gender': 1}, {'name': 'Melina Kanakaredes', 'character': 'Nicolette Karas', 'gender': 0}, {'name': 'Karel Roden', 'character': 'Emil Slovak', 'gender': 1}, {'name': 'Oleg Taktarov', 'character': 'Oleg Razgul', 'gender': 1}, {'name': 'John DiResta', 'character': 'Bobby Korfin', 'gender': 1}, {'name': 'James Handy', 'character': 'Deputy Chief Fire Marshal Declan Duffy', 'gender': 1}, {'name': 'Darius McCrary', 'character': 'Detective Tommy Cullen', 'gender': 1}, {'name': 'Brantley Bush', 'character': 'Fireman', 'gender': 1}, {'name': 'Bruce Cutler', 'character': 'Himself', 'gender': 1}, {'name': 'Vladimir Mashkov', 'character': 'Milos Karlova', 'gender': 1}, {'name': 'Noelle Evans', 'character': 'Honey the Hooker', 'gender': 0}, {'name': 'Mindy Marin', 'character': 'Maggie', 'gender': 0}, {'name': 'Gabriel Casseus', 'character': 'Unique', 'gender': 1}, {'name': 'Joe Lisi', 'character': 'Police Captain', 'gender': 1}, {'name': 'Tygh Runyan', 'character': 'Stephen Geller', 'gender': 1}, {'name': 'Ritchie Coster', 'character': 'News Stand Vendor', 'gender': 1}, {'name': 'Bobby Field', 'character': 'Talk Show Son', 'gender': 1}, {'name': 'Kurt Andon', 'character': 'Pizza Man', 'gender': 1}, {'name': 'Rikki Klieman', 'character': "Daphne's Lawyer", 'gender': 0}, {'name': 'Sebastian Roché', 'character': 'Ludwig the Hairdresser', 'gender': 1}, {'name': 'Kata Dobó', 'character': "Redhead at Ludwig's", 'gender': 0}, {'name': 'Dorit Sauer', 'character': 'Escort Service Woman #3', 'gender': 0}, {'name': 'Marco Khan', 'character': 'Emigrant', 'gender': 1}, {'name': 'Toni Kalem', 'character': 'Woman at Planet Hollywood', 'gender': 0}, {'name': 'Lennox Brown', 'character': 'Lookout Guy', 'gender': 1}, {'name': 'Peter Arnett', 'character': 'Himself', 'gender': 1}, {'name': 'Jane Velez-Mitchell', 'character': 'TV Newscaster #2', 'gender': 0}, {'name': 'Barry Primus', 'character': 'Cab Driver', 'gender': 1}, {'name': 'Scott Dillin', 'character': 'Battery Park Policeman', 'gender': 1}, {'name': 'Malcolm-Jamal Warner', 'character': '', 'gender': 1}, {'name': 'Dwayne Adway', 'character': 'Additional voices (voice) (uncredited)', 'gender': 1}, {'name': 'William H. Burns', 'character': 'Uniformed Cop (uncredited)', 'gender': 1}, {'name': 'Jamie Burton-Oare', 'character': 'Reporter (uncredited)', 'gender': 0}, {'name': 'Giovanni Capitello', 'character': 'Police Officer (uncredited)', 'gender': 1}, {'name': 'Phillip V. Caruso', 'character': 'Arguing Husband (uncredited)', 'gender': 1}, {'name': 'Rose Colasanti', 'character': 'Woman in Bar (uncredited)', 'gender': 0}, {'name': 'Ben Fiore', 'character': 'Police Officer (uncredited)', 'gender': 1}, {'name': 'Zuhair Haddad', 'character': 'Airport Passenger (uncredited)', 'gender': 1}, {'name': 'Philippe Hartmann', 'character': 'Daily News Reporter (uncredited)', 'gender': 1}, {'name': 'Everton Lawrence', 'character': 'Restaurant Patron (uncredited)', 'gender': 1}, {'name': 'George F. Miller', 'character': 'TV News Reporter (uncredited)', 'gender': 1}, {'name': 'Ellen Whyte', 'character': 'Woman #2 at Planet Hollywood (uncredited)', 'gender': 0}, {'name': 'Irina Gasanova', 'character': 'Tamina Karlova', 'gender': 0}, {'name': 'Christine Claibourne', 'character': "Maggie's Assistant", 'gender': 0}, {'name': 'Stephen Davis', 'character': 'Murphy', 'gender': 1}, {'name': 'William Stanton', 'character': 'Garcia', 'gender': 1}, {'name': 'Louis F. Garcia', 'character': 'Fire Chief', 'gender': 1}, {'name': 'Mike Camello', 'character': 'Fireman Camello', 'gender': 1}, {'name': 'Barry Blueian', 'character': 'Immigration Supervisor', 'gender': 1}, {'name': 'Ashton Dane', 'character': 'Talk Show Father', 'gender': 1}, {'name': 'Gail Greaves', 'character': 'Talk Show Wife', 'gender': 0}, {'name': 'Lynette Ruiz', 'character': 'Composite Artist', 'gender': 0}, {'name': 'Elma Vann', 'character': 'Escort Service Woman Linda', 'gender': 0}, {'name': 'Olga Morgunova', 'character': 'Escort Service Woman #2', 'gender': 0}, {'name': 'Haik Garibyan', 'character': 'Emigrant', 'gender': 1}, {'name': 'Jorge H. Guzman', 'character': 'Bouncer at Planet Hollywood', 'gender': 1}, {'name': 'George Poulos', 'character': 'Greek Waiter', 'gender': 1}, {'name': 'Sylvia Lopez', 'character': 'TV Newscaster #1', 'gender': 0}, {'name': 'Bryan Foster', 'character': "Hawkins' Cameraman", 'gender': 1}, {'name': 'Alan Cohen', 'character': "Hawkins' Sound Man", 'gender': 1}, {'name': 'Spero Stamboulis', 'character': "Nicolette's Cameraman", 'gender': 1}, {'name': 'Joseph Alfieri', 'character': 'Reporter', 'gender': 1}, {'name': 'Julieta Espinoza', 'character': "Unique's Girlfriend", 'gender': 0}, {'name': 'Macy Merced', 'character': 'Paramedic #1', 'gender': 0}, {'name': 'Tracey Mulqueen', 'character': 'Paramedic #2', 'gender': 0}, {'name': 'Dominic Maglione', 'character': 'Police Officer #1', 'gender': 2}, {'name': 'Ron Wall', 'character': 'Police Officer #2', 'gender': 1}, {'name': 'Sophia Alexis', 'character': 'Witness (uncredited)', 'gender': 0}, {'name': 'Michael Arthur', 'character': 'NYPD #2 (uncredited)', 'gender': 1}, {'name': 'Mark Cheek', 'character': 'Cameraman (uncredited)', 'gender': 1}, {'name': 'Celeste Gregoire', 'character': 'Girl in Restaurant (uncredited)', 'gender': 0}, {'name': 'Janean Christine Mariani', 'character': 'Restaurant Guest (uncredited)', 'gender': 0}, {'name': 'Riley G. Matthews Jr.', 'character': 'NYPD Police Officer (uncredited)', 'gender': 1}, {'name': 'Leda Mulholland', 'character': 'Reporter (uncredited)', 'gender': 0}, {'name': 'Floyd Resnick', 'character': "NYPD Officer at Ludwig's (uncredited)", 'gender': 1}, {'name': 'Erica Taylor', 'character': 'Nightclub Patron (uncredited)', 'gender': 0}], [{'name': 'Adam Sandler', 'character': 'Zohan', 'gender': 1}, {'name': 'John Turturro', 'character': 'The Phantom', 'gender': 1}, {'name': 'Emmanuelle Chriqui', 'character': 'Dalia', 'gender': 0}, {'name': 'Nick Swardson', 'character': 'Michael', 'gender': 1}, {'name': 'Lainie Kazan', 'character': 'Gail', 'gender': 0}, {'name': 'Daoud Heidami', 'character': 'Nasi', 'gender': 2}, {'name': 'Dave Matthews', 'character': 'James', 'gender': 1}, {'name': 'Charlotte Rae', 'character': 'Mrs. Greenhouse', 'gender': 0}, {'name': 'Michael Buffer', 'character': 'Walbridge', 'gender': 1}, {'name': 'Rob Schneider', 'character': 'Salim', 'gender': 1}, {'name': 'Ido Mosseri', 'character': 'Oori', 'gender': 1}, {'name': 'Sayed Badreya', 'character': 'Hamdi', 'gender': 1}, {'name': 'Kevin Nealon', 'character': 'Kevin', 'gender': 1}, {'name': 'Robert Smigel', 'character': 'Yosi', 'gender': 1}, {'name': 'Mariah Carey', 'character': 'Mariah Carey', 'gender': 0}, {'name': 'Ido Ezra', 'character': 'Hassan', 'gender': 2}, {'name': 'Yamit Sol', 'character': 'Dorit', 'gender': 0}, {'name': 'Donna Feldman', 'character': 'Michal', 'gender': 0}, {'name': 'Norma Michaels', 'character': 'Older Lady in Salon', 'gender': 0}, {'name': 'Danny A. Abeckaser', 'character': "Ze'ev", 'gender': 1}, {'name': 'Adria Tennor', 'character': "Kids' Salon Owner", 'gender': 0}, {'name': 'Bunny Levine', 'character': 'Older Lady in Salon', 'gender': 0}, {'name': 'Bruce Vilanch', 'character': 'Bruce Vilanch', 'gender': 1}, {'name': 'John McEnroe', 'character': 'Himself', 'gender': 1}, {'name': 'Dina Doron', 'character': "Zohan's Mother", 'gender': 0}, {'name': 'Shelley Berman', 'character': "Zohan's Father", 'gender': 1}, {'name': 'Chris Rock', 'character': 'Taxi Driver', 'gender': 1}, {'name': 'John Paul DeJoria', 'character': 'Paul Mitchell', 'gender': 2}, {'name': 'Alec Mapa', 'character': 'Claude', 'gender': 1}, {'name': 'Ahmed Ahmed', 'character': 'Waleed', 'gender': 1}, {'name': 'Ben Wise', 'character': 'Yitzhak', 'gender': 2}, {'name': 'John Farley', 'character': 'Tom', 'gender': 1}, {'name': 'Yossi Marshek', 'character': 'Pinchas', 'gender': 2}, {'name': 'Guri Weinberg', 'character': 'Aharon', 'gender': 2}, {'name': 'Mousa Kraish', 'character': 'Bashir', 'gender': 2}, {'name': 'Roni Levy', 'character': 'Ephraim', 'gender': 2}, {'name': 'Reuven Bar-Yotam', 'character': 'Levi', 'gender': 2}, {'name': 'Barry Livingston', 'character': 'Gray Kleibolt', 'gender': 1}, {'name': 'Rick Gifford', 'character': 'Philip', 'gender': 2}, {'name': 'Daniel Browning Smith', 'character': 'Real Estate Agent', 'gender': 2}, {'name': 'Tyler Spindel', 'character': 'Doorman', 'gender': 1}, {'name': 'Julia Lea Wolov', 'character': "Mariah's Assistant", 'gender': 0}, {'name': 'Dana Goodman', 'character': "Mariah's Assistant", 'gender': 0}, {'name': 'Todd Holland', 'character': 'Fred', 'gender': 1}, {'name': 'Kevin Grady', 'character': 'Coleman', 'gender': 1}, {'name': 'Bobby Tisdale', 'character': 'Skizzy', 'gender': 2}, {'name': 'Herzl Tobey', 'character': 'Commander', 'gender': 1}, {'name': 'Ori Pfeffer', 'character': 'Second Commander', 'gender': 1}, {'name': 'Alex Luria', 'character': 'Koby', 'gender': 2}, {'name': 'Guy Oseary', 'character': 'Avi', 'gender': 1}, {'name': 'Yinon Sapir', 'character': 'Danny', 'gender': 2}, {'name': 'Naama Nativ', 'character': 'Sara', 'gender': 0}, {'name': 'Dennis Dugan', 'character': 'Homeless Guy', 'gender': 1}, {'name': 'Gerry Del Sol', 'character': 'Exploded Shop Owner', 'gender': 2}, {'name': 'Shulie Cowen', 'character': 'Debbie', 'gender': 0}, {'name': 'Maysoon Zayid', 'character': 'Nadira', 'gender': 0}, {'name': 'Helen Siff', 'character': 'Mrs. Skitzer', 'gender': 0}, {'name': 'Cynthia Frost', 'character': 'Mrs. Paulson', 'gender': 0}, {'name': 'Anna Berger', 'character': 'Older Lady in Salon', 'gender': 0}, {'name': 'Susan Grace', 'character': 'Older Lady in Salon', 'gender': 0}, {'name': 'Marjorie Loomis', 'character': 'Older Lady in Salon', 'gender': 0}, {'name': 'Carol Schlanger', 'character': 'Older Lady in Salon', 'gender': 0}, {'name': 'Edith S. Wolfrey', 'character': 'Older Lady in Salon', 'gender': 0}, {'name': 'Phoebe Dorin', 'character': 'Older Lady in Salon', 'gender': 0}, {'name': 'Lina So', 'character': 'Hip Salon Receptionist', 'gender': 0}, {'name': 'Eloise DeJoria', 'character': 'Blonde at Salon', 'gender': 0}, {'name': 'Vanessa Long', 'character': 'Catty Hairdresser', 'gender': 0}, {'name': 'Donielle Artese', 'character': 'African American Salon Owner', 'gender': 2}, {'name': 'Blake Bertrand', 'character': 'Little Boy', 'gender': 1}, {'name': 'Sid Ganis', 'character': 'Doctor', 'gender': 1}, {'name': 'Laurie Meghan Phelps', 'character': 'Plus Size Girl at Disco', 'gender': 0}, {'name': 'Kenneth Greenaway', 'character': 'DJ', 'gender': 2}, {'name': 'Constance Barron', 'character': 'First Woman in Cab', 'gender': 0}, {'name': 'Kristen Lowman', 'character': 'First Woman in Cab', 'gender': 0}, {'name': 'Penelope Windust', 'character': 'Second Woman in Cab', 'gender': 0}, {'name': 'Kathleen Noone', 'character': 'Second Woman in Cab', 'gender': 0}, {'name': 'Christopher Innvar', 'character': 'Angry Tall Driver', 'gender': 2}, {'name': 'Ray Garvey', 'character': 'Truck Driver', 'gender': 1}, {'name': 'Barbara Ann Davison', 'character': 'Dog Owner', 'gender': 0}, {'name': 'Eric Lamonsoff', 'character': "Hamdi's Passenger", 'gender': 2}, {'name': 'Connor Wiles', 'character': 'Boy Customer', 'gender': 2}, {'name': 'Jennifer De Minco', 'character': 'Arguing Female Customer', 'gender': 0}, {'name': 'Nicole Bennett', 'character': "Walbridge's Girlfriend", 'gender': 0}, {'name': 'Edmund Lyndeck', 'character': 'Pharmacist', 'gender': 1}, {'name': 'Lily Javaherpour', 'character': 'Inaz', 'gender': 0}, {'name': 'Kristina Haddad', 'character': "Hamdi's Wife (voice)", 'gender': 0}, {'name': 'Larry Marko', 'character': "Phantom's Trainer", 'gender': 2}, {'name': 'Anne Marie Howard', 'character': 'Reporter', 'gender': 0}, {'name': 'Veerta Motiani', 'character': 'Beach Girl', 'gender': 0}, {'name': 'Billy Concha', 'character': 'Kayaker', 'gender': 2}, {'name': 'Marco Khan', 'character': 'Terrorist with Hand', 'gender': 1}, {'name': 'Sam Sagheb', 'character': 'Disco Merchant', 'gender': 2}, {'name': 'Latif Marotti', 'character': 'Disco Merchant', 'gender': 2}, {'name': 'Christian Reeve', 'character': 'Disco Merchant', 'gender': 2}, {'name': 'Aesop Aquarian', 'character': 'Rabbi', 'gender': 2}, {'name': 'Michelle Arenal', 'character': "Girl on Zohan's Shoulders", 'gender': 0}, {'name': 'Jordan Ashbrook', 'character': 'Hot Girl in Club', 'gender': 0}, {'name': 'Moran Atias', 'character': 'Eti', 'gender': 0}, {'name': 'Ashley Ausburn', 'character': 'Girl at Mall', 'gender': 0}, {'name': 'Barbara Jean Barrielle', 'character': 'Cheerleader', 'gender': 0}, {'name': 'Mike Basone', 'character': 'Isreali Fan', 'gender': 2}, {'name': 'Mike Batayeh', 'character': 'Fry Cook', 'gender': 1}, {'name': 'Robert Benny', 'character': 'Autograph Seeker #1', 'gender': 2}, {'name': 'Bart Blackburn', 'character': 'Isreali Hacky Sack Coach', 'gender': 2}, {'name': 'Denise Bradley', 'character': 'Hackey Sack Enthusiast', 'gender': 0}, {'name': 'Guile Branco', 'character': 'Isreali Soldier', 'gender': 2}, {'name': 'Lamar Brown', 'character': 'Businessman', 'gender': 2}, {'name': 'Melanie Bulujian', 'character': 'Isreali Dancer', 'gender': 0}, {'name': 'Monica Cabral', 'character': 'Fizzy-Bubbeleh Girl', 'gender': 0}, {'name': 'Kevin Cannon', 'character': 'Times Square Crazy Man', 'gender': 2}, {'name': 'Ella Christopher', 'character': "Phantom's Wife", 'gender': 0}, {'name': 'Nick Clark', 'character': 'Nick', 'gender': 1}, {'name': 'Cheryl Cosenza', 'character': 'Ribbon Cutting Girl', 'gender': 0}, {'name': 'Rossie Cottrell', 'character': "Phantom's Wife", 'gender': 0}, {'name': 'John Dardenne', 'character': 'Isreali Hacky Sack Fan', 'gender': 2}, {'name': 'Joseph DeLuca', 'character': 'Hot Dog Vendor', 'gender': 2}, {'name': 'Kendall Draughn', 'character': 'Picnic Girl', 'gender': 0}, {'name': 'John Duerler', 'character': 'Electronic Store Customer', 'gender': 1}, {'name': 'Marshall Factora', 'character': 'Airline Terminal Passenger', 'gender': 2}, {'name': 'Eric Feliciano', 'character': 'Isreali Fan', 'gender': 2}, {'name': 'Ian Fisher', 'character': 'Disco Merchant', 'gender': 1}, {'name': 'Anna Garcia Williams', 'character': "Phantom's Wife", 'gender': 0}, {'name': 'Maite Garcia', 'character': "Phantom's Wife (uncredited)", 'gender': 2}, {'name': 'Leslie Garza', 'character': 'Disco Hip Girl', 'gender': 0}, {'name': 'Jessica Mei Gershen', 'character': 'Girl in Club', 'gender': 0}, {'name': 'Nikki Giavasis', 'character': 'Waitress', 'gender': 0}, {'name': 'Ronald E. Giles', 'character': 'Commuter', 'gender': 2}, {'name': 'Sharlene Grover', 'character': 'Isreali Soldier', 'gender': 0}, {'name': 'Sepideh Haftgoli', 'character': 'Isreali Soldier', 'gender': 0}, {'name': 'Zahra Heydari', 'character': 'Dalohan Receptionist', 'gender': 0}, {'name': 'Seth Howard', 'character': 'Party Guy', 'gender': 2}, {'name': 'Kevin James', 'character': 'Hacky Sack Tournament Celebrity Judge', 'gender': 1}, {'name': 'Paul Jeans', 'character': 'Hotel Manager', 'gender': 2}, {'name': 'Dave Johnson', 'character': 'Spectator', 'gender': 1}, {'name': 'Todd Justin', 'character': 'Nightclub Bouncer', 'gender': 2}, {'name': 'Sam Kalidi', 'character': 'Amir', 'gender': 2}, {'name': 'Sona Karia', 'character': 'Reporter', 'gender': 0}, {'name': 'Rock Kohli', 'character': 'Vendor with No Fizzi Bubblech', 'gender': 1}, {'name': 'Mark Kubr', 'character': 'Criminal #3', 'gender': 1}, {'name': 'Timothy Dean Lee', 'character': 'Shocked Businessman', 'gender': 1}, {'name': 'Paula Lemes', 'character': 'Go Go Dancer', 'gender': 0}, {'name': 'Bernadette Lords', 'character': 'Businesswoman', 'gender': 0}, {'name': 'Anthony Martins', 'character': 'Arab #3', 'gender': 1}, {'name': 'Roni Meron', 'character': 'Hot Girl / Waitress', 'gender': 0}, {'name': 'Nicole Montano', 'character': 'Cheerleader', 'gender': 0}, {'name': 'Rachel Mower', 'character': 'Girl in Park', 'gender': 0}, {'name': 'Rachael Murphy', 'character': 'Girl in Club', 'gender': 0}, {'name': 'Bibiana Navas', 'character': "Phantom's Wife", 'gender': 0}, {'name': 'Christina Neferis', 'character': 'Hacky Sack Fan', 'gender': 0}, {'name': 'Adam Ohl', 'character': 'Street Merchant', 'gender': 2}, {'name': 'Dianne Perry', 'character': "Phantom's Wife", 'gender': 0}, {'name': 'John Radcliff', 'character': 'Redneck Posse Man #2', 'gender': 2}, {'name': 'Klavdia Ramnareine', 'character': 'The Hot Chick', 'gender': 0}, {'name': 'Deborah Rombaut', 'character': 'Snooty French Girl at the Club', 'gender': 0}, {'name': 'Sandra Rosko', 'character': 'Middle Eastern Girl', 'gender': 0}, {'name': 'Rob Shapiro', 'character': 'Score Keeper', 'gender': 2}, {'name': 'Mikael Sharafyan', 'character': 'Fan in the Crowd', 'gender': 1}, {'name': 'Jhennah Sinclaire', 'character': 'Dancer', 'gender': 0}, {'name': 'Terrie Snell', 'character': 'Woman in Beauty Shop', 'gender': 0}, {'name': 'Chris Spinelli', 'character': 'Angry Palestinian', 'gender': 2}, {'name': 'Arne Starr', 'character': 'Isreali Weapons Trainer', 'gender': 2}, {'name': 'Riley Thomas Stewart', 'character': 'Little Boy #2', 'gender': 2}, {'name': 'Lee Stickler', 'character': 'Isreali Soldier', 'gender': 2}, {'name': 'Sarah Stringer', 'character': 'Party Goer', 'gender': 0}, {'name': 'Christa Texeira', 'character': 'Ruth', 'gender': 2}, {'name': 'Keana Texeira', 'character': 'Rock-throwing Girl', 'gender': 0}, {'name': 'Seth Texeira', 'character': 'Zeb', 'gender': 2}, {'name': 'Sophie Texeira', 'character': 'Alena', 'gender': 2}, {'name': 'Ryan Phillip Thomas', 'character': 'Angry Palestinian', 'gender': 2}, {'name': 'Gina Tleel', 'character': 'Shopper', 'gender': 0}, {'name': 'Uzimann', 'character': 'Cafe Customer', 'gender': 2}, {'name': 'Kevin Van Doorslaer', 'character': 'Isreali Soldier', 'gender': 1}, {'name': 'Dave Vij', 'character': 'Disco-Tech', 'gender': 2}, {'name': 'Holly Weber', 'character': 'Girlfriend', 'gender': 0}, {'name': 'Molly Weiner', 'character': 'Salon Patron', 'gender': 0}, {'name': 'Joe Van Wie', 'character': 'Hot Dog Patron', 'gender': 2}, {'name': 'Henry Winkler', 'character': 'Limousine Passenger', 'gender': 1}, {'name': 'Mark Winn', 'character': 'News Cameraman', 'gender': 2}, {'name': 'Saye Yabandeh', 'character': 'Girlfriend', 'gender': 0}, {'name': 'Julius Zagon', 'character': 'Ukrossi', 'gender': 2}, {'name': 'David Zahedian', 'character': "Phantom's Best Friend", 'gender': 2}, {'name': 'Maria Zambrana', 'character': 'Taxi Cab Patron', 'gender': 0}, {'name': 'Betty Murphy', 'character': 'Mrs. Haines', 'gender': 0}], [{'name': 'Dane DeHaan', 'character': 'Valerian', 'gender': 1}, {'name': 'Cara Delevingne', 'character': 'Laureline', 'gender': 0}, {'name': 'Clive Owen', 'character': 'Commander Arün Filitt', 'gender': 1}, {'name': 'Rihanna', 'character': 'Bubble', 'gender': 0}, {'name': 'Ethan Hawke', 'character': 'Jolly the Pimp', 'gender': 1}, {'name': 'Herbie Hancock', 'character': 'Defence Minister', 'gender': 1}, {'name': 'Kris Wu', 'character': 'Sergeant Neza', 'gender': 1}, {'name': 'Rutger Hauer', 'character': 'President of the World State Federation', 'gender': 1}, {'name': 'John Goodman', 'character': 'Igon Siruss (voice)', 'gender': 1}, {'name': 'Elizabeth Debicki', 'character': 'Emperor Haban-Limaï (voice)', 'gender': 0}, {'name': 'Sam Spruell', 'character': 'General Okto Bar', 'gender': 1}, {'name': 'Ola Rapace', 'character': 'Major Gibson', 'gender': 1}, {'name': 'Alain Chabat', 'character': 'Bob the Pirate', 'gender': 1}, {'name': 'Thom Findlay', 'character': 'Pirates (voice)', 'gender': 2}, {'name': 'Mathieu Kassovitz', 'character': 'Hawker', 'gender': 1}, {'name': 'Aymeline Valade', 'character': 'Emperor Haban-Limaï', 'gender': 2}, {'name': 'Sasha Luss', 'character': "Princess Lïhio-Minaa / Creature 'Candy Dress'", 'gender': 2}, {'name': 'Pauline Hoarau', 'character': "Empress Aloï / Creature 'Swing Angel'", 'gender': 0}, {'name': 'Jonas Bloquet', 'character': 'K-Tron Warrior / Control Room Soldier', 'gender': 1}, {'name': 'Sand Van Roy', 'character': "Creature 'Jessica Rabbit'", 'gender': 2}, {'name': 'Louis Leterrier', 'character': 'Captain Welcoming Mercurys', 'gender': 1}, {'name': 'Olivier Megaton', 'character': 'Captain Welcoming KCO2', 'gender': 1}, {'name': 'Sam Douglas', 'character': 'Male Tourist', 'gender': 1}, {'name': 'Diva Cam', 'character': "Mâatri (Tsûuri's Friend)", 'gender': 2}, {'name': 'Eric Lampaert', 'character': 'Guide Thaziit', 'gender': 2}, {'name': 'Alexandre Willaume', 'character': 'Captain Kris', 'gender': 1}, {'name': 'Gavin Drea', 'character': 'Sergent Cooper', 'gender': 2}, {'name': 'Réginal Kudiwu', 'character': 'Major Samk', 'gender': 2}, {'name': 'David Michie', 'character': 'Pearl Tsuuri (voice)', 'gender': 1}, {'name': 'Marion Roussignol', 'character': 'Pearl', 'gender': 2}, {'name': 'Emilie Pommelet', 'character': 'Pearl', 'gender': 2}, {'name': 'Roman Blomme', 'character': 'Boulan Bathor', 'gender': 1}, {'name': 'Emilie Livingston', 'character': 'Bubble Dancer', 'gender': 2}, {'name': 'Alexandre Nguyen', 'character': 'Japanese Delegate', 'gender': 2}, {'name': 'Aurelien Gaya', 'character': 'Igon Sirus Guard', 'gender': 2}, {'name': 'Mikael Buxton', 'character': 'Paradise Alley Soldier', 'gender': 2}, {'name': 'Yun Lai', 'character': 'Valley Chinois', 'gender': 2}, {'name': 'Anders Heinrichsen', 'character': 'Police Officer', 'gender': 2}, {'name': 'Tristan Robin', 'character': 'Roper', 'gender': 1}, {'name': 'Yun-Ping He', 'character': 'Controller of the Technical Chamber', 'gender': 1}, {'name': 'James Flynn', 'character': 'Guide #2', 'gender': 2}, {'name': 'David Clark', 'character': 'Astronaut', 'gender': 2}, {'name': 'Ben Mauro', 'character': 'Commander Arün Filitt Guard', 'gender': 2}, {'name': 'Jean-François Lenogue', 'character': 'Igon Sirus Guard', 'gender': 2}, {'name': 'Peter Eberst', 'character': 'Igon Sirus Guard', 'gender': 2}, {'name': 'Peter Hudson', 'character': 'Captain Crowford', 'gender': 2}, {'name': 'Xavier Giannoli', 'character': 'Captain Norton', 'gender': 1}, {'name': 'Eric Rochant', 'character': 'Captain Welcoming Palm Müret', 'gender': 1}, {'name': 'Benoît Jacquot', 'character': 'Captain Welcoming Arysum', 'gender': 1}, {'name': 'Gérard Krawczyk', 'character': 'Captain Welcoming Martapuraïs', 'gender': 1}, {'name': 'Pierre Cachia', 'character': 'Kortan Dahük', 'gender': 2}, {'name': 'David Saada', 'character': 'Kortan Dahük', 'gender': 1}, {'name': 'Hippolyte Burkhart-Uhlen', 'character': 'Kortan Dahük', 'gender': 2}, {'name': 'Florian Guichard', 'character': 'Mercury', 'gender': 2}, {'name': 'Stéphane Mir', 'character': 'Mercury', 'gender': 2}, {'name': 'Thierry Barthe', 'character': 'Mercury', 'gender': 2}, {'name': 'Pier Ewudu', 'character': 'Palm Müret', 'gender': 2}, {'name': 'Andrew Tisba', 'character': 'Palm Müret', 'gender': 2}, {'name': 'Yannick Lorté', 'character': 'Palm Müret', 'gender': 2}, {'name': 'Charly Akakpo', 'character': 'Arysum-Kormn', 'gender': 2}, {'name': 'Clément Beauruelle', 'character': 'Arysum-Kormn', 'gender': 2}, {'name': 'Audrey Kamp', 'character': 'Arysum-Kormn', 'gender': 2}, {'name': 'Anthony Hornez', 'character': 'Martapuraï', 'gender': 2}, {'name': 'Julien Bleitrach', 'character': 'Martapuraï', 'gender': 2}, {'name': 'Maxime Babara Touré', 'character': 'Martapuraï', 'gender': 2}, {'name': 'Barbare Weber Scaff', 'character': 'Empress Aloï (voice)', 'gender': 2}, {'name': 'Marilhéa Peillard', 'character': 'Tsûuri', 'gender': 2}, {'name': 'Lee Delong', 'character': 'Tsûuri / Tourist (voice)', 'gender': 2}, {'name': 'Judith Brunett', 'character': "Mâatri (Tsûuri's Friend) (voice)", 'gender': 2}, {'name': 'Marie Barrouillet', 'character': 'Tsûuri Child', 'gender': 2}, {'name': 'Cindy Bruna', 'character': 'Guard', 'gender': 0}, {'name': 'Sija Titko', 'character': 'Guard', 'gender': 2}, {'name': 'Daphnée Kbidi', 'character': 'Tortured Pearl', 'gender': 2}, {'name': 'Nick Marzock', 'character': 'Tortured Pearl (voice)', 'gender': 1}, {'name': 'Stefan Konarske', 'character': 'Captain Zito', 'gender': 1}, {'name': 'Mahamadou Coulibaly', 'character': "Major Gibon's Soldier / Igon Junio", 'gender': 2}, {'name': 'Julien Marlin', 'character': "Major Gibon's Soldier", 'gender': 2}, {'name': 'Yvan Lucker', 'character': "Major Gibon's Soldier", 'gender': 2}, {'name': 'Tom Hygreck', 'character': "Major Gibon's Soldier", 'gender': 2}, {'name': 'Abel Jafri', 'character': 'Bus Driver', 'gender': 1}, {'name': 'Laurent Ferraro', 'character': 'Igon Siruss / Doghan Dagui', 'gender': 2}, {'name': 'Jean-Robert Lombard', 'character': 'Igon Sirus Guard', 'gender': 2}, {'name': 'Leonid Glushchenko', 'character': 'Igon Sirus Guard', 'gender': 2}, {'name': 'Akim Chir', 'character': 'Igon Sirus Guard', 'gender': 1}, {'name': 'Laminé Ba', 'character': 'Igon Sirus Guard', 'gender': 2}, {'name': 'Paul Lefèvre', 'character': 'Siirt Guard / Siirt Salesman', 'gender': 1}, {'name': 'Guillaume Maison', 'character': 'Siirt Salesman', 'gender': 2}, {'name': 'Jean-Christophe Brizard', 'character': 'Siirt Salesman', 'gender': 2}, {'name': 'Patrick Cottet-Moine', 'character': 'Siirt Cop', 'gender': 1}, {'name': 'Irene Palko', 'character': 'Female Tourist', 'gender': 2}, {'name': 'Tonio Descanvelle', 'character': 'DA / Doghan-Dagui', 'gender': 2}, {'name': "Velvet D'Amour", 'character': 'Tourist', 'gender': 0}, {'name': 'Doug Rand', 'character': 'Doghan-Dagui', 'gender': 2}, {'name': 'Grant Moninger', 'character': 'Doghan-Dagui (voice)', 'gender': 2}, {'name': 'Robbie Rist', 'character': 'Doghan-Dagui (voice)', 'gender': 1}, {'name': 'Christopher Swindel', 'character': 'Doghan-Dagui (voice)', 'gender': 2}, {'name': 'Claire Tran', 'character': 'Bubble / Glamopod / Control Room Sergeant', 'gender': 0}, {'name': 'Alain Etoundi', 'character': 'Club Bouncer', 'gender': 2}, {'name': 'Fayet Nsumoto', 'character': 'Club Bouncer', 'gender': 2}, {'name': 'Kristina Kachinskaya', 'character': "Creature 'Twin'", 'gender': 2}, {'name': 'Veronika Khayla', 'character': "Creature 'Twin'", 'gender': 2}, {'name': 'Sissi Duparc', 'character': "Creature 'Louis XVI'", 'gender': 2}, {'name': 'Noam Frost', 'character': "Creature 'Bird Woman'", 'gender': 2}, {'name': 'Pauline Serreau', 'character': "Creature 'Tattoo'", 'gender': 2}, {'name': 'Philippe Rigot', 'character': 'Hero Merchant', 'gender': 2}, {'name': 'Étienne Ménard', 'character': 'Red Zone Captain', 'gender': 2}, {'name': 'Peter Lamarque', 'character': 'Warship Captain', 'gender': 2}, {'name': 'Maximilien Seweryn', 'character': 'K-Tron Captain', 'gender': 2}, {'name': 'Zouheir Zerhouni', 'character': 'K-Tron Warrior', 'gender': 2}, {'name': 'Tristan Zerbib', 'character': 'K-Tron Warrior', 'gender': 2}, {'name': 'Elliot Jenicot', 'character': 'Interrogation Room Captain', 'gender': 2}, {'name': 'Staiv Gentis', 'character': 'Control Room Soldier', 'gender': 2}, {'name': 'Chloé Hollings', 'character': "Alex-Intruder's Voice (voice)", 'gender': 0}, {'name': 'Aguendia Fotabong', 'character': 'Boulan Bathor Emperor', 'gender': 2}, {'name': 'Tania Dessources', 'character': "Boulan Bathor Emperor's Wife", 'gender': 2}, {'name': 'Gaëtan Cotigny', 'character': 'Fisherman', 'gender': 2}, {'name': 'François Bredon', 'character': 'Fisherman', 'gender': 2}, {'name': 'Cédric Chevalme', 'character': 'Bubble / Boulan-Bathor / Valerian', 'gender': 2}], [{'name': 'David Duchovny', 'character': 'Fox Mulder', 'gender': 1}, {'name': 'Gillian Anderson', 'character': 'Dana Scully', 'gender': 0}, {'name': 'Amanda Peet', 'character': 'ASAC Dakota Whitney', 'gender': 0}, {'name': 'Billy Connolly', 'character': 'Father Joseph Crissman', 'gender': 1}, {'name': 'Xzibit', 'character': 'Agent Mosley Drummy', 'gender': 1}, {'name': 'Mitch Pileggi', 'character': 'Walter Skinner', 'gender': 1}, {'name': 'Callum Keith Rennie', 'character': '2nd Abductor - Janke Dacyshyn', 'gender': 1}, {'name': 'Adam Godley', 'character': 'Father Ybarra', 'gender': 1}, {'name': 'Alex Diakun', 'character': 'Gaunt Man', 'gender': 1}, {'name': 'Nicki Aycox', 'character': '2nd Victim - Cheryl Cunningham', 'gender': 0}, {'name': 'Fagin Woodcock', 'character': '1st Abductor - Franz Tomczeszyn', 'gender': 2}, {'name': 'Marco Niccoli', 'character': 'Christian Fearon', 'gender': 1}, {'name': 'Carrie Ruscheinsky', 'character': 'Margaret Fearon', 'gender': 0}, {'name': 'Spencer Maybee', 'character': 'Blair Fearon', 'gender': 1}, {'name': 'Veronika Hadrava', 'character': 'Female Assistant', 'gender': 2}, {'name': 'Denis Krasnogolov', 'character': 'Male Assistant', 'gender': 2}, {'name': 'Patrick Keating', 'character': 'Slight Man', 'gender': 2}, {'name': 'Roger Horchow', 'character': 'Elderly Gent', 'gender': 2}, {'name': 'Stephen E. Miller', 'character': 'Feed Store Proprietor', 'gender': 1}, {'name': 'Xantha Radley', 'character': 'Monica Bannan', 'gender': 0}, {'name': 'Lorena Gale', 'character': 'On Screen Doctor', 'gender': 0}, {'name': 'Donavon Stinson', 'character': 'Suited Man', 'gender': 1}, {'name': 'Dion Johnstone', 'character': '1st Cop', 'gender': 2}, {'name': 'Sarah-Jane Redmond', 'character': 'Special Agent in Charge', 'gender': 0}, {'name': "Christina D'Alimonte", 'character': "Doctor's Colleague", 'gender': 2}, {'name': 'Vanesa Tomasino', 'character': 'Hallway Agent', 'gender': 2}, {'name': 'Luvia Petersen', 'character': 'O.R. Nurse', 'gender': 0}, {'name': 'Babs Chula', 'character': 'Surgeon', 'gender': 0}, {'name': 'Marci T. House', 'character': 'Sheriff', 'gender': 0}, {'name': 'Joseph Patrick Finn', 'character': 'Whispering Priest', 'gender': 2}, {'name': 'Beth Siegler', 'character': 'Anesthesiologist', 'gender': 2}, {'name': 'Stacee Copeland', 'character': 'Doctor', 'gender': 0}, {'name': 'Tom Charron', 'character': 'Sheriff Horton', 'gender': 2}, {'name': "Brent C.S. O'Connor", 'character': 'Tow Truck Driver', 'gender': 2}, {'name': 'Dave Cote', 'character': 'D-Man', 'gender': 1}, {'name': 'Chris Carter', 'character': 'Man sitting in hospital hallway', 'gender': 1}, {'name': 'Paul Mitton', 'character': 'FBI Agent', 'gender': 2}, {'name': 'Vanessa Morley', 'character': 'Female Hallway Agent', 'gender': 2}, {'name': 'Michael Stevens', 'character': 'Board Room Doctor', 'gender': 2}, {'name': 'Lynne Burnett', 'character': 'Murder Victim (uncredited)', 'gender': 2}, {'name': 'Celeste Insell', 'character': 'Board Room Doctor (uncredited)', 'gender': 2}, {'name': 'Teana-Marie Smith', 'character': 'Nurse (uncredited)', 'gender': 2}], [{'name': 'Hugh Jackman', 'character': 'Jean Valjean', 'gender': 1}, {'name': 'Russell Crowe', 'character': 'Javert', 'gender': 1}, {'name': 'Anne Hathaway', 'character': 'Fantine', 'gender': 0}, {'name': 'Amanda Seyfried', 'character': 'Cosette', 'gender': 0}, {'name': 'Eddie Redmayne', 'character': 'Marius Pontmercy', 'gender': 1}, {'name': 'Helena Bonham Carter', 'character': 'Madame Thénardier', 'gender': 0}, {'name': 'Sacha Baron Cohen', 'character': 'Monsieur Thénardier', 'gender': 1}, {'name': 'Aaron Tveit', 'character': 'Enjolras', 'gender': 1}, {'name': 'Samantha Barks', 'character': 'Éponine Thénardier', 'gender': 0}, {'name': 'Daniel Huttlestone', 'character': 'Gavroche', 'gender': 1}, {'name': 'Cavin Cornwall', 'character': 'Convict 1', 'gender': 2}, {'name': 'Josef Altin', 'character': 'Convict 2', 'gender': 1}, {'name': 'David Hawley', 'character': 'Convict 3', 'gender': 2}, {'name': 'Adam Jones', 'character': 'Convict 4', 'gender': 2}, {'name': 'John Barr', 'character': 'Convict 5', 'gender': 2}, {'name': 'Tony Rohr', 'character': 'Overseer', 'gender': 1}, {'name': 'Richard Dixon', 'character': 'Mairie Officer', 'gender': 1}, {'name': 'Andy Beckwith', 'character': 'Innkeeper', 'gender': 2}, {'name': 'Colm Wilkinson', 'character': 'Bishop', 'gender': 1}, {'name': 'Stephen Bent', 'character': 'Jailer', 'gender': 2}, {'name': 'Georgie Glen', 'character': 'Madame Baptistine', 'gender': 0}, {'name': 'Heather Chasen', 'character': 'Madame Magloire', 'gender': 2}, {'name': 'Paul Thornley', 'character': 'Constable 1', 'gender': 2}, {'name': 'Paul Howell', 'character': 'Constable 2', 'gender': 2}, {'name': 'Stephen Tate', 'character': 'Fauchelevent', 'gender': 2}, {'name': 'Michael Jibson', 'character': 'Foreman', 'gender': 1}, {'name': 'Kate Fleetwood', 'character': 'Factory Woman 1', 'gender': 2}, {'name': 'Hannah Waddingham', 'character': 'Factory Woman 2', 'gender': 0}, {'name': 'Clare Foster', 'character': 'Factory Woman 3', 'gender': 0}, {'name': 'Kirsty Hoiles', 'character': 'Factory Woman 4', 'gender': 2}, {'name': 'Jenna Boyd', 'character': 'Factory Woman 5', 'gender': 2}, {'name': 'Alice Fearn', 'character': 'Factory Woman 6', 'gender': 2}, {'name': 'Alison Tennant', 'character': 'Factory Woman 7', 'gender': 2}, {'name': 'Marilyn Cutts', 'character': 'Factory Woman 8', 'gender': 2}, {'name': 'Cathy Breeze', 'character': 'Factory Woman 9', 'gender': 0}, {'name': 'John Albasiny', 'character': 'Factory Onlooker', 'gender': 1}, {'name': 'Bertie Carvel', 'character': 'Bamatabois', 'gender': 2}, {'name': 'Tim Downie', 'character': 'Brevet', 'gender': 1}, {'name': 'Andrew Havill', 'character': 'Cochepaille', 'gender': 1}, {'name': 'Dick Ward', 'character': "Bamatabois' Valet", 'gender': 1}, {'name': 'Nicola Sloane', 'character': 'Hair Crone', 'gender': 0}, {'name': 'Daniel Evans', 'character': 'Pimp', 'gender': 2}, {'name': 'David Stoller', 'character': 'Sailor 1', 'gender': 2}, {'name': 'Ross McCormack', 'character': 'Sailor 2', 'gender': 2}, {'name': 'Jaygann Ayeh', 'character': 'Sailor 3', 'gender': 2}, {'name': 'Adrian Scarborough', 'character': 'Toothman', 'gender': 1}, {'name': 'Frances Ruffelle', 'character': 'Whore 1', 'gender': 2}, {'name': 'Lynne Wilmot', 'character': 'Whore 2', 'gender': 2}, {'name': 'Charlotte Spencer', 'character': 'Whore 3', 'gender': 2}, {'name': 'Julia Worsley', 'character': 'Head Whore', 'gender': 2}, {'name': 'Keith Dunphy', 'character': 'Client', 'gender': 1}, {'name': 'Ashley Artus', 'character': 'Pawn Broker', 'gender': 1}, {'name': 'John Surman', 'character': 'Organ Grinder', 'gender': 2}, {'name': 'David Cann', 'character': 'Magistrate', 'gender': 1}, {'name': 'James Simmons', 'character': 'Champmathieu', 'gender': 2}, {'name': 'Polly Kemp', 'character': 'Nursing Sister', 'gender': 2}, {'name': 'Ian Pirie', 'character': 'Babet', 'gender': 1}, {'name': 'Adam Pearce', 'character': 'Brujon', 'gender': 2}, {'name': 'Julian Bleach', 'character': 'Claquesous', 'gender': 1}, {'name': 'Marc Pickering', 'character': 'Montparnasse', 'gender': 1}, {'name': 'Isabelle Allen', 'character': 'Young Cosette', 'gender': 0}, {'name': 'Natalya Wallace', 'character': 'Young Éponine', 'gender': 0}, {'name': 'Phil Snowden', 'character': 'Customer 1', 'gender': 2}, {'name': 'Hadrian Delacey', 'character': 'Customer 2', 'gender': 2}, {'name': 'Lottie Steer', 'character': 'Baby', 'gender': 2}, {'name': 'Sam Parks', 'character': 'Tall Customer', 'gender': 2}, {'name': 'Mark Donovan', 'character': 'Portly Customer', 'gender': 2}, {'name': 'Lewis Kirk', 'character': 'Handsome Soldier 1', 'gender': 2}, {'name': 'Leighton Rafferty', 'character': 'Handsome Soldier 2', 'gender': 2}, {'name': 'Peter Mair', 'character': 'Father Christmas 1', 'gender': 2}, {'name': 'Jack Chissick', 'character': 'Father Christmas 2', 'gender': 2}, {'name': 'Dianne Pilkington', 'character': 'Inn Whore 1', 'gender': 0}, {'name': 'Robyn North', 'character': 'Inn Whore 2', 'gender': 2}, {'name': 'Norma Atallah', 'character': 'Mother Whore', 'gender': 2}, {'name': 'Patrick Godfrey', 'character': 'Gillenormand', 'gender': 1}, {'name': 'Mark Roper', 'character': 'Citizen', 'gender': 1}, {'name': 'Paul Leonard', 'character': 'Citizen', 'gender': 1}, {'name': 'Miles Roughley', 'character': "Gavroche's Urchin", 'gender': 2}, {'name': 'Cameron Strefford', 'character': "Gavroche's Urchin", 'gender': 2}, {'name': 'Alfie Davis', 'character': "Gavroche's Urchin", 'gender': 2}, {'name': 'Joseph West', 'character': "Gavroche's Urchin", 'gender': 2}, {'name': 'Joel Phillimore', 'character': 'Beggar 1', 'gender': 1}, {'name': 'Jacqueline Dankworth', 'character': 'Beggar 2', 'gender': 2}, {'name': 'Amelia Jefford', 'character': 'Beggar 3', 'gender': 2}, {'name': 'Chris Barnes', 'character': 'Beggar 4', 'gender': 2}, {'name': 'Richard Cordery', 'character': 'Duc De Raguse', 'gender': 1}, {'name': 'Killian Donnelly', 'character': 'Combeferre', 'gender': 2}, {'name': 'Fra Fee', 'character': 'Courfeyrac', 'gender': 2}, {'name': 'Gabriel Vick', 'character': 'Feuilly', 'gender': 2}, {'name': 'George Blagden', 'character': 'Grantaire', 'gender': 1}, {'name': 'Hugh Skinner', 'character': 'Joly', 'gender': 2}, {'name': 'Stuart Neal', 'character': 'Lesgles', 'gender': 2}, {'name': 'Alistair Brammer', 'character': 'Prouvaire', 'gender': 1}, {'name': 'Iwan Lewis', 'character': 'Bahorel', 'gender': 2}, {'name': 'Katy Secombe', 'character': 'Madame Hucheloup', 'gender': 2}, {'name': 'Hadley Fraser', 'character': 'Army Officer', 'gender': 1}, {'name': 'Linzi Hateley', 'character': 'Turning Woman 1', 'gender': 0}, {'name': 'Gemma Wardle', 'character': 'Turning Woman 2', 'gender': 2}, {'name': 'Gina Beck', 'character': 'Turning Woman 3', 'gender': 2}, {'name': 'Katie Hall', 'character': 'Turning Woman 4', 'gender': 2}, {'name': 'Lisa Hull', 'character': 'Turning Woman 5', 'gender': 2}, {'name': 'Andrea Deck', 'character': 'Turning Woman 6', 'gender': 2}, {'name': 'Jessica Duncan', 'character': 'Turning Woman 7', 'gender': 2}, {'name': 'Kerry Ingram', 'character': 'Turning Woman 8', 'gender': 0}, {'name': 'John Warnaby', 'character': 'Majordomo', 'gender': 1}, {'name': 'Mike Sarne', 'character': 'Father Mabeuf', 'gender': 2}, {'name': 'Freya Parks', 'character': 'Café Barmaid', 'gender': 0}, {'name': 'Richard Bremmer', 'character': 'Ensemble At The End of the Day', 'gender': 1}, {'name': 'Alexander Brooks', 'character': 'Ensemble At The End of the Day', 'gender': 2}, {'name': 'Eleanor Bruce', 'character': 'Ensemble At The End of the Day', 'gender': 2}, {'name': 'Emma Dukes', 'character': 'Ensemble At The End of the Day', 'gender': 2}, {'name': 'Stephen Matthews', 'character': 'Ensemble At The End of the Day', 'gender': 2}, {'name': 'Peter Saracen', 'character': 'Ensemble At The End of the Day', 'gender': 2}, {'name': 'Sebastian Sykes', 'character': 'Ensemble At The End of the Day', 'gender': 2}, {'name': 'Phil Zimmerman', 'character': 'Ensemble At The End of the Day', 'gender': 2}, {'name': 'Bessie Carter', 'character': 'Ensemble Factory Women', 'gender': 2}, {'name': 'Helen Cotterill', 'character': 'Ensemble Factory Women', 'gender': 0}, {'name': 'Tricia Deighton', 'character': 'Ensemble Factory Women', 'gender': 2}, {'name': 'Mandy Holliday', 'character': 'Ensemble Factory Women', 'gender': 2}, {'name': 'Charlotte Hope', 'character': 'Ensemble Factory Women', 'gender': 0}, {'name': 'Jackie Marks', 'character': 'Ensemble Factory Women', 'gender': 2}, {'name': 'Sara Pelosi', 'character': 'Ensemble Factory Women', 'gender': 2}, {'name': 'Mary Roscoe', 'character': 'Ensemble Factory Women', 'gender': 2}, {'name': 'Amelia Scaramucci', 'character': 'Ensemble Factory Women', 'gender': 2}, {'name': 'Caroline Sheen', 'character': 'Ensemble Factory Women', 'gender': 2}, {'name': 'Rachael Archer', 'character': 'Ensemble Lovely Ladies', 'gender': 2}, {'name': 'Lorna Brown', 'character': 'Ensemble Lovely Ladies', 'gender': 0}, {'name': 'Antonia Clarke', 'character': 'Ensemble Lovely Ladies', 'gender': 2}, {'name': 'Mary Cormack', 'character': 'Ensemble Lovely Ladies', 'gender': 2}, {'name': 'Sonya Cullingford', 'character': 'Ensemble Lovely Ladies', 'gender': 2}, {'name': 'Holly Dale Spencer', 'character': 'Ensemble Lovely Ladies', 'gender': 2}, {'name': 'Amy Griffiths', 'character': 'Ensemble Lovely Ladies', 'gender': 2}, {'name': 'Fania Grigoriou', 'character': 'Ensemble Lovely Ladies', 'gender': 2}, {'name': 'Amanda Henderson', 'character': 'Ensemble Lovely Ladies', 'gender': 2}, {'name': 'Alexia Khadime', 'character': 'Ensemble Lovely Ladies', 'gender': 2}, {'name': 'Luisa Lazzaro', 'character': 'Ensemble Lovely Ladies', 'gender': 2}, {'name': "Gemma O'Duffy", 'character': 'Ensemble Lovely Ladies', 'gender': 2}, {'name': "Rosa O'Reilly", 'character': 'Ensemble Lovely Ladies', 'gender': 2}, {'name': 'Amy Ellen Richardson', 'character': 'Ensemble Lovely Ladies', 'gender': 2}, {'name': 'Olivia Rose-Aaron', 'character': 'Ensemble Lovely Ladies', 'gender': 2}, {'name': 'Robyn Miranda Simpson', 'character': 'Ensemble Lovely Ladies', 'gender': 2}, {'name': 'Rachel Stanley', 'character': 'Ensemble Lovely Ladies / Ensemble Wedding Dancers', 'gender': 2}, {'name': 'Nancy Sullivan', 'character': 'Ensemble Lovely Ladies', 'gender': 2}, {'name': 'Rebecca Sutherland', 'character': 'Ensemble Lovely Ladies', 'gender': 2}, {'name': 'Tabitha Webb', 'character': 'Ensemble Lovely Ladies', 'gender': 2}, {'name': 'Gerard Bentall', 'character': 'Ensemble Master of the House', 'gender': 2}, {'name': 'Tony Bignell', 'character': 'Ensemble Master of the House', 'gender': 2}, {'name': 'Michael Cahill', 'character': 'Ensemble Master of the House', 'gender': 2}, {'name': 'Richard Colson', 'character': 'Ensemble Master of the House', 'gender': 2}, {'name': 'Kerry Ellis', 'character': 'Ensemble Master of the House', 'gender': 2}, {'name': 'Simon Fisher-Becker', 'character': 'Ensemble Master of the House', 'gender': 1}, {'name': 'Sarah Flind', 'character': 'Ensemble Master of the House', 'gender': 2}, {'name': 'Kelly-Anne Gower', 'character': 'Ensemble Master of the House', 'gender': 2}, {'name': 'James Greene', 'character': 'Ensemble Master of the House', 'gender': 1}, {'name': 'Nick Holder', 'character': 'Ensemble Master of the House', 'gender': 1}, {'name': 'Chris Howell', 'character': 'Ensemble Master of the House', 'gender': 2}, {'name': 'Alison Jiear', 'character': 'Ensemble Master of the House', 'gender': 2}, {'name': 'Terence Keely', 'character': 'Ensemble Master of the House', 'gender': 2}, {'name': 'Martin Marquez', 'character': 'Ensemble Master of the House', 'gender': 1}, {'name': 'Sally Mates', 'character': 'Ensemble Master of the House', 'gender': 2}, {'name': 'Jeff Nicholson', 'character': 'Ensemble Master of the House', 'gender': 2}, {'name': 'Adam Searles', 'character': 'Ensemble Master of the House', 'gender': 2}, {'name': 'Simon Shorten', 'character': 'Ensemble Master of the House', 'gender': 2}, {'name': 'Juliet Alderdice', 'character': 'Ensemble Paris Beggars', 'gender': 2}, {'name': 'Sean Buckley', 'character': 'Ensemble Paris Beggars', 'gender': 1}, {'name': 'Valerie Cutko', 'character': 'Ensemble Paris Beggars', 'gender': 2}, {'name': 'Spike Grimsey', 'character': 'Ensemble Paris Beggars', 'gender': 2}, {'name': 'Matt Harrop', 'character': 'Ensemble Paris Beggars', 'gender': 2}, {'name': 'Georgina Jackson', 'character': 'Ensemble Paris Beggars', 'gender': 2}, {'name': 'Perry Millward', 'character': 'Ensemble Paris Beggars', 'gender': 2}, {'name': 'Phil Philmar', 'character': 'Ensemble Paris Beggars', 'gender': 2}, {'name': 'Joyce Springer', 'character': 'Ensemble Paris Beggars', 'gender': 2}, {'name': 'Julie Stark', 'character': 'Ensemble Paris Beggars', 'gender': 2}, {'name': 'Dominic Applewhite', 'character': 'Ensemble Students', 'gender': 2}, {'name': 'Matthew Corner', 'character': 'Ensemble Students', 'gender': 2}, {'name': 'Andy Coxon', 'character': 'Ensemble Students', 'gender': 2}, {'name': 'Jonathan Dudley', 'character': 'Ensemble Students', 'gender': 2}, {'name': 'Rhidian Marc', 'character': 'Ensemble Students', 'gender': 2}, {'name': 'Chris Milford', 'character': 'Ensemble Students', 'gender': 2}, {'name': 'Jamie Muscato', 'character': 'Ensemble Students', 'gender': 2}, {'name': 'Joseph Peters', 'character': 'Ensemble Students', 'gender': 2}, {'name': 'David Roberts', 'character': 'Ensemble Students', 'gender': 1}, {'name': 'Stevee Davies', 'character': 'Ensemble Students', 'gender': 2}, {'name': 'Jonny Purchase', 'character': 'Ensemble Students', 'gender': 2}, {'name': 'Matt Seadon Young', 'character': 'Ensemble Students', 'gender': 2}, {'name': 'Jos Slovick', 'character': 'Ensemble Students', 'gender': 2}, {'name': 'Samuel J. Weir', 'character': 'Ensemble Students', 'gender': 2}, {'name': 'Sophie Huchinson', 'character': 'Ensemble Turning Women', 'gender': 2}, {'name': 'Ella Hunt', 'character': 'Ensemble Turning Women', 'gender': 0}, {'name': 'Claire Machin', 'character': 'Ensemble Turning Women', 'gender': 2}, {'name': 'Brenda Moore', 'character': 'Ensemble Turning Women', 'gender': 2}, {'name': 'Mischa Purnell', 'character': 'Ensemble Turning Women', 'gender': 2}, {'name': 'Annette Yeo', 'character': 'Ensemble Turning Women', 'gender': 2}, {'name': 'Josephine Darvill-Mills', 'character': 'Ensemble Wedding Dancers', 'gender': 2}, {'name': 'Jennifer Essex', 'character': 'Ensemble Wedding Dancers', 'gender': 2}, {'name': 'Vicky Evans', 'character': 'Ensemble Wedding Dancers', 'gender': 2}, {'name': 'Edward Lewis French', 'character': 'Ensemble Wedding Dancers', 'gender': 2}, {'name': 'Nigel Garton', 'character': 'Ensemble Wedding Dancers', 'gender': 2}, {'name': 'Lynn Jezzard', 'character': 'Ensemble Wedding Dancers', 'gender': 2}, {'name': 'Nicholas Keegan', 'character': 'Ensemble Wedding Dancers', 'gender': 2}, {'name': 'Steve Kirkham', 'character': 'Ensemble Wedding Dancers', 'gender': 2}, {'name': 'Vanessa Lee Hicks', 'character': 'Ensemble Wedding Dancers', 'gender': 2}, {'name': 'Ian Parsons', 'character': 'Ensemble Wedding Dancers', 'gender': 2}, {'name': 'Gemma Payne', 'character': 'Ensemble Wedding Dancers', 'gender': 2}, {'name': 'Clinten Pearce', 'character': 'Ensemble Wedding Dancers', 'gender': 2}, {'name': 'Claire Piquemal', 'character': 'Ensemble Wedding Dancers', 'gender': 2}, {'name': 'Aaron Sillis', 'character': 'Ensemble Wedding Dancers', 'gender': 2}, {'name': 'Ian Waller', 'character': 'Ensemble Wedding Dancers', 'gender': 2}], [], [{'name': 'Julianne Hough', 'character': 'Sherrie Christian', 'gender': 0}, {'name': 'Diego Boneta', 'character': 'Drew Boley', 'gender': 1}, {'name': 'Tom Cruise', 'character': 'Stacee Jaxx', 'gender': 1}, {'name': 'Alec Baldwin', 'character': 'Dennis Dupree', 'gender': 1}, {'name': 'Russell Brand', 'character': 'Lonny Barnett', 'gender': 1}, {'name': 'Malin Åkerman', 'character': 'Constance Sack', 'gender': 0}, {'name': 'Paul Giamatti', 'character': 'Paul Gill', 'gender': 1}, {'name': 'Bryan Cranston', 'character': 'Mayor Mike Whitmore', 'gender': 1}, {'name': 'Catherine Zeta-Jones', 'character': 'Patricia Whitmore', 'gender': 0}, {'name': 'Erica Frene', 'character': 'Beth', 'gender': 0}, {'name': 'Shane Hartline', 'character': 'Bartender Jimmy', 'gender': 2}, {'name': 'James Martin Kelly', 'character': 'Doug Flintlock', 'gender': 1}, {'name': 'Mary J. Blige', 'character': 'Justice', 'gender': 0}, {'name': 'Will Forte', 'character': 'Mitch Miley', 'gender': 1}, {'name': 'Eli Roth', 'character': 'Stefano', 'gender': 1}, {'name': 'Matthew Rush Sullivan', 'character': 'Random Guy', 'gender': 2}, {'name': 'Michael Olusczak', 'character': 'Crook', 'gender': 2}, {'name': 'Anthony Bellissimo', 'character': 'Rocker Thief', 'gender': 2}, {'name': 'Celina Beach', 'character': "Mayor's Secretary", 'gender': 0}, {'name': 'Angelo Valderrama', 'character': 'Chico', 'gender': 2}, {'name': 'Dan Finnerty', 'character': 'Tour Manager - Stacee', 'gender': 2}, {'name': 'Kevin Nash', 'character': "Stacee's Bodyguard", 'gender': 1}, {'name': 'Jeff Chase', 'character': "Stacee's Bodyguard", 'gender': 1}, {'name': 'Chantel Gonsalves', 'character': 'Stacee Groupie', 'gender': 0}, {'name': 'Tyne Stecklein', 'character': 'Stacee Groupie', 'gender': 0}, {'name': 'Hanna Lee Sakakibara', 'character': 'Stacee Groupie', 'gender': 0}, {'name': 'Jaimie Goodwin', 'character': 'Stacee Groupie', 'gender': 0}, {'name': 'Anne Fletcher', 'character': 'Church Horse Mother', 'gender': 0}, {'name': 'Denise Faye', 'character': 'Protest Mother #1', 'gender': 0}, {'name': 'Marriann Nelson', 'character': 'Protest Mother #2', 'gender': 0}, {'name': 'C.J. Tywoniak', 'character': "Drew's Band", 'gender': 2}, {'name': 'Maxwell Terlecki', 'character': "Drew's Band", 'gender': 2}, {'name': 'Robert Reef', 'character': "Drew's Band", 'gender': 2}, {'name': 'Marcus Johns', 'character': "Drew's Band", 'gender': 1}, {'name': 'Vivi Pineda', 'character': 'Waitress #1', 'gender': 0}, {'name': 'Aniela McGuinness', 'character': 'Waitress #2', 'gender': 0}, {'name': 'Elvire Emanuelle', 'character': 'Waitress #3', 'gender': 0}, {'name': 'David Gibbs', 'character': 'Arsenal Band', 'gender': 2}, {'name': 'Daniel Wills', 'character': 'Arsenal Band', 'gender': 2}, {'name': 'Prince Shah', 'character': 'Store Manager', 'gender': 2}, {'name': 'Benjamin Malone', 'character': 'Motel Manager', 'gender': 2}, {'name': 'Arielle Reitsma', 'character': 'Slutty Girl', 'gender': 0}, {'name': 'Anya Garnis', 'character': 'Destiny', 'gender': 0}, {'name': 'Barry Habib', 'character': 'Record Executive', 'gender': 2}, {'name': 'Constantine Maroulis', 'character': 'Record Executive', 'gender': 2}, {'name': 'Heather Leigh Davis', 'character': 'Clothes Stylist', 'gender': 0}, {'name': 'Josh Randall', 'character': 'Z-Guyeezz #1 - Donny', 'gender': 2}, {'name': 'Jack Mountford', 'character': 'Z-Guyeezz #2 - Joey', 'gender': 2}, {'name': 'Elgin Kos Aponte', 'character': 'Z-Guyeezz #3 - Kevy', 'gender': 2}, {'name': 'Veronica Berry', 'character': 'Louise / Assistant #2', 'gender': 0}, {'name': 'Karelix Alicea', 'character': 'Sinnamon', 'gender': 0}, {'name': 'T.J. Miller', 'character': 'Rolling Stone Receptionist', 'gender': 1}, {'name': 'Choice Gray', 'character': 'Kissing Girl', 'gender': 0}, {'name': 'Sophie Cook', 'character': 'Female Fan', 'gender': 0}, {'name': 'Tawni Edwards', 'character': 'Dancer - Protesting Mother', 'gender': 0}, {'name': 'Natalia Gonzalez', 'character': 'Dancer - Protesting Mother', 'gender': 0}, {'name': 'Tara Nicole Hughes', 'character': 'Dancer - Protesting Mother', 'gender': 0}, {'name': 'Kelsey Lack', 'character': 'Dancer - Protesting Mother', 'gender': 0}, {'name': 'Mary Ann Lamb', 'character': 'Dancer - Protesting Mother', 'gender': 0}, {'name': 'Isis Masoud', 'character': 'Dancer - Protesting Mother', 'gender': 0}, {'name': 'Jill Nicklaus', 'character': 'Dancer - Protesting Mother', 'gender': 0}, {'name': 'Maria Ines Serritella', 'character': 'Dancer - Protesting Mother', 'gender': 0}, {'name': 'Jenni Lynn Thomasson', 'character': 'Dancer - Protesting Mother', 'gender': 0}, {'name': 'Kelly Truzzolino', 'character': 'Dancer - Protesting Mother', 'gender': 0}, {'name': 'Shana Burns', 'character': 'Dancer - Bourbon Worker', 'gender': 0}, {'name': 'Ian Joseph Paget', 'character': 'Dancer - Bourbon Worker', 'gender': 2}, {'name': 'Shekitra Starke', 'character': 'Dancer - Bourbon Worker', 'gender': 2}, {'name': 'Alexis Adler', 'character': 'Dancer - Bourbon Patron', 'gender': 0}, {'name': 'Danielle Core', 'character': 'Dancer - Bourbon Patron', 'gender': 2}, {'name': 'Chelsea Corp', 'character': 'Dancer - Bourbon Patron', 'gender': 0}, {'name': 'Chriz Embroz', 'character': 'Dancer - Bourbon Patron', 'gender': 2}, {'name': 'Lina Ferrera', 'character': 'Dancer - Bourbon Patron', 'gender': 0}, {'name': 'Susie Garcia', 'character': 'Dancer - Bourbon Patron', 'gender': 0}, {'name': 'Tony Gonzalez', 'character': 'Dancer - Bourbon Patron', 'gender': 2}, {'name': 'Michelle Elizabeth Griffith', 'character': 'Dancer - Bourbon Patron', 'gender': 0}, {'name': 'Milena Hale', 'character': 'Dancer - Bourbon Patron', 'gender': 0}, {'name': 'Ann Lewin', 'character': 'Dancer - Bourbon Patron', 'gender': 0}, {'name': 'Marissa Alma Nick', 'character': 'Dancer - Bourbon Patron', 'gender': 0}, {'name': 'Hector Silva', 'character': 'Dancer - Bourbon Patron', 'gender': 2}, {'name': 'Katrina Rose Tandy', 'character': 'Dancer - Bourbon Patron', 'gender': 2}, {'name': 'Katherine Akra', 'character': "Dancer - Venus Gentleman's Club", 'gender': 0}, {'name': 'Billy Angell', 'character': "Dancer - Venus Gentleman's Club", 'gender': 1}, {'name': 'Alethea Austin', 'character': "Dancer - Venus Gentleman's Club", 'gender': 2}, {'name': 'Michael Balderrama', 'character': "Dancer - Venus Gentleman's Club", 'gender': 2}, {'name': 'Janet Briceño', 'character': "Dancer - Venus Gentleman's Club", 'gender': 0}, {'name': 'Jenyne Butterfly', 'character': "Dancer - Venus Gentleman's Club", 'gender': 0}, {'name': 'Bubba Carr', 'character': "Dancer - Venus Gentleman's Club", 'gender': 2}, {'name': 'Lisa Marie Cordoba', 'character': "Dancer - Venus Gentleman's Club", 'gender': 0}, {'name': 'Jocelyn Dowling', 'character': "Dancer - Venus Gentleman's Club", 'gender': 0}, {'name': 'Jennifer Fain', 'character': "Dancer - Venus Gentleman's Club", 'gender': 0}, {'name': 'Marlo Fisken', 'character': "Dancer - Venus Gentleman's Club", 'gender': 0}, {'name': 'Scott Fowler', 'character': "Dancer - Venus Gentleman's Club", 'gender': 2}, {'name': 'Anthony Jude Francisco', 'character': "Dancer - Venus Gentleman's Club", 'gender': 2}, {'name': 'Courtney Galiano', 'character': "Dancer - Venus Gentleman's Club", 'gender': 0}, {'name': 'Brittany Grimsley', 'character': "Dancer - Venus Gentleman's Club", 'gender': 0}, {'name': 'Neil Haskell', 'character': "Dancer - Venus Gentleman's Club", 'gender': 2}, {'name': 'Michael Higgins', 'character': "Dancer - Venus Gentleman's Club", 'gender': 2}, {'name': 'Scott Hislop', 'character': "Dancer - Venus Gentleman's Club", 'gender': 1}, {'name': 'Grace Jones', 'character': "Dancer - Venus Gentleman's Club", 'gender': 0}, {'name': 'Cris Judd ', 'character': "Dancer - Venus Gentleman's Club", 'gender': 2}, {'name': 'Keith Kuhl', 'character': "Dancer - Venus Gentleman's Club", 'gender': 0}, {'name': 'Brooke Upton', 'character': "Dancer - Venus Gentleman's Club", 'gender': 0}, {'name': 'Tiffany Maher', 'character': "Dancer - Venus Gentleman's Club", 'gender': 0}, {'name': 'Tiger Martina', 'character': "Dancer - Venus Gentleman's Club", 'gender': 2}, {'name': 'Angelique Martinez', 'character': "Dancer - Venus Gentleman's Club", 'gender': 0}, {'name': 'Mark Myars', 'character': "Dancer - Venus Gentleman's Club", 'gender': 2}, {'name': 'Frank Meli', 'character': "Dancer - Venus Gentleman's Club", 'gender': 1}, {'name': 'Carolyn Pace', 'character': "Dancer - Venus Gentleman's Club", 'gender': 0}, {'name': 'Nathan Prevost', 'character': "Dancer - Venus Gentleman's Club", 'gender': 2}, {'name': 'Danielle Rueda-Watts', 'character': "Dancer - Venus Gentleman's Club", 'gender': 0}, {'name': 'Julie Wiesman', 'character': "Dancer - Venus Gentleman's Club", 'gender': 0}, {'name': 'Tovaris Wilson', 'character': "Dancer - Venus Gentleman's Club", 'gender': 2}, {'name': 'William Wingfield', 'character': "Dancer - Venus Gentleman's Club", 'gender': 2}, {'name': 'Joseph Akharoh Jr.', 'character': 'Burgundy Room Rocker', 'gender': 2}, {'name': 'Marnie Alexenburg', 'character': 'News Reporter', 'gender': 0}, {'name': 'Liannette Altuve', 'character': 'Groupie', 'gender': 0}, {'name': 'Alain Alvarez', 'character': 'Bourbon Room Rocker', 'gender': 2}, {'name': 'Pearce Blair', 'character': 'Game Concession Hawker', 'gender': 2}, {'name': 'Andrew Blandina', 'character': 'Bandana Rocker', 'gender': 2}, {'name': 'Brandon Blondun', 'character': 'Singing Bus Passenger', 'gender': 2}, {'name': 'Lashawn Butler', 'character': 'Lady at the Park', 'gender': 2}, {'name': 'Gail Byer', 'character': 'Protestor Mom', 'gender': 0}, {'name': 'Alex Caceres', 'character': 'Rocker', 'gender': 2}, {'name': 'Lorraine Caporaso', 'character': 'Amusement Park Guest', 'gender': 0}, {'name': 'Santos Caraballo', 'character': 'Homeless Man', 'gender': 2}, {'name': 'Brad Champion', 'character': 'Band Member', 'gender': 2}, {'name': 'Heather Ashley Chase', 'character': 'Venus Room Dancer', 'gender': 0}, {'name': 'Jordan M. Cohen', 'character': 'Tranny', 'gender': 2}, {'name': 'Shawn Copenhaver', 'character': 'Rocker', 'gender': 2}, {'name': 'Anna Crossley', 'character': 'Protestor / Pedestrian', 'gender': 0}, {'name': 'Patrice Cruz', 'character': 'Rocker', 'gender': 0}, {'name': 'Ellen Marguerite Cullivan', 'character': 'The Bourbon Room Rocker / Dancer', 'gender': 0}, {'name': 'Sarah Dandashy', 'character': 'Rocker Chick', 'gender': 0}, {'name': 'Terra Dawn', 'character': 'Punk Girl', 'gender': 0}, {'name': 'Christopher De Stefano', 'character': 'Rocker', 'gender': 2}, {'name': 'Jordana DePaula', 'character': 'Waitress', 'gender': 0}, {'name': 'Robert di Scipio', 'character': 'Arsenal Roadie', 'gender': 2}, {'name': 'Christopher Diaz', 'character': 'Rocker', 'gender': 2}, {'name': 'Eric Ditman', 'character': 'Businessman', 'gender': 2}, {'name': 'Adam Dormi', 'character': 'Security Guard', 'gender': 2}, {'name': 'Donald Dowd', 'character': 'Stoned Rocker', 'gender': 1}, {'name': 'Corinne Engstrom', 'character': 'Rocker Chick', 'gender': 0}, {'name': 'Ely Feldman', 'character': 'Rocker', 'gender': 2}, {'name': 'Marcelo Feldman', 'character': 'Venice Beach Rockstar', 'gender': 2}, {'name': 'Tracey Ferrara', 'character': 'Rocker Babe', 'gender': 0}, {'name': 'Frenchi Firecracker', 'character': 'Roller Skater', 'gender': 0}, {'name': 'Katie Flynn', 'character': 'Arsenal Rock Fan', 'gender': 0}, {'name': 'Patrick Fox', 'character': 'Automobile Driver', 'gender': 2}, {'name': 'Blake French', 'character': 'Rocker', 'gender': 2}, {'name': 'Eduardo M. Freyre', 'character': 'Barricade Pedestrian', 'gender': 2}, {'name': 'Brett Friedmann', 'character': 'Singing Bus Passenger', 'gender': 1}, {'name': 'Ricardo Fumasoli', 'character': 'Arsenal Roadie', 'gender': 2}, {'name': 'Juan Gallo', 'character': 'Front Row Concert Attendee', 'gender': 2}, {'name': 'Jason Gatto', 'character': 'Roadie', 'gender': 2}, {'name': 'Katherine Gilton', 'character': 'Rocker', 'gender': 0}, {'name': 'Buddha Gonzalez', 'character': 'Front Row Concert Attendee', 'gender': 2}, {'name': 'Jessica Guadix', 'character': 'Dancer', 'gender': 0}, {'name': 'Mike Guzman', 'character': 'Protestor', 'gender': 2}, {'name': 'Randy Herman', 'character': 'Rocker', 'gender': 2}, {'name': 'Angel Luis Hernández', 'character': 'Rocker', 'gender': 2}, {'name': 'JLynne Herrington', 'character': 'Phonebooth Rocker', 'gender': 0}, {'name': 'Derek Hough', 'character': 'Dancer', 'gender': 1}, {'name': 'Tamara Jones', 'character': 'Singing Bus Passenger', 'gender': 0}, {'name': 'Shauna Kae Lauritzen', 'character': 'Rocker Chick', 'gender': 0}, {'name': 'Valentina Karantoni', 'character': 'Bourbon Room Rocker', 'gender': 2}, {'name': 'Irene Kelly', 'character': 'Rocker', 'gender': 0}, {'name': 'Sean Patrick Kennedy', 'character': 'Protestor (uncredited)', 'gender': 1}, {'name': 'Grant Koo', 'character': 'Driver', 'gender': 2}, {'name': 'Helmy Lavezzari', 'character': 'Sexy Rock Girl Pedestrian', 'gender': 0}, {'name': 'David W. LeBlanc', 'character': 'Conservative Protester', 'gender': 2}, {'name': 'Tracy Lee', 'character': 'Doorman', 'gender': 2}, {'name': 'Gisselle Legere', 'character': 'Dancer', 'gender': 0}, {'name': 'Laurel Levey', 'character': 'Stacee Jaxx Fan', 'gender': 0}, {'name': 'John Archer Lundgren', 'character': 'Concession Operator', 'gender': 1}, {'name': 'Constantine Malahias', 'character': 'Rock Fan at Bar', 'gender': 2}, {'name': 'Janelle Marie', 'character': 'Fan', 'gender': 0}, {'name': 'Kira Marie', 'character': 'Rocker 303', 'gender': 0}, {'name': 'John McGlothlin', 'character': 'Stoned Surfer Rocker', 'gender': 2}, {'name': 'John McKarthy', 'character': 'Record Executive', 'gender': 2}, {'name': 'Stephanie McLane', 'character': 'Protest Mom', 'gender': 0}, {'name': 'Tom Milana', 'character': 'Man in Street', 'gender': 2}, {'name': 'Taylor Anthony Miller', 'character': 'Stoned Dude', 'gender': 1}, {'name': 'Jim Mitchell', 'character': 'Rocker Guy', 'gender': 1}, {'name': 'Kary Musa', 'character': "Stacee's Fan", 'gender': 0}, {'name': 'Maria Musebrink', 'character': 'Fan', 'gender': 0}, {'name': 'Margeaux Nelson', 'character': 'Video Vixen', 'gender': 0}, {'name': 'Scott Opiela', 'character': 'Bourbon Room Roadie / Venice Beach Lead Guitar', 'gender': 2}, {'name': 'Mina Ownlee', 'character': 'Rocker', 'gender': 0}, {'name': 'Antonio Padin', 'character': 'Rocking Cool Dude', 'gender': 2}, {'name': 'Susan Pages', 'character': 'Midwestern Mom', 'gender': 0}, {'name': 'Tatiana Paris', 'character': 'Rocker', 'gender': 0}, {'name': 'B.J. Parker', 'character': "Mayor's Assistant", 'gender': 2}, {'name': 'Jennie Parrish', 'character': 'Arsenal Rock Fan', 'gender': 0}, {'name': 'Nicholas X. Parsons', 'character': 'Guy in Crowd', 'gender': 1}, {'name': 'Indra Patel', 'character': 'Rocker', 'gender': 0}, {'name': 'Heathyre Perara', 'character': 'Arsenal Fan', 'gender': 0}, {'name': 'Ryan Powers', 'character': 'Singing Bus Passenger', 'gender': 2}, {'name': 'Julie Anne Prescott', 'character': 'Staircase Dancer', 'gender': 0}, {'name': 'Lisa Marie Quillinan', 'character': 'Narrator (voice)', 'gender': 0}, {'name': 'Heidi Rae', 'character': 'Rocker', 'gender': 0}, {'name': 'Andrew Raffelson', 'character': 'Arsenal Groupie', 'gender': 1}, {'name': 'Terry L. Reed', 'character': 'Gas Station Customer', 'gender': 2}, {'name': 'Christina Rodriguez', 'character': 'Campaign Supporter', 'gender': 0}, {'name': 'Joe Rodriguez', 'character': 'Rocker', 'gender': 2}, {'name': 'Tony Saiz', 'character': 'Driver', 'gender': 2}, {'name': 'Katie Schaar', 'character': 'Dancer', 'gender': 0}, {'name': 'Metisha Schaefer', 'character': 'Dancer', 'gender': 0}, {'name': 'Danny Shamon', 'character': 'Arsenal Fan', 'gender': 2}, {'name': 'Ron Shimshilashvili', 'character': 'Comedian', 'gender': 2}, {'name': 'Julia Shure', 'character': 'Rocker', 'gender': 0}, {'name': 'Veronica Viruet Simpson', 'character': 'Arsenal Fan', 'gender': 0}, {'name': 'Jaime Ray Soto', 'character': 'Rocker', 'gender': 2}, {'name': 'Craig C. Stewart', 'character': 'Rocker', 'gender': 2}, {'name': 'Frank Tedesco', 'character': 'Sunset Strip Dude / Cafe Patron', 'gender': 2}, {'name': 'Lacey Toups', 'character': 'Girl in Crowd', 'gender': 0}, {'name': 'Mayra Vallejo', 'character': 'Rocker', 'gender': 0}, {'name': 'Charles Van Dyke', 'character': 'Bus Passenger', 'gender': 2}, {'name': 'Giovanni Velazquez', 'character': 'Dancer', 'gender': 2}, {'name': 'Vanessa Volt', 'character': 'Rocker', 'gender': 0}, {'name': 'Tracy Weisert', 'character': 'Popcorn Woman', 'gender': 0}, {'name': 'Jennifer Wolfson', 'character': 'Street Rocker', 'gender': 0}, {'name': 'Jason Wool', 'character': 'Record Store Customer / Rocker / Bouncer / Taxi Driver', 'gender': 2}, {'name': 'Carmela Zumbado', 'character': 'Rocker', 'gender': 0}, {'name': 'Christian Ijin Link', 'character': "Grauman's Theater Tourist (uncredited)", 'gender': 1}], [{'name': 'Chris Pratt', 'character': 'Owen Grady', 'gender': 1}, {'name': 'Bryce Dallas Howard', 'character': 'Claire Dearing', 'gender': 0}, {'name': 'Irrfan Khan', 'character': 'Simon Masrani', 'gender': 1}, {'name': "Vincent D'Onofrio", 'character': 'Vic Hoskins', 'gender': 1}, {'name': 'Nick Robinson', 'character': 'Zach Mitchell', 'gender': 1}, {'name': 'Ty Simpkins', 'character': 'Gray Mitchell', 'gender': 1}, {'name': 'Jake Johnson', 'character': 'Lowery Cruthers', 'gender': 1}, {'name': 'Omar Sy', 'character': 'Barry', 'gender': 1}, {'name': 'BD Wong', 'character': 'Henry Wu', 'gender': 1}, {'name': 'Judy Greer', 'character': 'Karen Mitchell', 'gender': 0}, {'name': 'Lauren Lapkus', 'character': 'Vivian', 'gender': 0}, {'name': 'Brian Tee', 'character': 'Hamada', 'gender': 1}, {'name': 'Katie McGrath', 'character': 'Zara Young', 'gender': 0}, {'name': 'Andy Buckley', 'character': 'Scott Mitchell', 'gender': 1}, {'name': 'Eric Edelstein', 'character': 'Paddock Supervisor', 'gender': 1}, {'name': 'Jimmy Fallon', 'character': 'Jimmy Fallon', 'gender': 1}, {'name': 'James DuMont', 'character': 'Hal Osterly', 'gender': 1}, {'name': 'Matt Burke', 'character': 'Jim Drucker', 'gender': 2}, {'name': 'Anna Talakkottur', 'character': 'Erica Brand', 'gender': 2}, {'name': 'Colin Trevorrow', 'character': 'Mr. DNA', 'gender': 1}, {'name': 'Courtney J. Clark', 'character': 'Mosasaurus Announcer', 'gender': 2}, {'name': 'Kelly Washington', 'character': "Zach's Girlfriend", 'gender': 0}, {'name': 'Matthew Cardarople', 'character': 'Gyrosphere Operator', 'gender': 1}, {'name': 'Chad Randall', 'character': 'InGen Soldier', 'gender': 1}, {'name': 'Yvonne Angulo', 'character': 'Italian Mother', 'gender': 2}, {'name': 'Chloe Perrin', 'character': 'Italian Girl', 'gender': 0}, {'name': 'Erika Erica', 'character': 'Park Visitor (uncredited)', 'gender': 0}, {'name': 'Tiffany Forest', 'character': 'Park Visitor (uncredited)', 'gender': 0}, {'name': 'Brandon Marc Higa', 'character': 'Petting Zoo Staff', 'gender': 2}, {'name': 'Arlene Newman', 'character': 'Woman with Baby', 'gender': 0}], [{'name': 'Eddie Redmayne', 'character': 'Newt Scamander', 'gender': 1}, {'name': 'Colin Farrell', 'character': 'Percival Graves', 'gender': 1}, {'name': 'Katherine Waterston', 'character': 'Porpentina "Tina" Goldstein', 'gender': 0}, {'name': 'Dan Fogler', 'character': 'Jacob Kowalski', 'gender': 1}, {'name': 'Alison Sudol', 'character': 'Queenie Goldstein', 'gender': 0}, {'name': 'Jon Voight', 'character': 'Henry Shaw Sr', 'gender': 1}, {'name': 'Ron Perlman', 'character': 'Gnarlack', 'gender': 1}, {'name': 'Johnny Depp', 'character': 'Gellert Grindelwald', 'gender': 1}, {'name': 'Zoë Kravitz', 'character': 'Leta Lestrange', 'gender': 0}, {'name': 'Ezra Miller', 'character': 'Credence Barebone', 'gender': 1}, {'name': 'Samantha Morton', 'character': 'Mary Lou Barebone', 'gender': 0}, {'name': 'Carmen Ejogo', 'character': 'Seraphina Picquery', 'gender': 0}, {'name': 'Josh Cowdery', 'character': 'Henry Shaw, Jr', 'gender': 1}, {'name': 'Ronan Raftery', 'character': 'Langdon Shaw', 'gender': 1}, {'name': 'Faith Wood-Blagrove', 'character': 'Modesty Barebone', 'gender': 0}, {'name': 'Jenn Murray', 'character': 'Chastity Barebone', 'gender': 0}, {'name': 'Gemma Chan', 'character': 'Madame Ya Zou', 'gender': 0}, {'name': 'Peter Breitmayer', 'character': 'Gilbert Bingley', 'gender': 1}, {'name': 'Kevin Guthrie', 'character': 'Mr. Abernathy', 'gender': 2}, {'name': 'Sean Cronin', 'character': 'Nearby Criminal', 'gender': 2}, {'name': 'Sam Redford', 'character': 'Customs Official', 'gender': 2}, {'name': 'Akin Gazi', 'character': 'Auror', 'gender': 2}, {'name': 'Todd Boyce', 'character': 'The Announcer', 'gender': 1}, {'name': 'Anne Wittman', 'character': 'Housewife', 'gender': 0}, {'name': 'Andreea Păduraru', 'character': 'Auror 9', 'gender': 0}, {'name': 'Matthew Sim', 'character': 'Photographer 1', 'gender': 1}, {'name': 'Elizabeth Moynihan', 'character': 'Executioner #1', 'gender': 0}, {'name': 'Adam Lazarus', 'character': 'Wizard', 'gender': 1}, {'name': 'Lucie Pohl', 'character': 'Secretary', 'gender': 0}, {'name': 'Tim Bentinck', 'character': 'Witness', 'gender': 1}, {'name': 'Bart Edwards', 'character': 'New York Chief of Police', 'gender': 1}, {'name': 'Brian F. Mulvey', 'character': 'The Watchman', 'gender': 1}, {'name': 'Tristan Tait', 'character': 'Reporter', 'gender': 1}, {'name': 'Tom Clarke Hill', 'character': 'Photographer 2', 'gender': 2}, {'name': 'Cory Peterson', 'character': 'Bank Employee', 'gender': 2}, {'name': 'Jake Samuels', 'character': 'Bank Guard', 'gender': 2}, {'name': 'Max Cazier', 'character': 'Young Man', 'gender': 2}, {'name': 'Dan Hedaya', 'character': 'Red', 'gender': 1}, {'name': 'Christy Meyer', 'character': 'Guide - Inside Macusa', 'gender': 0}, {'name': 'Guy Paul', 'character': 'Auror 2', 'gender': 2}, {'name': 'Walles Hamonde', 'character': 'Auror 10', 'gender': 2}, {'name': 'Dominique Tipper', 'character': 'Auror 1', 'gender': 0}, {'name': 'Leo Heller', 'character': 'Boy with Birthmark', 'gender': 2}, {'name': 'Miles Roughley', 'character': 'Poor Boy', 'gender': 2}, {'name': 'Erick Hayden', 'character': 'Policeman', 'gender': 1}, {'name': 'Paul Birchard', 'character': 'Hobo', 'gender': 1}, {'name': 'Tom Hodgkins', 'character': 'Barker', 'gender': 1}, {'name': 'Ellie Haddington', 'character': 'Mrs. Esposito', 'gender': 0}, {'name': 'Joseph Macnab', 'character': 'Police Officer 1', 'gender': 2}, {'name': 'Martin Oelbermann', 'character': 'Heinrich Eberstadt', 'gender': 2}, {'name': 'Richard Clothier', 'character': 'British Envoy', 'gender': 2}, {'name': 'Christian Dixon', 'character': 'Momolou Wotorson', 'gender': 2}, {'name': 'Richard Hardisty', 'character': 'Auror 4', 'gender': 2}, {'name': 'Miquel Brown', 'character': 'Executioner #2', 'gender': 2}, {'name': 'Wunmi Mosaku', 'character': 'Beryl', 'gender': 0}, {'name': 'Cristian Solimeno', 'character': 'Executioner #3', 'gender': 1}, {'name': 'Matthew Wilson', 'character': 'Sam the Obliviator', 'gender': 2}, {'name': 'Aretha Ayeh', 'character': 'Singer - The Blind Pig Speakeasy', 'gender': 2}, {'name': 'Emmi', 'character': 'Singer - The Blind Pig Speakeasy', 'gender': 2}, {'name': 'Nicholas McGaughey', 'character': 'Australian Wizard', 'gender': 1}, {'name': 'Arinzé Kene', 'character': 'Auror 6', 'gender': 1}, {'name': 'Jane Perry', 'character': 'Bakery Customer', 'gender': 0}, {'name': 'Abi Adeyemi', 'character': 'International Royal Dignitar (uncredited)', 'gender': 2}, {'name': 'Reid Anderson', 'character': 'Auror (uncredited)', 'gender': 2}, {'name': 'Aileen Archer', 'character': 'Criminal (uncredited)', 'gender': 0}, {'name': 'Lee Asquith-Coe', 'character': 'Officer 247 (uncredited)', 'gender': 1}, {'name': 'Lasco Atkins', 'character': 'Pedestrian (uncredited)', 'gender': 1}, {'name': 'Robert-Anthony Artlett', 'character': 'Upperclass Wizard (uncredited)', 'gender': 2}, {'name': 'Alphonso Austin', 'character': 'Boat Crew (uncredited)', 'gender': 2}, {'name': 'Michael Barron', 'character': 'New Yorker (uncredited)', 'gender': 2}, {'name': 'Roy Beck', 'character': "Senator Shaw's Speechwriter (uncredited)", 'gender': 2}, {'name': 'Marc Benanti', 'character': 'Journalist (uncredited)', 'gender': 1}, {'name': 'Nathan Benham', 'character': 'Bank Guard (uncredited)', 'gender': 2}, {'name': 'Paul Bergquist', 'character': 'Gala Dinner Guest (uncredited)', 'gender': 2}, {'name': 'Laura Bernardeschi', 'character': 'Italian Tenant in New York (uncredited)', 'gender': 0}, {'name': 'David J Biscoe', 'character': 'Journalist (uncredited)', 'gender': 2}, {'name': 'Lee Bolton', 'character': 'Photographer (uncredited)', 'gender': 2}, {'name': 'Annarie Boor', 'character': 'New Yorker (uncredited)', 'gender': 0}, {'name': 'Elizabeth Briand', 'character': 'Gala Guest (uncredited)', 'gender': 0}, {'name': 'Neil Broome', 'character': 'Gala Dinner Guest (uncredited)', 'gender': 1}, {'name': 'Greg Brummel', 'character': 'New York Tenement (uncredited)', 'gender': 2}, {'name': 'Douglas Byrne', 'character': 'Gala Guest (uncredited)', 'gender': 2}, {'name': 'Fanny Carbonnel', 'character': 'Mrs. Goldstein (uncredited)', 'gender': 0}, {'name': 'David Charles-Cully', 'character': 'Gentleman (uncredited)', 'gender': 2}, {'name': 'Stacey Clegg', 'character': 'Gala Dinner Guest (uncredited)', 'gender': 0}, {'name': 'Chloe Collingwood', 'character': 'Taxi Passenger (uncredited)', 'gender': 2}, {'name': 'Claire Cooper-King', 'character': 'Gala Dinner Guest (uncredited)', 'gender': 0}, {'name': 'Carmen Cowell', 'character': 'Tenement Woman (uncredited)', 'gender': 0}, {'name': 'Silvia Crastan', 'character': 'New Yorker (uncredited)', 'gender': 0}, {'name': 'Tom Dab', 'character': 'Factory Worker (uncredited)', 'gender': 1}, {'name': 'Craig Davies', 'character': 'New Yorker (uncredited)', 'gender': 2}, {'name': 'Nick Davison', 'character': 'Factory Worker (uncredited)', 'gender': 2}, {'name': 'Chloe de Burgh', 'character': 'Tenement Woman (uncredited)', 'gender': 0}, {'name': 'Paul Dewdney', 'character': 'New York Journalist (uncredited)', 'gender': 2}, {'name': 'Rudi Dharmalingam', 'character': 'Ranjit (uncredited)', 'gender': 2}, {'name': 'Joshua Diffley', 'character': 'Young Husband (uncredited)', 'gender': 2}, {'name': 'Nick Donald', 'character': 'Bank Customer (uncredited)', 'gender': 2}, {'name': 'Richard Douglas', 'character': 'Gentleman (uncredited)', 'gender': 1}, {'name': 'Henry Douthwaite', 'character': 'Skender (uncredited)', 'gender': 2}, {'name': 'Stephanie Eccles', 'character': 'Gala Dinner Guest (uncredited)', 'gender': 0}, {'name': 'Dino Fazzani', 'character': 'Factory Worker (uncredited)', 'gender': 2}, {'name': 'Flor Ferraco', 'character': 'Bank Client (uncredited)', 'gender': 0}, {'name': 'Marketa Flynn', 'character': 'New Yorker (uncredited)', 'gender': 0}, {'name': 'Lobna Futers', 'character': 'Witch (uncredited)', 'gender': 0}, {'name': 'Michael Gabbitas', 'character': 'Gala Guest (uncredited)', 'gender': 2}, {'name': 'David Goodson', 'character': 'Porter (uncredited)', 'gender': 2}, {'name': 'Kirsty Grace', 'character': 'Jazz Girl (uncredited)', 'gender': 0}, {'name': 'Rudy Valentino Grant', 'character': 'New York Street Sweeper (uncredited)', 'gender': 1}, {'name': 'Guna Gultniece', 'character': 'Immigrant (uncredited)', 'gender': 0}, {'name': 'Abigayle Honeywill', 'character': 'Criminal (uncredited)', 'gender': 0}, {'name': 'Luke Hope', 'character': 'NYC Banker (uncredited)', 'gender': 1}, {'name': 'Kornelia Horvath', 'character': "South American Passenger on Newt's Boat (uncredited)", 'gender': 0}, {'name': 'Ashley Hudson', 'character': 'Young Gentleman (uncredited)', 'gender': 2}, {'name': 'Alan Wyn Hughes', 'character': 'Gala Photographer (uncredited)', 'gender': 2}, {'name': 'Alex Jaep', 'character': 'Pedestrian (uncredited)', 'gender': 1}, {'name': 'Ian Jenkins', 'character': 'High Society Gentleman (uncredited)', 'gender': 2}, {'name': 'Patrick Carney Junior', 'character': 'Gala Dinner Guest (uncredited)', 'gender': 2}, {'name': 'Solomon Taiwo Justified', 'character': 'Wizard (uncredited)', 'gender': 2}, {'name': 'Lampros Kalfuntzos', 'character': "Male Passenger on Newt's Boat (uncredited)", 'gender': 1}, {'name': 'Attila G. Kerekes', 'character': 'New York Citizen (uncredited)', 'gender': 2}, {'name': 'Simon Kerrison', 'character': 'High Society Gentleman / Gala Guest (uncredited)', 'gender': 2}, {'name': 'Denis Khoroshko', 'character': 'Banker Smirnoff (uncredited)', 'gender': 2}, {'name': 'Cole Leman', 'character': 'Customs Officer (uncredited)', 'gender': 2}, {'name': 'Adam Lezemore', 'character': 'Gala Guest (uncredited)', 'gender': 2}, {'name': 'Keith Lomas', 'character': 'New York Socialite (uncredited)', 'gender': 2}, {'name': 'Paul Low-Hang', 'character': 'Pentagram Nobleman (uncredited)', 'gender': 2}, {'name': 'Khristopher MacLeod', 'character': 'Higher Class Gentleman (uncredited)', 'gender': 2}, {'name': 'Joe Malone', 'character': 'Young Wizard (uncredited)', 'gender': 2}, {'name': 'Alan Mandel', 'character': 'Mr. Goldstein (uncredited)', 'gender': 2}, {'name': 'Christopher Marsh', 'character': 'New York Banker (uncredited)', 'gender': 2}, {'name': 'Jorge Leon Martinez', 'character': 'Spaniard Ship Passenger (uncredited)', 'gender': 2}, {'name': 'Christine Marzano', 'character': 'Exterminator (uncredited)', 'gender': 0}, {'name': 'Pete Meads', 'character': 'Factory Worker (uncredited)', 'gender': 2}, {'name': 'Andy Mihalache', 'character': 'Bank Customer (uncredited)', 'gender': 2}, {'name': 'Arnold Montey', 'character': 'Journalist (uncredited)', 'gender': 2}, {'name': 'James M.L. Muller', 'character': 'Newsroom Reporter (uncredited)', 'gender': 2}, {'name': 'Paul A Munday', 'character': 'Journalist (uncredited)', 'gender': 2}, {'name': 'John Murray', 'character': 'Blind Pig Criminal (uncredited)', 'gender': 1}, {'name': "Dennis O'Donnell", 'character': 'Circus Escapologist (uncredited)', 'gender': 1}, {'name': "Yves O'Hara", 'character': 'Gala Dinner Guest (uncredited)', 'gender': 2}, {'name': 'Andrew G. Ogleby', 'character': 'New York Factory Worker (uncredited)', 'gender': 2}, {'name': 'Edd Osmond', 'character': 'Creature Performer (uncredited)', 'gender': 2}, {'name': 'Nick Owenford', 'character': 'New York Gentleman (uncredited)', 'gender': 2}, {'name': 'Andrew Parker', 'character': 'Gangster at Bar (uncredited)', 'gender': 2}, {'name': 'Gino Picciano', 'character': 'Gangster (uncredited)', 'gender': 2}, {'name': 'Richard Price', 'character': 'Gala Guest (uncredited)', 'gender': 2}, {'name': 'Olivia Quinn', 'character': 'Aunt of the Bride (uncredited)', 'gender': 2}, {'name': 'Paul Redfern', 'character': 'Gala Guest (uncredited)', 'gender': 2}, {'name': 'Jason Redshaw', 'character': 'Higher Class Gentleman (uncredited)', 'gender': 2}, {'name': 'Anthony J. Sacco', 'character': 'Gala Guest (uncredited)', 'gender': 1}, {'name': 'Bernardo Santos', 'character': 'Smart New Yorker (uncredited)', 'gender': 1}, {'name': 'Andrei Satalov', 'character': 'Wizard (uncredited)', 'gender': 2}, {'name': 'Dave Simon', 'character': 'New York Socialite (uncredited)', 'gender': 1}, {'name': 'David Soffe', 'character': 'Gala Dinner Guest (uncredited)', 'gender': 2}, {'name': 'Ryan Storey', 'character': 'Waiter (uncredited)', 'gender': 2}, {'name': 'Connor Sullivan', 'character': 'Gala Guest (uncredited)', 'gender': 2}, {'name': 'Camilla Talarowska', 'character': 'New York Citizen (uncredited)', 'gender': 0}, {'name': 'Dan Trotter', 'character': 'Street Vendor (uncredited)', 'gender': 2}, {'name': 'Vassiliki Tzanakou', 'character': 'Boat Passenger (uncredited)', 'gender': 0}, {'name': 'Geeta Vij', 'character': 'Auror (uncredited)', 'gender': 0}, {'name': 'Morgan Walters', 'character': 'Watchman (uncredited)', 'gender': 2}, {'name': 'Dean Weir', 'character': 'Featured Bank Guard (uncredited)', 'gender': 2}, {'name': 'Anick Wiget', 'character': "Criminal on 'Wanted' Poster (uncredited)", 'gender': 2}, {'name': 'Miroslav Zaruba', 'character': 'Bank Guard (uncredited)', 'gender': 2}], [{'name': 'Aaron Eckhart', 'character': 'Dr. Josh Keyes', 'gender': 1}, {'name': 'Hilary Swank', 'character': 'Maj. Rebecca Childs', 'gender': 0}, {'name': 'Delroy Lindo', 'character': "Dr. Ed 'Braz' Brazzleton", 'gender': 1}, {'name': 'Stanley Tucci', 'character': 'Dr. Conrad Zimsky', 'gender': 1}, {'name': 'Tchéky Karyo', 'character': 'Serge', 'gender': 1}, {'name': 'DJ Qualls', 'character': "Theodore Donald 'Rat' Finch", 'gender': 1}, {'name': 'Richard Jenkins', 'character': 'Gen. Thomas Purcell', 'gender': 1}, {'name': 'Bruce Greenwood', 'character': 'Cmdr. Robert Iverson', 'gender': 1}, {'name': 'Christopher Shyer', 'character': 'Dave Perry', 'gender': 1}, {'name': 'Ray Galletti', 'character': 'Paul', 'gender': 1}, {'name': 'Eileen Pedde', 'character': 'Lynne', 'gender': 0}, {'name': 'Rekha Sharma', 'character': 'Danni', 'gender': 0}, {'name': 'Tom Scholte', 'character': 'Acker', 'gender': 1}, {'name': 'Glenn Morshower', 'character': 'FBI Agent', 'gender': 1}, {'name': 'Anthony Harrison', 'character': 'FBI Agent', 'gender': 1}, {'name': 'Alfre Woodard', 'character': 'Stickley', 'gender': 0}, {'name': 'Fred Keating', 'character': 'Court-Martial Presiding Officer', 'gender': 1}, {'name': 'Bart Anderson', 'character': 'Dad', 'gender': 1}, {'name': 'Nicole Leroux', 'character': 'Mom', 'gender': 0}, {'name': 'Justin Callan', 'character': 'Little Boy', 'gender': 2}, {'name': 'Chris Humphreys', 'character': 'GBTV Reporter - Trafalgar Square', 'gender': 1}, {'name': 'Dion Johnstone', 'character': 'Flight Engineer Timmins', 'gender': 2}, {'name': 'Fred Ewanuick', 'character': 'Endeavor Flight Engineer', 'gender': 1}, {'name': 'Hrothgar Mathews', 'character': 'Chief Engineer Mission Control', 'gender': 1}, {'name': 'Ming-Tzong Hong', 'character': 'Scientist', 'gender': 2}, {'name': 'Jennifer Spence', 'character': "Zimsky's Assistant", 'gender': 0}, {'name': 'Michael St. John Smith', 'character': 'Pentagon General', 'gender': 1}, {'name': 'John Shaw', 'character': "FBI Agent - Rat's Apartment", 'gender': 1}, {'name': 'Nickolas Baric', 'character': 'Security Policeman - Tribunal', 'gender': 2}, {'name': 'Rosa Di Brigida', 'character': 'Rome Café Patron', 'gender': 2}, {'name': 'Roberto Roberto', 'character': 'Rome Café Patron', 'gender': 2}, {'name': 'Ermanno De Biagi', 'character': 'Rome Café Patron', 'gender': 2}, {'name': 'Marcello Laurentis', 'character': 'Rome Café Patron', 'gender': 2}, {'name': 'Matt Winston', 'character': 'Luke Barry', 'gender': 1}, {'name': 'Claire Riley', 'character': 'News Anchor', 'gender': 0}, {'name': 'Marke Driesschen', 'character': 'News Anchor', 'gender': 2}, {'name': 'Laurie Murdoch', 'character': 'Project Destiny Engineer', 'gender': 1}, {'name': 'Costa Spanos', 'character': 'Project Destiny Engineer', 'gender': 2}, {'name': 'Monique Martel', 'character': 'Paris Café Lady', 'gender': 2}, {'name': 'Lenie Scoffié', 'character': 'Paris Café Lady', 'gender': 0}, {'name': 'Nathaniel DeVeaux', 'character': 'U.S.S. Constellation Captain', 'gender': 1}, {'name': 'Robert Manitopyes', 'character': 'U.S.S. Constellation Screen Ops', 'gender': 2}, {'name': 'Alejandro Abellan', 'character': 'U.S.S. Soldier (uncredited)', 'gender': 2}, {'name': 'Benjamin Barton', 'character': 'Scientist / Extra       (uncredited)', 'gender': 2}, {'name': 'Angelica Hayden', 'character': 'Coffeehouse Patron (uncredited)', 'gender': 2}, {'name': 'Robert Paul Lewis', 'character': 'Beach-goer (uncredited)', 'gender': 1}, {'name': 'Alec Medlock', 'character': '(voice) (uncredited)', 'gender': 1}, {'name': 'Alonso Oyarzun', 'character': 'Security Policeman (uncredited)', 'gender': 1}, {'name': 'Priscilla Poland', 'character': 'Scientist (uncredited)', 'gender': 0}, {'name': 'Shawn Green', 'character': 'Himself', 'gender': 2}, {'name': 'Greg Bennett', 'character': 'Technician - Virgil Base', 'gender': 1}, {'name': 'Jeffrey Gold', 'character': 'Engineer (uncredited)', 'gender': 2}, {'name': 'Rick Kain', 'character': 'Chief Petty Officer (uncredited)', 'gender': 1}, {'name': 'Pamela Martin', 'character': 'News Anchor (uncredited)', 'gender': 0}], [{'name': 'Vin Diesel', 'character': 'Riddick', 'gender': 1}, {'name': 'Thandie Newton', 'character': 'Dame Vaako', 'gender': 0}, {'name': 'Karl Urban', 'character': 'Vaako', 'gender': 1}, {'name': 'Colm Feore', 'character': 'Lord Marshal', 'gender': 1}, {'name': 'Linus Roache', 'character': 'Purifier', 'gender': 1}, {'name': 'Keith David', 'character': 'Imam', 'gender': 1}, {'name': 'Yorick van Wageningen', 'character': 'The Guv', 'gender': 1}, {'name': 'Nick Chinlund', 'character': 'Toombs', 'gender': 1}, {'name': 'Alexa Davalos', 'character': 'Kyra', 'gender': 0}, {'name': 'Roger Cross', 'character': 'Toal', 'gender': 1}, {'name': 'Kim Hawthorne', 'character': 'Lajjun', 'gender': 0}, {'name': 'Judi Dench', 'character': 'Aereon', 'gender': 0}, {'name': 'Mark Gibbon', 'character': 'Irgun', 'gender': 1}, {'name': 'Terry Chen', 'character': 'Merc Pilot', 'gender': 1}, {'name': 'Christina Cox', 'character': 'Eve Logan', 'gender': 0}, {'name': 'Nigel Vonas', 'character': 'Merc', 'gender': 2}, {'name': 'Raoul Ganeev', 'character': 'Slam Guard', 'gender': 1}, {'name': 'Alexis Llewellyn', 'character': 'Ziza', 'gender': 0}, {'name': 'Charles Zuckermann', 'character': 'Scales', 'gender': 1}, {'name': 'Steven Barr', 'character': 'ADR Loop Group (voice)', 'gender': 1}, {'name': 'Jack Blessing', 'character': 'ADR Loop Group (voice)', 'gender': 1}, {'name': 'Steve Bulen', 'character': 'ADR Loop Group (voice)', 'gender': 2}, {'name': 'Robert Clotworthy', 'character': 'ADR Loop Group (voice)', 'gender': 1}, {'name': 'Richard Epcar', 'character': 'ADR Loop Group (voice)', 'gender': 1}, {'name': 'Elisa Gabrielli', 'character': 'ADR Loop Group (voice)', 'gender': 2}, {'name': 'Jean Gilpin', 'character': 'ADR Loop Group (voice)', 'gender': 2}, {'name': 'Bridget Hoffman', 'character': 'ADR Loop Group (voice)', 'gender': 0}, {'name': 'Hope Levy', 'character': 'ADR Loop Group (voice)', 'gender': 2}, {'name': 'Edie Mirman', 'character': 'ADR Loop Group (voice)', 'gender': 0}, {'name': 'Richard Miro', 'character': 'ADR Loop Group (voice)', 'gender': 1}, {'name': 'Peter Renaday', 'character': 'ADR Loop Group (voice)', 'gender': 1}, {'name': 'Philece Sampler', 'character': 'ADR Loop Group (voice)', 'gender': 2}, {'name': 'Michael Sorich', 'character': 'ADR Loop Group (voice)', 'gender': 1}], [], [{'name': 'Fionn Whitehead', 'character': 'Tommy', 'gender': 1}, {'name': 'Tom Glynn-Carney', 'character': 'Peter Dawson', 'gender': 1}, {'name': 'Aneurin Barnard', 'character': 'Gibson', 'gender': 1}, {'name': 'Jack Lowden', 'character': 'Collins', 'gender': 1}, {'name': 'Harry Styles', 'character': 'Alex', 'gender': 1}, {'name': "James D'Arcy", 'character': 'Lieutenant-Colonel Winnant, BEF', 'gender': 1}, {'name': 'Barry Keoghan', 'character': 'George Mills', 'gender': 1}, {'name': 'Kenneth Branagh', 'character': 'Commander Bolton of the Royal Navy', 'gender': 1}, {'name': 'Cillian Murphy', 'character': 'Shivering Soldier', 'gender': 1}, {'name': 'Mark Rylance', 'character': 'Mr. Dawson', 'gender': 1}, {'name': 'Tom Hardy', 'character': 'Farrier', 'gender': 1}, {'name': 'Michael Caine', 'character': 'Radio Communication (voice)', 'gender': 1}, {'name': 'Billy Howle', 'character': 'Petty Officer', 'gender': 1}, {'name': 'Bobby Lockwood', 'character': 'Seaman on Hospital Ship', 'gender': 1}, {'name': 'Miranda Nolan', 'character': 'Nurse on Destroyer', 'gender': 0}, {'name': 'Kevin Guthrie', 'character': 'A&SH BEF soldier', 'gender': 2}, {'name': 'Brian Vernel', 'character': 'A&SH BEF soldier', 'gender': 1}, {'name': 'Elliott Tittensor', 'character': 'Highlander', 'gender': 1}, {'name': 'Matthew Marsh', 'character': 'Rear-Admiral of the Royal Navy', 'gender': 1}, {'name': 'Jochum ten Haaf', 'character': 'Dutch Trawler Man', 'gender': 1}, {'name': 'Damien Bonnard', 'character': 'French Soldier', 'gender': 1}, {'name': 'Adam Long', 'character': 'Second Lt. Jeffries', 'gender': 1}, {'name': 'Michael Fox', 'character': 'Royal Engineer, BEF', 'gender': 1}, {'name': 'Harry Richardson', 'character': 'Private', 'gender': 2}, {'name': 'Will Attenborough', 'character': 'Lt. Howe of the Royal Navy', 'gender': 2}, {'name': 'James Bloor', 'character': 'Oily Survivor', 'gender': 2}, {'name': 'Paul Riley Fox', 'character': 'British Soldier', 'gender': 1}, {'name': 'Richard Sanderson', 'character': 'Heinkel Spotter', 'gender': 2}, {'name': 'Valiant Michael', 'character': 'Warrant Officer', 'gender': 2}, {'name': 'Johnny Otto', 'character': 'Captain Marshall', 'gender': 2}, {'name': 'Jan-Michael Rosner', 'character': 'British Soldier', 'gender': 2}, {'name': 'Brandon Duracher', 'character': 'Soldier', 'gender': 2}, {'name': 'Samgar Jacobs', 'character': 'Royal Navy', 'gender': 2}, {'name': 'Robby Prinsen', 'character': 'Royal Navy Officer', 'gender': 2}, {'name': 'Simon Ates', 'character': 'British Expeditionary Force Soldier', 'gender': 2}, {'name': 'Caleb Bailey', 'character': 'Soldier', 'gender': 2}, {'name': 'Thomas Millet', 'character': 'French Soldier', 'gender': 2}, {'name': 'Constantin Balsan', 'character': 'French Soldier', 'gender': 2}, {'name': 'Aldo Beqiri', 'character': 'French Soldier', 'gender': 2}, {'name': 'Sander Huisman', 'character': 'British Expeditionary Force Soldier', 'gender': 2}, {'name': 'Callum Blake', 'character': 'Private Davies', 'gender': 2}, {'name': 'Luke Thompson', 'character': 'Warrant Officer', 'gender': 2}, {'name': 'Crystal Pereyra', 'character': 'Nurse on Destroyer', 'gender': 2}, {'name': 'Bram Vlot', 'character': 'Royal Navy Sailor', 'gender': 2}, {'name': 'Calam Lynch', 'character': 'Sailor from Deal', 'gender': 2}, {'name': 'Jack Riddiford', 'character': 'Private Edwards', 'gender': 2}, {'name': 'Merlijn Willemsen', 'character': 'British Soldier on the Sittard', 'gender': 2}, {'name': 'Michel Biel', 'character': 'French Soldier', 'gender': 2}, {'name': "Niels van 't Dek", 'character': 'Sailor on Destroyer', 'gender': 2}, {'name': 'Nirman Wolf', 'character': 'Marine', 'gender': 2}, {'name': 'Tom Gill', 'character': 'Furious Soldier', 'gender': 1}, {'name': 'Nick Vorsselman', 'character': 'British Expeditionary Force Soldier', 'gender': 1}, {'name': 'Jedediah Jenk', 'character': 'British Expeditionary Force Soldier', 'gender': 1}, {'name': 'Christian Janner', 'character': 'British Soldier', 'gender': 2}, {'name': 'Bradley Hall', 'character': 'Sailor on Destroyer', 'gender': 2}, {'name': 'Jack Gover', 'character': 'British Soldier', 'gender': 2}, {'name': 'Davey Jones', 'character': 'British Soldier / Station Officer', 'gender': 2}, {'name': 'Charley Palmer Rothwell', 'character': 'Corporal', 'gender': 2}, {'name': 'Kim Hartman', 'character': 'Stewardess', 'gender': 0}, {'name': 'Bill Milner', 'character': 'Lone Private', 'gender': 1}, {'name': 'Eric Richard', 'character': 'Man at Railway Window', 'gender': 1}, {'name': 'Harry Collett', 'character': 'Boy', 'gender': 1}, {'name': 'Christian Roberts', 'character': 'Royal Navy Officer (uncredited)', 'gender': 2}], [{'name': 'John Goodman', 'character': 'James P. "Sulley" Sullivan (voice)', 'gender': 1}, {'name': 'Billy Crystal', 'character': 'Michael "Mike" Wazowski (voice)', 'gender': 1}, {'name': 'Steve Buscemi', 'character': 'Randall Boggs (voice)', 'gender': 1}, {'name': 'James Coburn', 'character': 'Henry J. Waternoose III (voice)', 'gender': 1}, {'name': 'Jennifer Tilly', 'character': 'Celia Mae (voice)', 'gender': 0}, {'name': 'Mary Gibbs', 'character': 'Mary "Boo" (voice)', 'gender': 0}, {'name': 'Bob Peterson', 'character': 'Roz (voice)', 'gender': 1}, {'name': 'John Ratzenberger', 'character': 'Yeti (voice)', 'gender': 1}, {'name': 'Frank Oz', 'character': 'Jeff Fungus (voice)', 'gender': 1}, {'name': 'Daniel Gerson', 'character': 'Needleman / Smitty (voice)', 'gender': 1}, {'name': 'Steve Susskind', 'character': 'Jerry Slugworth (voice)', 'gender': 1}, {'name': 'Bonnie Hunt', 'character': 'Ms. Flint (voice)', 'gender': 0}, {'name': 'Jeff Pidgeon', 'character': 'Thaddeus "Phlegm" Bile (voice)', 'gender': 1}, {'name': 'Samuel Lord Black', 'character': 'George Sanderson (voice)', 'gender': 2}, {'name': 'Phil Proctor', 'character': 'Charlie (voice)', 'gender': 1}, {'name': 'Joe Ranft', 'character': 'Pete "Claws" Ward (voice)', 'gender': 2}, {'name': 'Jack Angel', 'character': 'Additional Voices (voice)', 'gender': 1}, {'name': 'Bob Bergen', 'character': 'Additional Voices (voice)', 'gender': 1}, {'name': 'Rodger Bumpass', 'character': 'Additional Voices (voice)', 'gender': 1}, {'name': 'Gino Conforti', 'character': 'Additional Voices (voice)', 'gender': 1}, {'name': 'Jennifer Darling', 'character': 'Additional Voices (voice)', 'gender': 0}, {'name': 'Patti Deutsch', 'character': 'Additional Voices (voice)', 'gender': 0}, {'name': 'Pete Docter', 'character': 'Additional Voices (voice)', 'gender': 1}, {'name': 'Bobby Edner', 'character': 'Additional Voices (voice)', 'gender': 1}, {'name': 'Ashley Edner', 'character': 'Additional Voices (voice)', 'gender': 0}, {'name': 'Paul Eiding', 'character': 'Additional Voices (voice)', 'gender': 1}, {'name': 'Katie Scarlettr', 'character': 'Additional Voices (voice)', 'gender': 2}, {'name': 'Bill Farmer', 'character': 'Additional Voices (voice)', 'gender': 1}, {'name': 'Keegan Farrell', 'character': 'Additional Voices (voice)', 'gender': 2}, {'name': 'Pat Fraley', 'character': 'Additional Voices (voice)', 'gender': 1}, {'name': 'Teresa Ganzel', 'character': 'Additional Voices (voice)', 'gender': 0}, {'name': 'Taylor Gifaldi', 'character': 'Additional Voices (voice)', 'gender': 2}, {'name': 'Marc John Jefferies', 'character': 'Additional Voices (voice)', 'gender': 1}, {'name': 'Joe Lala', 'character': 'Additional Voices (voice)', 'gender': 1}, {'name': 'Noah Luke', 'character': 'Additional Voices (voice)', 'gender': 2}, {'name': 'Sherry Lynn', 'character': 'Additional Voices (voice)', 'gender': 0}, {'name': 'Danny Mann', 'character': 'Additional Voices (voice)', 'gender': 1}, {'name': 'Mona Marshall', 'character': 'Additional Voices (voice)', 'gender': 2}, {'name': 'Laraine Newman', 'character': 'Additional Voices (voice)', 'gender': 0}, {'name': 'Kay Panabaker', 'character': 'Additional Voices (voice)', 'gender': 0}, {'name': "Bret 'Brook' Parker", 'character': 'Trailer Mom (voice) (as Bret Parker)', 'gender': 0}, {'name': 'Josh Qualtieri', 'character': 'Additional Voices (voice)', 'gender': 2}, {'name': 'Guido Quaroni', 'character': 'Additional Voices (voice)', 'gender': 1}, {'name': 'Jan Rabson', 'character': 'Additional Voices (voice)', 'gender': 1}, {'name': 'Lisa Raggio', 'character': 'Additional Voices (voice)', 'gender': 0}, {'name': 'Katherine Ringgold', 'character': 'Additional Voices (voice)', 'gender': 2}, {'name': 'Sophia Ranft', 'character': 'Additional Voices (voice)', 'gender': 2}, {'name': 'Bob Scott', 'character': 'Additional Voices (voice)', 'gender': 1}, {'name': 'Jim Thornton', 'character': 'Additional Voices (voice)', 'gender': 2}, {'name': 'Lee Unkrich', 'character': 'Additional Voices (voice)', 'gender': 1}, {'name': 'Wallace Shawn', 'character': 'Rex (outtakes) (voice) (uncredited)', 'gender': 1}, {'name': 'David Silverman', 'character': 'Additional Voices (voice)', 'gender': 1}, {'name': 'Joe Ranft', 'character': "Pete 'Claws' Ward (voice)", 'gender': 2}], [{'name': 'Hugh Jackman', 'character': 'Charlie Kenton', 'gender': 1}, {'name': 'Dakota Goyo', 'character': 'Max Kenton', 'gender': 1}, {'name': 'Evangeline Lilly', 'character': 'Bailey Tallet', 'gender': 0}, {'name': 'Anthony Mackie', 'character': 'Finn', 'gender': 1}, {'name': 'Kevin Durand', 'character': 'Ricky', 'gender': 1}, {'name': 'Hope Davis', 'character': 'Aunt Debra', 'gender': 0}, {'name': 'James Rebhorn', 'character': 'Marvin', 'gender': 1}, {'name': 'Karl Yune', 'character': 'Tak Mashido', 'gender': 1}, {'name': 'Olga Fonda', 'character': 'Farra Lemkova', 'gender': 0}, {'name': 'John Gatins', 'character': 'Kingpin', 'gender': 1}, {'name': 'Sophie Levy', 'character': 'Big Sister', 'gender': 0}, {'name': 'Tess Levy', 'character': 'Little Sister', 'gender': 0}, {'name': 'Charlie Levy', 'character': 'Littlest Sister', 'gender': 2}, {'name': 'Gregory Sims', 'character': 'Bill Panner', 'gender': 1}, {'name': 'Torey Adkins', 'character': 'Large Texan Man', 'gender': 2}, {'name': 'Tom Carlson', 'character': 'San Leandro Gentleman #1', 'gender': 2}, {'name': 'John Hawkinson', 'character': 'San Leandro Gentleman #2', 'gender': 2}, {'name': 'David Alan Basche', 'character': 'ESPN Boxing Commentator', 'gender': 1}, {'name': 'Phil LaMarr', 'character': 'ESPN Boxing Commentator', 'gender': 1}, {'name': 'David Herbst', 'character': 'Starblaze Arena Reporter', 'gender': 2}, {'name': 'Julian Gant', 'character': 'Starblaze Arena Reporter', 'gender': 2}, {'name': 'Ken Alter', 'character': 'Virgin America Spectrum Ring Announcer', 'gender': 2}, {'name': 'Leilani Barrett', 'character': 'Virgin America Spectrum Ref', 'gender': 2}, {'name': 'Eric Gutman', 'character': 'WRB Promoter', 'gender': 2}, {'name': 'Nicholas Yu', 'character': 'Twin Cities Corner Tech', 'gender': 2}, {'name': 'D.B. Dickerson', 'character': 'Twin Cities Controller', 'gender': 2}, {'name': 'Peter Carey', 'character': 'Bing Arena Announcer', 'gender': 2}, {'name': 'Dan Lemieux', 'character': 'Bing Arena Ref', 'gender': 1}, {'name': 'Richard Goteri', 'character': 'Older Gentleman', 'gender': 2}, {'name': 'Tim Holmes', 'character': 'Blacktop Controller', 'gender': 2}, {'name': 'Ricky Wayne', 'character': 'Underground Promoter', 'gender': 1}, {'name': 'Taris Tyler', 'character': 'Robot Promoter', 'gender': 2}, {'name': 'Kevin Dorman', 'character': 'Atom Performance Capture', 'gender': 2}, {'name': 'John Manfredi', 'character': 'Sergei Lemkova', 'gender': 2}, {'name': 'Wendy Aaron', 'character': 'Robot Handler / Fight Fan (uncredited)', 'gender': 0}, {'name': 'Rachael Adams', 'character': 'Pretty Girl (uncredited)', 'gender': 0}, {'name': 'Mike Ancrile', 'character': 'Fight Fan (uncredited)', 'gender': 2}, {'name': 'Lamar Babi', 'character': 'Russian / Boxing Fan (uncredited)', 'gender': 2}, {'name': 'Apollo Bacala', 'character': 'Paparazzi (uncredited)', 'gender': 2}, {'name': 'Leah Barkoff', 'character': 'Upscale Fight Fan (uncredited)', 'gender': 0}, {'name': 'Joshua Ray Bell', 'character': 'Rodeo Cowboy (uncredited)', 'gender': 2}, {'name': 'Christina M. Bender', 'character': 'Fight Fan (uncredited)', 'gender': 0}, {'name': 'Clark Birchmeier', 'character': 'Bar Patron (uncredited)', 'gender': 2}, {'name': 'Don Boerst', 'character': 'VIP Fight Fan (uncredited)', 'gender': 2}, {'name': 'Mark Bonto', 'character': 'Funny Security Guard (uncredited)', 'gender': 2}, {'name': 'Zachary Borromeo', 'character': 'Fair Goer (uncredited)', 'gender': 2}, {'name': 'Ben Hernandez Bray', 'character': "Ricky's Henchman (uncredited)", 'gender': 1}, {'name': 'Wayne Brinston', 'character': 'Fight Fan (uncredited)', 'gender': 2}, {'name': 'Wayne E. Brown', 'character': 'Upscale Boxing Fan (uncredited)', 'gender': 1}, {'name': 'Geneva Brunetti', 'character': 'Fight Fan (uncredited)', 'gender': 0}, {'name': 'Justin Calkins', 'character': 'Rodeo Cowboy (uncredited)', 'gender': 2}, {'name': 'Steven Campbell', 'character': 'Crash Palace Fight Fan (uncredited)', 'gender': 2}, {'name': 'Jeff Caponigro', 'character': "Bailey's Dad (uncredited)", 'gender': 2}, {'name': 'Michael Patrick Carmody', 'character': 'Axelrod Handler (uncredited)', 'gender': 2}, {'name': 'Ron Causey ', 'character': "Kingpin's Henchman (uncredited)", 'gender': 2}, {'name': 'Justin Chrzanowski', 'character': 'Twin Cities Fight Fan (uncredited)', 'gender': 2}, {'name': 'Michael B. Clark', 'character': 'Rodeo Fan (uncredited)', 'gender': 1}, {'name': 'Will Clarke', 'character': 'Photographer (uncredited)', 'gender': 2}, {'name': 'Chris Coldren', 'character': 'Twin Cities Upscale Fight Fan (uncredited)', 'gender': 2}, {'name': 'Sandy Coonan', 'character': 'Metro Fan - Big Rock / Zoo (uncredited)', 'gender': 0}, {'name': 'Jahnel Curfman', 'character': 'Panoramic Fight Fan (uncredited)', 'gender': 0}, {'name': 'Haylee Detroit Dewar', 'character': 'Gang Member (uncredited)', 'gender': 0}, {'name': 'John Dezsi', 'character': 'Drunkard (uncredited)', 'gender': 2}, {'name': 'P.J. Edwards', 'character': 'Axelrod Controller (uncredited)', 'gender': 2}, {'name': 'Shannon Edwards', 'character': 'Ringside VIP - New York (uncredited)', 'gender': 2}, {'name': 'Raffi Elias', 'character': 'Ringside Security Guard (uncredited)', 'gender': 2}, {'name': 'Rima Fakih', 'character': 'Herself (uncredited)', 'gender': 2}, {'name': 'Tom Feldpausch', 'character': 'Fight Fan (uncredited)', 'gender': 2}, {'name': 'Johnny Flynn', 'character': 'ESPN Commentator (uncredited)', 'gender': 1}, {'name': 'Stephen C. Forsell', 'character': 'Rodeo Vendor (uncredited)', 'gender': 2}, {'name': 'Sarah Forton', 'character': 'Girl at Fair (uncredited)', 'gender': 0}, {'name': 'Stephen Fosner', 'character': 'Vendor at Rodeo (uncredited)', 'gender': 2}, {'name': 'Logan Fry', 'character': 'Rabid Fight Fan (uncredited)', 'gender': 2}, {'name': 'Nesti Gee', 'character': 'Robot Handler (uncredited)', 'gender': 2}, {'name': 'Ryan Genther', 'character': 'Texas Fairgoer (uncredited)', 'gender': 2}, {'name': 'Dan Gerics', 'character': 'Townsperson (uncredited)', 'gender': 2}, {'name': 'Joey Ghinelli', 'character': 'Fight Fan (uncredited)', 'gender': 2}, {'name': 'Jacob Godzak', 'character': 'Fight Fan (uncredited)', 'gender': 2}, {'name': 'Ashley Goulson', 'character': 'Fight Fan (uncredited)', 'gender': 0}, {'name': 'Megan Grant', 'character': 'Vendor (uncredited)', 'gender': 0}, {'name': 'Brandon Green', 'character': 'Fight Fan (uncredited)', 'gender': 2}, {'name': 'J.J. Green', 'character': 'Crash Palace Fan (uncredited)', 'gender': 2}, {'name': 'Rebecca Guth', 'character': 'Fight Fan (uncredited)', 'gender': 0}, {'name': 'Steven Gätjen', 'character': 'Starblaze Arena Reporter (uncredited)', 'gender': 1}, {'name': 'Shane Hagedorn', 'character': 'Photographer (uncredited)', 'gender': 2}, {'name': 'Joey Harlow', 'character': 'Upper Class Upscale Fight Fan (uncredited)', 'gender': 2}, {'name': 'Pennie-Marie Hawkins', 'character': 'ESPN Box (uncredited)', 'gender': 2}, {'name': 'Ron Heisler', 'character': 'Fight Fan (uncredited)', 'gender': 2}, {'name': 'Robert Herrick', 'character': 'Bar Date (uncredited)', 'gender': 2}, {'name': 'Kathryn Henzler', 'character': 'Fight Fan (uncredited)', 'gender': 0}, {'name': 'Christopher W. Hodshire', 'character': 'Rodeo Cowboy (uncredited)', 'gender': 2}, {'name': 'Suzy Hunt', 'character': 'Upscale Fight Fan (uncredited)', 'gender': 0}, {'name': 'Markus James', 'character': 'Fight Fan (uncredited)', 'gender': 1}, {'name': 'Pauline Ann Johnson', 'character': 'Bull Fight Attendee (uncredited)', 'gender': 0}, {'name': 'Traver Johnson', 'character': 'Zoo Fight Fan (uncredited)', 'gender': 2}, {'name': 'Gary T. Jones', 'character': 'Rich Man (uncredited)', 'gender': 1}, {'name': 'Jennie Kahn-Jacques', 'character': 'Fight Fan (uncredited)', 'gender': 0}, {'name': 'Ko Kaiden', 'character': 'Japenese Reporter #1 (uncredited)', 'gender': 2}, {'name': 'Douglas King II', 'character': 'Zoo Fight Fan / Spectrum Roadie (uncredited)', 'gender': 2}, {'name': 'Lauren Kole', 'character': 'Girl at Fair (uncredited)', 'gender': 0}, {'name': 'Nathan Kranzo', 'character': 'Photographer (uncredited)', 'gender': 2}, {'name': 'Joe Kras', 'character': 'Boxing Fan (uncredited)', 'gender': 2}, {'name': 'Amy LaPlante', 'character': 'Fight Fan (uncredited)', 'gender': 0}, {'name': 'Kef Lee', 'character': 'Zeus Robot Handler #1 (uncredited)', 'gender': 1}, {'name': 'Amanda Lewan', 'character': 'Waitress (uncredited)', 'gender': 0}, {'name': 'Anna Li', 'character': 'Fight Fan (uncredited)', 'gender': 2}, {'name': 'Dervis Lici', 'character': 'Photographer (uncredited)', 'gender': 2}, {'name': 'Linda Linsley', 'character': 'Female Livestock Judge (uncredited)', 'gender': 0}, {'name': 'Nathaniel Loveland', 'character': 'Upper Class Fight Fan (uncredited)', 'gender': 2}, {'name': 'Bill Lowery', 'character': 'Ring Security Guard (uncredited)', 'gender': 2}, {'name': 'Rich Lozano', 'character': 'Fight Fan (uncredited)', 'gender': 2}, {'name': 'Bill Lumbert ', 'character': 'Fight Fan (uncredited)', 'gender': 2}, {'name': 'Brad Leo Lyon', 'character': 'Fight Fan (uncredited)', 'gender': 2}, {'name': 'Mary Magyari', 'character': 'Usher (uncredited)', 'gender': 0}, {'name': 'Chris Mannix', 'character': 'Writer #1 (uncredited)', 'gender': 2}, {'name': 'Melissa Marra', 'character': 'Fight Fan / Fair Goer (uncredited)', 'gender': 0}, {'name': 'Rance Martin', 'character': 'Fight Fan (uncredited)', 'gender': 2}, {'name': 'Bryan Matti', 'character': 'Fight Fan (uncredited)', 'gender': 2}, {'name': 'Jessica McLarty', 'character': 'Fight Fan (uncredited)', 'gender': 0}, {'name': 'Samuel Meadows', 'character': 'Boxing Fan (uncredited)', 'gender': 2}, {'name': 'Jordan Kenneth Messing', 'character': 'Vendor (uncredited)', 'gender': 2}, {'name': 'Ralph H. Meyer', 'character': 'Cowboy / Upscale Fan (uncredited)', 'gender': 2}, {'name': 'Antonio L. Miller', 'character': 'Lead Security Guard (uncredited)', 'gender': 2}, {'name': 'Wendel Millstead', 'character': "Farrah's Body Guard (uncredited)", 'gender': 2}, {'name': 'Gary L. Minix', 'character': 'Laughing Zoo Fight Fan (uncredited)', 'gender': 2}, {'name': 'Megan Mockensturm', 'character': 'Fight Fan (uncredited)', 'gender': 0}, {'name': 'Jermaine Moore', 'character': 'Ring Security (uncredited)', 'gender': 2}, {'name': 'Kirstie Munoz', 'character': 'Vendor (uncredited)', 'gender': 0}, {'name': 'Anton Narinskiy', 'character': "Farra's Body Guard (uncredited)", 'gender': 1}, {'name': 'Steven Hugh Nelson', 'character': 'Western Bar Patron (uncredited)', 'gender': 1}, {'name': 'Chris Newman', 'character': 'Starblaze Security Guard (uncredited)', 'gender': 1}, {'name': 'Michael Nicolae', 'character': 'Big Brother (uncredited)', 'gender': 2}, {'name': "Chris O'Brien", 'character': 'Crash Palace Fight Fan (uncredited)', 'gender': 2}, {'name': 'Wendy Paquette', 'character': 'Fight Fan (uncredited)', 'gender': 0}, {'name': 'Heather Park', 'character': "Kingpin's Girlfriend (uncredited)", 'gender': 2}, {'name': 'Jessica Petrik', 'character': 'Upscale Fight Fan (uncredited)', 'gender': 0}, {'name': 'Alan D. Purwin', 'character': 'Helicopter Pilot (uncredited)', 'gender': 1}, {'name': 'Ralph A. Recchia ', 'character': 'Boxing Fan (uncredited)', 'gender': 2}, {'name': 'Jay Reid', 'character': 'Fight Fan (uncredited)', 'gender': 2}, {'name': 'DaJuan Rippy', 'character': 'Fight Fan (uncredited)', 'gender': 2}, {'name': 'Nicholas Ritz', 'character': 'Fair Goer / Fight Fan Panaramic / Fight Fan Metro / Fight Fan Zoo (uncredited)', 'gender': 2}, {'name': 'Jeff Rosenfeld', 'character': 'Boxing Fan (uncredited)', 'gender': 2}, {'name': 'Marco Ruggeri', 'character': 'Cliff (uncredited)', 'gender': 2}, {'name': 'Maria Lucia Safi', 'character': 'Cat Woman (uncredited)', 'gender': 0}, {'name': 'Miguel Sandoval', 'character': 'Judge (uncredited)', 'gender': 1}, {'name': 'Sebastian Sarkissian', 'character': 'Fair Goer (uncredited)', 'gender': 2}, {'name': 'Steven Scott', 'character': 'Lieutenant - Kingpin (uncredited)', 'gender': 2}, {'name': 'Ron Shedd', 'character': 'Fight Crowd Member (uncredited)', 'gender': 2}, {'name': 'Daryl M. Simpson', 'character': 'Fight Fan / Robot Roadie (uncredited)', 'gender': 2}, {'name': 'Gary Lee Simpson', 'character': 'Upper Class Fight Fan (uncredited)', 'gender': 2}, {'name': 'Brian Anderson Smith', 'character': 'Fight Fan (uncredited)', 'gender': 2}, {'name': 'Joseph Smith', 'character': 'Robot Handler (uncredited)', 'gender': 1}, {'name': 'Nikki Smith', 'character': 'Fight Fan (uncredited)', 'gender': 2}, {'name': 'Dwight Sora', 'character': 'Japanese Reporter #2 (uncredited)', 'gender': 2}, {'name': 'Paul J. Spear', 'character': 'Fair Goer (uncredited)', 'gender': 2}, {'name': 'Alan Stefan', 'character': 'Fight Fan (uncredited)', 'gender': 2}, {'name': 'Tiffany Stone', 'character': "Tak's Girl (uncredited)", 'gender': 0}, {'name': 'Morris Lee Sullivan', 'character': 'Cowboy at Rodeo / Robot Fight Fan (uncredited)', 'gender': 2}, {'name': 'John E.L. Tenney', 'character': 'Salvage Yard Security Guard (uncredited)', 'gender': 2}, {'name': 'Melody Teodoro-Kurtis', 'character': 'Twin Cities Fight Fan (uncredited)', 'gender': 0}, {'name': 'Michael Trobaugh', 'character': 'Fight Fan (uncredited)', 'gender': 2}, {'name': 'Johnny Truong', 'character': 'Usher (uncredited)', 'gender': 2}, {'name': 'Eric Tuchelske', 'character': 'Fair Goer (uncredited)', 'gender': 2}, {'name': 'Laurie Valko', 'character': 'Cowgirl (uncredited)', 'gender': 0}, {'name': 'Brett VanDunk', 'character': 'Fight Fan (uncredited)', 'gender': 2}, {'name': 'Johnathon VanDusen', 'character': 'Fight Fan (uncredited)', 'gender': 2}, {'name': 'Dan Watson', 'character': 'Fight Fan (uncredited)', 'gender': 2}, {'name': 'Kaitlan Welton', 'character': 'Pretty Girl (uncredited)', 'gender': 0}, {'name': 'Jojuan Westmoreland', 'character': 'Zoo Fight Fan (uncredited)', 'gender': 2}, {'name': 'Ed Williams', 'character': 'Vendor (uncredited)', 'gender': 2}, {'name': 'Amanda Wright', 'character': "Ricky's Girl (uncredited)", 'gender': 0}, {'name': 'Paul Xiong', 'character': 'Photographer (uncredited)', 'gender': 2}], [{'name': 'Idris Elba', 'character': 'Roland Deschain', 'gender': 1}, {'name': 'Matthew McConaughey', 'character': 'Walter O’Dim', 'gender': 1}, {'name': 'Tom Taylor', 'character': 'Jake Chambers', 'gender': 1}, {'name': 'Claudia Kim', 'character': 'Arra Champignon', 'gender': 0}, {'name': 'Fran Kranz', 'character': 'Pimli Prentiss', 'gender': 1}, {'name': 'Abbey Lee', 'character': 'Tirana', 'gender': 0}, {'name': 'Jackie Earle Haley', 'character': 'Richard Sayre', 'gender': 1}, {'name': 'Katheryn Winnick', 'character': 'Laurie Chambers', 'gender': 0}, {'name': 'Dennis Haysbert', 'character': 'Steven Deschain', 'gender': 1}, {'name': 'Michael Barbieri', 'character': 'Timmy', 'gender': 1}, {'name': 'José Zúñiga', 'character': 'Dr. Hotchkiss', 'gender': 1}, {'name': 'Alex McGregor', 'character': 'Susan Delgado', 'gender': 2}, {'name': 'Nicholas Hamilton', 'character': 'Lucas Hanson', 'gender': 1}, {'name': 'Nicholas Pauling', 'character': 'Lon', 'gender': 1}, {'name': 'Eva Kaminsky', 'character': 'Jill', 'gender': 2}, {'name': 'Robbie McLean', 'character': 'Toby', 'gender': 2}, {'name': 'Matthew Thomson', 'character': 'Jonah', 'gender': 2}, {'name': 'Karl Thaning', 'character': 'Elmer Chambers', 'gender': 1}, {'name': 'Lemogang Tsipa', 'character': 'Phedon', 'gender': 2}, {'name': 'Robert Whitehead', 'character': 'Cantab', 'gender': 1}, {'name': 'Leeanda Reddy', 'character': 'Dr. Weizak', 'gender': 0}, {'name': 'Kenneth Fok', 'character': 'Johnny', 'gender': 2}, {'name': 'Charlize Churcher', 'character': 'Pretty Girl', 'gender': 0}, {'name': 'Ben Gavin', 'character': 'Soldier', 'gender': 1}, {'name': 'Sara Cicilian', 'character': 'Drunk Girl', 'gender': 0}, {'name': 'Lara Adine Lipschitz', 'character': 'Nurse', 'gender': 0}, {'name': 'Jabari Gray', 'character': 'Bystander', 'gender': 1}, {'name': 'Paul Riley Fox', 'character': 'Militia', 'gender': 1}, {'name': 'Mark Elderkin', 'character': 'Rat Man', 'gender': 2}, {'name': 'Joe Vaz', 'character': 'Worker Taheen', 'gender': 1}, {'name': 'Christopher Stein', 'character': 'Big Guard', 'gender': 1}, {'name': 'Andre Robinson', 'character': 'Boy in the Park', 'gender': 2}, {'name': 'Alfredo Narciso', 'character': 'Ex-Breaker', 'gender': 1}, {'name': 'Dan Hirst', 'character': 'Village Engineer', 'gender': 1}, {'name': 'Zak Rowlands', 'character': 'Taheen Tech', 'gender': 1}, {'name': 'Christiaan Schoombie', 'character': 'Young Reactor Man', 'gender': 2}, {'name': 'Keci Eatock', 'character': 'Vampire Girl', 'gender': 0}, {'name': 'Marmee Cosico', 'character': 'Chinatown Pedestrian', 'gender': 0}, {'name': 'Bill Walters', 'character': 'Homeless Man in Alley', 'gender': 2}, {'name': 'Inge Beckmann', 'character': 'Teacher', 'gender': 2}, {'name': 'Reon Van Der Watt', 'character': 'Tracker Leader', 'gender': 1}, {'name': 'Richard Lothian', 'character': 'Fierce Taheen', 'gender': 1}, {'name': 'Larry Nuñez', 'character': 'Van Vampire', 'gender': 1}, {'name': 'Michael Basile', 'character': 'Pedestrian', 'gender': 2}, {'name': 'Warren Masemola', 'character': 'Bald Old Man', 'gender': 2}, {'name': 'Emily Child', 'character': 'Herb Woman', 'gender': 0}, {'name': 'Zanne Solomon', 'character': 'Restaurant Lady', 'gender': 0}, {'name': 'Mary Twala', 'character': 'Bell Lady', 'gender': 0}, {'name': 'Sarah Potter', 'character': 'Mannie Mother', 'gender': 0}, {'name': 'Andrew Laubscher', 'character': 'Dying Man', 'gender': 2}], [{'name': 'Matt Damon', 'character': 'Jason Bourne', 'gender': 1}, {'name': 'Alicia Vikander', 'character': 'Heather Lee', 'gender': 0}, {'name': 'Tommy Lee Jones', 'character': 'Robert Dewey', 'gender': 1}, {'name': 'Vincent Cassel', 'character': 'Asset', 'gender': 1}, {'name': 'Julia Stiles', 'character': 'Nicky Parsons', 'gender': 0}, {'name': 'Riz Ahmed', 'character': 'Aaron Kalloor', 'gender': 1}, {'name': 'Ato Essandoh', 'character': 'Craig Jeffers', 'gender': 1}, {'name': 'Scott Shepherd', 'character': 'Director NI Edwin Russell', 'gender': 2}, {'name': 'Bill Camp', 'character': 'Malcolm Smith', 'gender': 1}, {'name': 'Vinzenz Kiefer', 'character': 'Christian Dassault', 'gender': 1}, {'name': 'Stephen Kunken', 'character': 'Baumen', 'gender': 1}, {'name': 'Ben Stylianou', 'character': 'Greek Van Driver', 'gender': 2}, {'name': 'Kaya Yuzuki', 'character': 'Hacker', 'gender': 0}, {'name': "Matthew O'Neill", 'character': 'Lead Hub Tech', 'gender': 2}, {'name': 'Lizzie Phillips', 'character': 'Cyber Hub Tech', 'gender': 0}, {'name': 'Paris Stangl', 'character': 'Athens Alpha Agent', 'gender': 2}, {'name': 'Gregg Henry', 'character': 'Richard Webb', 'gender': 1}, {'name': 'Matt Blair', 'character': 'Hub Tech', 'gender': 2}, {'name': 'Amy De Bhrún', 'character': 'Hub Tech', 'gender': 0}, {'name': 'Akie Kotabe', 'character': 'Hub Tech', 'gender': 1}, {'name': 'Robin Crouch', 'character': 'Hub Tech', 'gender': 2}, {'name': 'Joe Kennard', 'character': 'Athens Bravo Agent', 'gender': 2}, {'name': 'Miguel Alves-Khan', 'character': 'Athens Bravo Agent', 'gender': 2}, {'name': 'Robert Stanton', 'character': 'Government Lawyer', 'gender': 1}, {'name': 'Duran Fulton Brown', 'character': 'London Alpha Agent', 'gender': 1}, {'name': 'Charles Jarman', 'character': 'London Bravo Agent', 'gender': 2}, {'name': 'Jay Vincent Diaz', 'character': 'Immigration Officer', 'gender': 2}, {'name': 'Richard Nunez', 'character': 'CIA Security Detail', 'gender': 2}, {'name': 'Sonny Robertson', 'character': 'CIA Security Detail', 'gender': 2}, {'name': 'James Dormuth', 'character': 'CIA Security Detail', 'gender': 2}, {'name': 'Dexter Emery', 'character': 'CIA Security Detail', 'gender': 2}, {'name': 'Jorge Luis Alvarez', 'character': 'Las Vegas Event Organizer', 'gender': 2}, {'name': 'Ava Katharina Maria Hoeller', 'character': 'Las Vegas Hotel Receptionist', 'gender': 0}, {'name': 'Shane Williams', 'character': "Kalloor's Aria Security", 'gender': 2}, {'name': 'Frank Roskowski', 'character': 'Loading Dock Security', 'gender': 2}, {'name': 'Johnny Cicco', 'character': 'Bradley Samuels', 'gender': 1}, {'name': 'Martin Daniel Latham', 'character': "Dewey's CIA Agent", 'gender': 2}, {'name': 'Trevor White', 'character': 'Senior Agent Collier', 'gender': 1}, {'name': 'Sasha Larkin', 'character': 'Las Vegas Police', 'gender': 2}, {'name': 'Barrie Brown', 'character': 'Las Vegas Police', 'gender': 2}, {'name': 'Stuart Jeffrey Cram', 'character': 'Las Vegas Vendor', 'gender': 2}, {'name': 'Brian Duda', 'character': 'Las Vegas Security Guard', 'gender': 2}, {'name': 'Bobby Akers', 'character': 'Conventioneer (uncredited)', 'gender': 2}, {'name': 'Hugo Alonzo', 'character': 'ExoCon Conventioneer (uncredited)', 'gender': 2}, {'name': 'Griffin Andrews', 'character': 'ExoCon Conventioneer (uncredited)', 'gender': 2}, {'name': 'Jeremy Angel', 'character': 'ExoCon Photographer / Conventioneer (uncredited)', 'gender': 2}, {'name': 'Jozef Aoki', 'character': 'Japanese Hacker (uncredited)', 'gender': 1}, {'name': 'Sarah Armstrong', 'character': 'Tattoo (uncredited)', 'gender': 0}, {'name': 'Robert-Anthony Artlett', 'character': 'Office Worker (uncredited)', 'gender': 2}, {'name': 'Lasco Atkins', 'character': 'Conference Attendee (uncredited)', 'gender': 1}, {'name': 'Alphonso Austin', 'character': 'Las Vegas Squatter (uncredited)', 'gender': 2}, {'name': 'Martin Ballantyne', 'character': 'Commuter (uncredited)', 'gender': 2}, {'name': 'Tim Baros', 'character': 'CIA Agent (uncredited)', 'gender': 2}, {'name': 'Gintare Beinoraviciute', 'character': 'City Worker (uncredited)', 'gender': 0}, {'name': 'Marc Benanti', 'character': 'CIA Agent (uncredited)', 'gender': 1}, {'name': 'Greg Benoit', 'character': 'Hotel Chef (uncredited)', 'gender': 2}, {'name': 'Tom Bonello', 'character': 'High Roller / Driver (uncredited)', 'gender': 2}, {'name': 'Charles-Jean Boucher', 'character': 'Cyber Technician (uncredited)', 'gender': 2}, {'name': 'Allen Bracken', 'character': 'Black Tie Guest (uncredited)', 'gender': 2}, {'name': "Vin'Cenzo Burgess", 'character': 'Entourage (uncredited)', 'gender': 2}, {'name': 'Anthony Burkhalter', 'character': 'Traveler (uncredited)', 'gender': 2}, {'name': 'Eduardo Jed Camara', 'character': 'Student (uncredited)', 'gender': 2}, {'name': 'Hunter Cannistraci', 'character': 'ExoCon Conventioneer (uncredited)', 'gender': 2}, {'name': 'Lenisa Ann Careaga', 'character': 'Conventioneer (uncredited)', 'gender': 0}, {'name': 'Nicole Chauvet', 'character': 'High Roller / Conventioneer (uncredited)', 'gender': 0}, {'name': 'Shin-Fei Chen', 'character': 'Cyber Technician (uncredited)', 'gender': 2}, {'name': 'Jeannine Comeau', 'character': 'Casino Patron (uncredited)', 'gender': 0}, {'name': 'Constance Consola', 'character': 'ExoCon Vendor / Conventioneer (uncredited)', 'gender': 0}, {'name': 'Angel Contreras', 'character': 'ExoCon Conventioneer (uncredited)', 'gender': 2}, {'name': 'Alexander Cooper', 'character': 'City Worker (uncredited)', 'gender': 2}, {'name': 'Jesse M. Cooper', 'character': 'CIA Hub Agent (uncredited)', 'gender': 2}, {'name': 'Graig Couton', 'character': 'Police Officer (uncredited)', 'gender': 2}, {'name': 'Lisamarie Cowan', 'character': 'Casino Patron (uncredited)', 'gender': 0}, {'name': 'Roger Julian Cross', 'character': 'ExoCon Conventioneer / Pedestrian (uncredited)', 'gender': 2}, {'name': 'Gioacchino Jim Cuffaro', 'character': 'Greek Demostrator (uncredited)', 'gender': 1}, {'name': 'Graham Curry', 'character': 'City Worker (uncredited)', 'gender': 1}, {'name': 'Darlene Dalmaceda', 'character': 'Casino Patron / Pedestrian (uncredited)', 'gender': 0}, {'name': 'Erin C. Davis', 'character': "Amanda (Kalloor's Aide) (uncredited)", 'gender': 0}, {'name': 'Christine de Lota', 'character': 'ExoCon Conventioneer (uncredited)', 'gender': 0}, {'name': 'Steven I. Dillard', 'character': 'Media Photographer (uncredited)', 'gender': 1}, {'name': 'Nick Donald', 'character': 'Security gaurd (uncredited)', 'gender': 2}, {'name': 'Timothy Skyler Dunigan', 'character': 'Cab Driver (uncredited)', 'gender': 2}, {'name': 'Andrew Dunkelberger', 'character': 'Squatter / Cyberhub Hacker (uncredited)', 'gender': 1}, {'name': 'Paul Edney', 'character': 'Las Vegas Cab Driver (uncredited)', 'gender': 1}, {'name': 'Barbara Edwards', 'character': 'Business Woman (uncredited)', 'gender': 0}, {'name': 'Heiko Effenberger', 'character': 'Tourist (uncredited)', 'gender': 2}, {'name': 'Aaron Egawa', 'character': 'Casino Guest & Black Tie (uncredited)', 'gender': 2}, {'name': 'Daniel Eghan', 'character': 'Bare Knuckle Fight Spectator (uncredited)', 'gender': 2}, {'name': 'Clara Emanuel', 'character': 'Conventioneer (uncredited)', 'gender': 2}, {'name': 'Ricardo Ewert', 'character': 'Tourist (uncredited)', 'gender': 2}, {'name': 'Dino Fazzani', 'character': 'Bare Knuckle Fight Spectator (uncredited)', 'gender': 2}, {'name': 'Nathan Ferguson', 'character': 'Dream Staff (uncredited)', 'gender': 2}, {'name': 'Joseph D Fisher', 'character': 'Deep Dream Staff (uncredited)', 'gender': 2}, {'name': 'Ellie Fox', 'character': 'Officer Jones (uncredited)', 'gender': 0}, {'name': 'Neve Gachev', 'character': 'CIA Agent (uncredited)', 'gender': 0}, {'name': 'Alexander Garganera', 'character': 'Airplane Passenger (uncredited)', 'gender': 2}, {'name': 'Michael Garvey', 'character': 'Restaurant Patron (uncredited)', 'gender': 2}, {'name': 'Gladis Giada', 'character': 'ExoCon Conventioneer (uncredited)', 'gender': 0}, {'name': 'Alex Gillison', 'character': 'Driver (uncredited)', 'gender': 2}, {'name': 'Philip Greene', 'character': 'Jogger (uncredited)', 'gender': 1}, {'name': 'Adam Gregory', 'character': 'Flight Attendant (uncredited)', 'gender': 2}, {'name': 'Shane Griffin', 'character': 'Buinessman (uncredited)', 'gender': 2}, {'name': 'Guna Gultniece', 'character': 'Businesswoman (uncredited)', 'gender': 0}, {'name': 'Kevin Hager', 'character': 'Casino Guest (uncredited)', 'gender': 2}, {'name': 'Phillip Allen Hall III', 'character': 'Conventioneer (uncredited)', 'gender': 2}, {'name': 'Yumiko Hanasaka', 'character': 'Business Woman (uncredited)', 'gender': 0}, {'name': 'Jeanette Hatlestad', 'character': 'ExoCon Conventioneer (uncredited)', 'gender': 0}, {'name': 'Michael Haydon', 'character': 'City Worker (uncredited)', 'gender': 2}, {'name': 'John Heartstone', 'character': 'Cyberhub Tech (uncredited)', 'gender': 2}, {'name': 'David Hershwitzky', 'character': 'Conventioneer (uncredited)', 'gender': 2}, {'name': 'Jamie Hodge', 'character': 'Jogger (uncredited)', 'gender': 0}, {'name': 'Yusuf Hofri', 'character': 'Boy (uncredited)', 'gender': 1}, {'name': 'Ken Holliday', 'character': 'DC Pedestrian (uncredited)', 'gender': 2}, {'name': 'Oliver Hollingdale', 'character': 'Athens Spectator (uncredited)', 'gender': 2}, {'name': 'Kornelia Horvath', 'character': 'City Worker (uncredited)', 'gender': 0}, {'name': 'Romulus Hotea', 'character': 'Bare Knuckle Fight Spectator (uncredited)', 'gender': 2}, {'name': 'Jahmilla Jackson', 'character': 'Slot Machine Player (uncredited)', 'gender': 2}, {'name': 'Dolly Jagdeo', 'character': 'Pedestrian / Café Customer (uncredited)', 'gender': 0}, {'name': 'Bron James', 'character': 'Businessman (uncredited)', 'gender': 2}, {'name': 'Jimi James', 'character': 'Las Vegas Squatter (uncredited)', 'gender': 2}, {'name': 'Kyle James', 'character': "CFO's Personal Assistant (uncredited)", 'gender': 2}, {'name': 'Natasha Jenssen', 'character': 'Greek Protester (uncredited)', 'gender': 0}, {'name': 'Kyle Jerichow', 'character': 'Cyber Hub technician (uncredited)', 'gender': 1}, {'name': 'Jauhara Jivanji', 'character': 'CIA Agent (uncredited)', 'gender': 2}, {'name': 'Jemarko Jones', 'character': 'Conventioneer / Dishwasher (uncredited)', 'gender': 2}, {'name': 'Mark Justice', 'character': 'SWAT Leader (uncredited)', 'gender': 1}, {'name': 'KaiCarra', 'character': 'Casino Patron (uncredited)', 'gender': 0}, {'name': 'Lampros Kalfuntzos', 'character': 'Greek Protester (uncredited)', 'gender': 1}, {'name': 'Attila G. Kerekes', 'character': 'City Worker (uncredited)', 'gender': 2}, {'name': 'Kyle Kesterson', 'character': 'Drone Operator (uncredited)', 'gender': 2}, {'name': 'Marat Khairoullin', 'character': 'Bare Knuckle Fight Spectator (uncredited)', 'gender': 2}, {'name': 'Kaveh Khatiri', 'character': 'Boxing Spectator (uncredited)', 'gender': 2}, {'name': 'Slim Khezri', 'character': 'Exocon Attendee (uncredited)', 'gender': 1}, {'name': 'Elle van Knoll', 'character': 'Press Member (uncredited)', 'gender': 0}, {'name': 'Steven Krasner', 'character': 'Casino Slots Changer (uncredited)', 'gender': 2}, {'name': 'Vivian Yoon Lee', 'character': 'Executive Assistant (Deep Dream) (uncredited)', 'gender': 0}, {'name': 'Kamil Lemieszewski', 'character': 'Greek Protestant (uncredited)', 'gender': 1}, {'name': 'Matthew Leonhart', 'character': 'Conference attendee (uncredited)', 'gender': 2}, {'name': 'David Ae Levy', 'character': 'ExoCon Conventioneer (uncredited)', 'gender': 2}, {'name': 'Raul Limon', 'character': 'Casino Patron (uncredited)', 'gender': 2}, {'name': 'Frank Lui Geo', 'character': 'ExoCon Philanthropist (uncredited)', 'gender': 2}, {'name': 'Kevin Lye', 'character': 'Constitution Avenue businessman (uncredited)', 'gender': 2}, {'name': 'Rob Marchitti', 'character': 'Casino Guest (uncredited)', 'gender': 2}, {'name': 'Maria Marra', 'character': 'Conventioneer (uncredited)', 'gender': 0}, {'name': 'Garry Marriott', 'character': 'Paramedic (uncredited)', 'gender': 2}, {'name': 'Alex Martin', 'character': 'Athens Bravo Agent (uncredited)', 'gender': 2}, {'name': 'Jason Matthewson', 'character': 'Entourage (uncredited)', 'gender': 2}, {'name': 'Stephanie McIntyre', 'character': 'Conventioneer (uncredited)', 'gender': 0}, {'name': 'Amelie McKendry', 'character': 'Pedestrian (uncredited)', 'gender': 0}, {'name': 'Neal McNeil', 'character': 'Restaurant Patron (uncredited)', 'gender': 2}, {'name': 'Addyson Medley', 'character': 'Hacker (uncredited)', 'gender': 2}, {'name': 'Andy Mihalache', 'character': 'Businessman / City Worker (uncredited)', 'gender': 2}, {'name': 'Anthony Molinari', 'character': 'Cop 2 (uncredited)', 'gender': 1}, {'name': 'Robb Moon', 'character': 'Casino Patron (uncredited)', 'gender': 2}, {'name': 'Sean Moon', 'character': 'Sign Language Interpreter (uncredited)', 'gender': 2}, {'name': 'Paul A Munday', 'character': 'Passerby (uncredited)', 'gender': 2}, {'name': 'Ashley Nicole Murray', 'character': 'Conventioneer (uncredited)', 'gender': 2}, {'name': 'Celina Nessa', 'character': 'Tourist (uncredited)', 'gender': 0}, {'name': 'Emily Ng', 'character': 'CIA Agent (uncredited)', 'gender': 0}, {'name': 'Brodi Nicholas', 'character': 'ExoCon Conventioneer (uncredited)', 'gender': 2}, {'name': 'Jason Novak', 'character': 'Casino Patron (uncredited)', 'gender': 2}, {'name': 'Emeson Nwolie', 'character': 'Businessman (uncredited)', 'gender': 1}, {'name': 'Lisa Nygard-Pugh', 'character': 'Casino Guest (uncredited)', 'gender': 0}, {'name': 'Angelo Olivier', 'character': 'Reporter / Rioter (uncredited)', 'gender': 2}, {'name': 'Sarah Ospina', 'character': 'Exocon Conventioneer (uncredited)', 'gender': 0}, {'name': 'B.J. Parker', 'character': 'Police Officer (uncredited)', 'gender': 2}, {'name': 'Mac Pietowski', 'character': 'Office Interior (uncredited)', 'gender': 2}, {'name': 'Dean Preston', 'character': 'Casino Patron (uncredited)', 'gender': 2}, {'name': 'Damon Proctor', 'character': 'Convention Attendee (uncredited)', 'gender': 2}, {'name': 'Jon Prophet', 'character': 'CIA Worker (uncredited)', 'gender': 2}, {'name': 'Abigail Rich', 'character': 'Las Vegas Showgirl (uncredited)', 'gender': 0}, {'name': 'Lovell Richards', 'character': 'City Worker (uncredited)', 'gender': 2}, {'name': 'Jarrett Ricker', 'character': 'Goon (uncredited)', 'gender': 2}, {'name': 'Joan Riegert', 'character': 'Black Tie Guest (uncredited)', 'gender': 0}, {'name': 'Brian Robak', 'character': 'ExoCon Conventioneer (uncredited)', 'gender': 2}, {'name': 'Belen Rosenberg', 'character': 'ExoCon Attendee (uncredited)', 'gender': 0}, {'name': 'Andre Rutherford', 'character': 'ExoCon Conventioneer (uncredited)', 'gender': 2}, {'name': 'Nunzio Santoro', 'character': 'Hacker (uncredited)', 'gender': 2}, {'name': 'Shane Santos', 'character': 'ExoCon Conventioneer (uncredited)', 'gender': 2}, {'name': 'Lani Sarem', 'character': 'Conventioneer (uncredited)', 'gender': 0}, {'name': 'Kaleigh Saunders', 'character': 'Conference Attendee (uncredited)', 'gender': 0}, {'name': 'Nicolas Savidis-Macris', 'character': 'Fight Spectator (uncredited)', 'gender': 2}, {'name': 'Mark Sawtelle', 'character': 'Bus Passenger / Pedestrian (uncredited)', 'gender': 2}, {'name': 'Earl Vincent Sherwood II', 'character': 'Police Officer (uncredited)', 'gender': 2}, {'name': 'Adonis Simmons', 'character': 'Tech Convention Participant (uncredited)', 'gender': 2}, {'name': 'Tina Simmons', 'character': 'Businesswoman (uncredited)', 'gender': 0}, {'name': 'Neil Alexander Smith', 'character': 'Bare Knuckle Fight Punter (uncredited)', 'gender': 2}, {'name': 'Eric Sparks', 'character': 'Police Officer (uncredited)', 'gender': 2}, {'name': 'Richard Stanley', 'character': 'Casino Patron / Athens Protester (uncredited)', 'gender': 1}, {'name': 'Elicia Stokes Navarro', 'character': 'Conventioneer (uncredited)', 'gender': 0}, {'name': 'Alyx Stone', 'character': 'Doctor (uncredited)', 'gender': 2}, {'name': 'Aaron Strong', 'character': 'ExoCon Goon (uncredited)', 'gender': 2}, {'name': 'Georgie-May Tearle', 'character': 'CIA Agent (uncredited)', 'gender': 0}, {'name': 'Paul Terry', 'character': 'Bare Knuckle Fight Punter (uncredited)', 'gender': 2}, {'name': 'Tony Toste', 'character': 'CIA Agent (uncredited)', 'gender': 2}, {'name': 'Michael A. Tushaus', 'character': 'Casino Guest (uncredited)', 'gender': 2}, {'name': 'Vassiliki Tzanakou', 'character': 'Athenian Passerby (uncredited)', 'gender': 0}, {'name': 'Luke Van Bergen', 'character': 'Deep Dream Staff (uncredited)', 'gender': 2}, {'name': 'Tony Vivio', 'character': 'Casino Patron (uncredited)', 'gender': 2}, {'name': 'Sheila Vossough', 'character': 'Cocktail Waitress (uncredited)', 'gender': 0}, {'name': 'Marla Aaron Wapner', 'character': 'DC Diner (uncredited)', 'gender': 0}, {'name': 'Don Whatley', 'character': 'Secret Service (uncredited)', 'gender': 2}, {'name': 'LaFonda Whitehead', 'character': 'Tech Convention Participant (uncredited)', 'gender': 2}, {'name': 'Chris Wolfe', 'character': 'Deep Dream Staff (uncredited)', 'gender': 2}, {'name': 'Andie Ximenes', 'character': 'Clean Cut Man (uncredited)', 'gender': 2}, {'name': 'Alexander Yassin', 'character': 'Malaysian Tourist (uncredited)', 'gender': 2}, {'name': 'Brad Hamerly', 'character': 'Deep Dream Staff (uncredited)', 'gender': 2}], [{'name': 'David Niven', 'character': 'Phileas Fogg', 'gender': 1}, {'name': 'Cantinflas', 'character': 'Passepartout', 'gender': 1}, {'name': 'Finlay Currie', 'character': 'Whist Partner', 'gender': 1}, {'name': 'Robert Morley', 'character': 'Ralph', 'gender': 1}, {'name': 'Charles Boyer', 'character': 'Monsieur Gasse', 'gender': 1}, {'name': 'Robert Newton', 'character': 'Mr. Fix', 'gender': 1}, {'name': 'Gilbert Roland', 'character': 'Achmed Abdullah', 'gender': 1}, {'name': 'Marlene Dietrich', 'character': 'Saloon Hostess', 'gender': 0}, {'name': 'Frank Sinatra', 'character': 'Saloon Pianist', 'gender': 1}, {'name': 'Buster Keaton', 'character': 'Train Conductor', 'gender': 1}, {'name': 'John Carradine', 'character': 'Col. Proctor Stamp', 'gender': 1}, {'name': 'Peter Lorre', 'character': 'Steward', 'gender': 1}, {'name': 'George Raft', 'character': 'Saloon Bouncer', 'gender': 1}, {'name': 'Tim McCoy', 'character': 'Colonel', 'gender': 1}, {'name': 'Joe E. Brown', 'character': 'Stationmaster', 'gender': 2}, {'name': 'Shirley MacLaine', 'character': 'Princess Aouda', 'gender': 0}, {'name': 'Melville Cooper', 'character': 'Mr. Talley', 'gender': 1}, {'name': 'Reginald Denny', 'character': 'Police Chief', 'gender': 1}, {'name': 'Ronald Colman', 'character': 'Railway Official', 'gender': 1}, {'name': 'Trevor Howard', 'character': 'Denis Fallentin', 'gender': 1}, {'name': 'Harcourt Williams', 'character': 'Hinshaw', 'gender': 1}, {'name': 'Martine Carol', 'character': 'Tourist', 'gender': 2}, {'name': 'Cedric Hardwicke', 'character': 'Sir Francis Cromarty', 'gender': 1}, {'name': 'Noël Coward', 'character': 'Roland Hesketh-Baggott', 'gender': 1}, {'name': 'John Gielgud', 'character': 'Foster', 'gender': 1}, {'name': 'Fernandel', 'character': 'French Coachman', 'gender': 1}, {'name': 'Evelyn Keyes', 'character': 'Paris Tart', 'gender': 0}, {'name': 'José Greco', 'character': 'Flamenco Dancer', 'gender': 2}, {'name': 'Cesar Romero', 'character': "Achmed Abdullah's Henchman", 'gender': 1}, {'name': 'Alan Mowbray', 'character': 'British Consul - Suez', 'gender': 1}, {'name': 'Charles Coburn', 'character': 'Steamship Company Hong Kong Clerk', 'gender': 1}, {'name': 'Red Skelton', 'character': 'Drunk in Barbary Coast Saloon', 'gender': 1}, {'name': 'Andy Devine', 'character': 'First Mate of the S.S. Henrietta', 'gender': 1}, {'name': 'Edmund Lowe', 'character': 'Engineer of the S.S. Henrietta', 'gender': 1}, {'name': 'Victor McLaglen', 'character': 'Helmsman of the S.S. Henrietta', 'gender': 1}, {'name': 'John Mills', 'character': 'London Carriage Driver', 'gender': 1}, {'name': 'Glynis Johns', 'character': "Sporting Lady's Companion", 'gender': 0}, {'name': 'Hermione Gingold', 'character': 'Sporting Lady', 'gender': 0}, {'name': 'Edward R. Murrow', 'character': 'Prologue Narrator', 'gender': 2}, {'name': 'Mike Mazurki', 'character': 'Drunk in Hong Kong Dive', 'gender': 1}, {'name': 'Ronald Squire', 'character': 'Reform Club Member', 'gender': 2}, {'name': 'Basil Sydney', 'character': 'Reform Club Member', 'gender': 1}, {'name': 'Luis Miguel Dominguín', 'character': 'Bullfighter', 'gender': 2}, {'name': 'Robert Cabal', 'character': 'Elephant Driver-Guide', 'gender': 1}, {'name': 'Jack Oakie', 'character': "Captain of the 'S. S. Henrietta'", 'gender': 1}, {'name': 'Beatrice Lillie', 'character': 'Leader of London Revivalist Group', 'gender': 0}, {'name': 'A.E. Matthews', 'character': 'Club Member', 'gender': 1}, {'name': 'Walter Fitzgerald', 'character': 'Club Member', 'gender': 1}, {'name': 'Ronald Adam', 'character': 'Club Steward', 'gender': 1}, {'name': 'Frank Royde', 'character': 'Clergyman', 'gender': 2}, {'name': 'Abdullah Abbas', 'character': 'Extra (uncredited)', 'gender': 1}, {'name': 'Jesse Adams', 'character': 'Extra (uncredited)', 'gender': 1}, {'name': 'Fred Aldrich', 'character': 'Extra (uncredited)', 'gender': 1}, {'name': 'Ray Armstrong', 'character': 'Extra (uncredited)', 'gender': 2}, {'name': 'Gertrude Astor', 'character': 'Extra (uncredited)', 'gender': 0}, {'name': 'Walter Bacon', 'character': 'Extra (uncredited)', 'gender': 2}, {'name': 'Rama Bai', 'character': 'Extra (uncredited)', 'gender': 2}, {'name': 'Leah Baird', 'character': 'Extra (uncredited)', 'gender': 0}, {'name': 'Brandon Beach', 'character': 'Extra (uncredited)', 'gender': 1}, {'name': 'Eugene Beday', 'character': 'Extra (uncredited)', 'gender': 2}, {'name': 'Helena Benda', 'character': 'Extra (uncredited)', 'gender': 2}, {'name': 'Audrey Betz', 'character': 'Extra (uncredited)', 'gender': 0}, {'name': 'George Blagoi', 'character': 'Extra (uncredited)', 'gender': 2}, {'name': 'Eumenio Blanco', 'character': 'Extra (uncredited)', 'gender': 1}, {'name': 'Nina Borget', 'character': 'Extra (uncredited)', 'gender': 2}, {'name': 'Danny Borzage', 'character': 'Extra (uncredited)', 'gender': 1}, {'name': 'Hazel Boyne', 'character': 'Extra (uncredited)', 'gender': 0}, {'name': 'George Bruggeman', 'character': 'Extra (uncredited)', 'gender': 2}, {'name': 'Bob Burrows', 'character': 'Extra (uncredited)', 'gender': 1}, {'name': 'Paul Busch', 'character': 'Extra (uncredited)', 'gender': 1}, {'name': 'Gordon Carveth', 'character': 'Extra (uncredited)', 'gender': 2}, {'name': 'Spencer Chan', 'character': 'Extra (uncredited)', 'gender': 1}, {'name': 'Jack Chefe', 'character': 'Extra (uncredited)', 'gender': 1}, {'name': 'Sing Chen', 'character': 'Extra (uncredited)', 'gender': 2}, {'name': 'Dick Cherney', 'character': 'Extra (uncredited)', 'gender': 2}, {'name': 'Bud Cokes', 'character': 'Extra (uncredited)', 'gender': 1}, {'name': 'Louise Colombet', 'character': 'Extra (uncredited)', 'gender': 2}, {'name': 'Bill Couch', 'character': 'Extra (uncredited)', 'gender': 1}, {'name': 'Paul Cristo', 'character': 'Extra (uncredited)', 'gender': 1}, {'name': 'Roy Damron', 'character': 'Extra (uncredited)', 'gender': 2}, {'name': 'Eddie Das', 'character': 'Extra (uncredited)', 'gender': 2}, {'name': 'John Davidson', 'character': 'Extra (uncredited)', 'gender': 1}, {'name': 'Jack Davies', 'character': 'Extra (uncredited)', 'gender': 1}, {'name': 'Jack Davis', 'character': 'Extra (uncredited)', 'gender': 1}, {'name': 'Anna De Linsky', 'character': 'Extra (uncredited)', 'gender': 2}, {'name': 'Gloria Dea', 'character': 'Extra (uncredited)', 'gender': 2}, {'name': 'John Deauville', 'character': 'Extra (uncredited)', 'gender': 2}, {'name': 'Harry Denny', 'character': 'Extra (uncredited)', 'gender': 2}, {'name': 'James Dime', 'character': 'Extra (uncredited)', 'gender': 1}, {'name': 'Joe Dougherty', 'character': 'Extra (uncredited)', 'gender': 1}, {'name': 'Dan Dowling', 'character': 'Extra (uncredited)', 'gender': 2}, {'name': 'Harry Duff', 'character': 'Extra (uncredited)', 'gender': 1}, {'name': 'Arthur Dulac', 'character': 'Extra (uncredited)', 'gender': 2}, {'name': 'Charles Dunbar', 'character': 'Extra (uncredited)', 'gender': 2}, {'name': 'Renald Dupont', 'character': 'Extra (uncredited)', 'gender': 2}, {'name': 'Larry Duran', 'character': 'Extra (uncredited)', 'gender': 1}, {'name': 'Minta Durfee', 'character': 'Extra (uncredited)', 'gender': 2}, {'name': 'Jack Ellis', 'character': 'Extra (uncredited)', 'gender': 2}, {'name': 'Richard Elmore', 'character': 'Extra (uncredited)', 'gender': 2}, {'name': 'Frank Erickson', 'character': 'Extra (uncredited)', 'gender': 2}, {'name': 'Bob Evans', 'character': 'Extra (uncredited)', 'gender': 1}, {'name': 'Harry Evans', 'character': 'Extra (uncredited)', 'gender': 2}, {'name': 'Franklyn Farnum', 'character': 'Extra (uncredited)', 'gender': 1}, {'name': 'Art Felix', 'character': 'Extra (uncredited)', 'gender': 2}, {'name': 'Grace Field', 'character': 'Extra (uncredited)', 'gender': 2}, {'name': 'Sam Finn', 'character': 'Extra (uncredited)', 'gender': 1}, {'name': 'Bess Flowers', 'character': 'Extra (uncredited)', 'gender': 0}, {'name': 'Frances Fong', 'character': 'Extra (uncredited)', 'gender': 0}, {'name': 'Frances Fong', 'character': 'Featured Player (uncredited)', 'gender': 0}, {'name': 'Otto Forrest', 'character': 'Extra (uncredited)', 'gender': 2}, {'name': 'Helen Foster', 'character': 'Extra (uncredited)', 'gender': 0}, {'name': 'Jesús Franco', 'character': 'Extra (uncredited)', 'gender': 1}, {'name': 'Ben Frommer', 'character': 'Extra (uncredited)', 'gender': 2}, {'name': 'Curt Furburg', 'character': 'Extra (uncredited)', 'gender': 2}, {'name': 'Joe Garcio', 'character': 'Extra (uncredited)', 'gender': 2}, {'name': 'Joe Gilbert', 'character': 'Extra (uncredited)', 'gender': 2}, {'name': 'Mary Ellen Gleason', 'character': 'Extra (uncredited)', 'gender': 2}, {'name': 'June Glory', 'character': 'Extra (uncredited)', 'gender': 2}, {'name': 'Albert Godderis', 'character': 'Extra (uncredited)', 'gender': 2}, {'name': 'James Gonzalez', 'character': 'Extra (uncredited)', 'gender': 2}, {'name': 'Carmelita González', 'character': 'Extra (uncredited)', 'gender': 0}, {'name': 'Dick Gordon', 'character': 'Extra (uncredited)', 'gender': 1}, {'name': 'Bernie Gozier', 'character': 'Extra (uncredited)', 'gender': 1}, {'name': 'William Graeff Jr.', 'character': 'Extra (uncredited)', 'gender': 1}, {'name': 'Herschel Graham', 'character': 'Extra (uncredited)', 'gender': 2}, {'name': 'Donald Gray', 'character': 'Extra (uncredited)', 'gender': 1}, {'name': 'Kit Guard', 'character': 'Extra (uncredited)', 'gender': 1}, {'name': 'Paul Gustine', 'character': 'Extra (uncredited)', 'gender': 2}, {'name': 'Herman Hack', 'character': 'Extra (uncredited)', 'gender': 1}, {'name': 'Robert Haines', 'character': 'Extra (uncredited)', 'gender': 2}, {'name': 'Stuart Hall', 'character': 'Extra (uncredited)', 'gender': 2}, {'name': 'Chick Hannan', 'character': 'Extra (uncredited)', 'gender': 2}, {'name': 'Silver Harr', 'character': 'Extra (uncredited)', 'gender': 2}, {'name': 'Sam Harris', 'character': 'Extra (uncredited)', 'gender': 1}, {'name': 'Clarence Hennecke', 'character': 'Extra (uncredited)', 'gender': 1}, {'name': 'Lars Hensen', 'character': 'Extra (uncredited)', 'gender': 1}, {'name': 'George Hickman', 'character': 'Extra (uncredited)', 'gender': 2}, {'name': 'Chuck Hicks', 'character': 'Extra (uncredited)', 'gender': 1}, {'name': 'Stuart Holmes', 'character': 'Extra (uncredited)', 'gender': 1}, {'name': 'Hans Hopf', 'character': 'Extra (uncredited)', 'gender': 2}, {'name': 'Jimmie Horan', 'character': 'Extra (uncredited)', 'gender': 1}, {'name': 'Yoneo Iguchi', 'character': 'Extra (uncredited)', 'gender': 2}, {'name': 'Allen Jaffe', 'character': 'Extra (uncredited)', 'gender': 1}, {'name': 'Robert James', 'character': 'Extra (uncredited)', 'gender': 1}, {'name': 'Dolly Jarvis', 'character': 'Extra (uncredited)', 'gender': 2}, {'name': 'Michael Jeffers', 'character': 'Extra (uncredited)', 'gender': 1}, {'name': 'Leroy Johnson', 'character': 'Extra (uncredited)', 'gender': 2}, {'name': 'Dick Johnstone', 'character': 'Extra (uncredited)', 'gender': 1}, {'name': 'Harvey Karels', 'character': 'Extra (uncredited)', 'gender': 2}, {'name': 'Willie Keeler', 'character': 'Extra (uncredited)', 'gender': 2}, {'name': 'Jack Kenny', 'character': 'Extra (uncredited)', 'gender': 1}, {'name': 'Al Kikume', 'character': 'Extra (uncredited)', 'gender': 1}, {'name': 'Max Kleven', 'character': 'Extra (uncredited)', 'gender': 1}, {'name': 'Paul Kruger', 'character': 'Extra (uncredited)', 'gender': 1}, {'name': 'Ann Kunde', 'character': 'Extra (uncredited)', 'gender': 2}, {'name': 'Jeanne Lafayette', 'character': 'Extra (uncredited)', 'gender': 2}, {'name': 'Richard LaMarr', 'character': 'Extra (uncredited)', 'gender': 2}, {'name': 'Alfredo Landa', 'character': 'Extra (uncredited)', 'gender': 1}, {'name': 'Jack Lee', 'character': 'Extra (uncredited)', 'gender': 2}, {'name': 'Virginia Ann Lee', 'character': 'Dancer (uncredited)', 'gender': 2}, {'name': 'Harry Leroy', 'character': 'Extra (uncredited)', 'gender': 2}, {'name': 'Carl M. Leviness', 'character': 'Extra (uncredited)', 'gender': 2}, {'name': 'Shirley Lew', 'character': 'Extra (uncredited)', 'gender': 2}, {'name': 'Tien-Lang Li', 'character': 'Extra (uncredited)', 'gender': 2}, {'name': 'Baron James Lichter', 'character': 'Extra (uncredited)', 'gender': 2}, {'name': 'Marco López', 'character': 'Extra (uncredited)', 'gender': 1}, {'name': 'Casey MacGregor', 'character': 'Train Fireman (uncredited)', 'gender': 2}, {'name': 'Joseph Marievsky', 'character': 'Extra (uncredited)', 'gender': 1}, {'name': 'Johnny Marlin', 'character': 'Extra (uncredited)', 'gender': 2}, {'name': 'Thomas Martin', 'character': 'Extra (uncredited)', 'gender': 1}, {'name': 'Frank McLure', 'character': 'Extra (uncredited)', 'gender': 1}, {'name': 'Marie Melesh', 'character': 'Extra (uncredited)', 'gender': 2}, {'name': 'Harold Miller', 'character': 'Extra (uncredited)', 'gender': 1}, {'name': 'Gordon Mitchell', 'character': 'Extra (uncredited)', 'gender': 1}, {'name': 'King Mojave', 'character': 'Extra (uncredited)', 'gender': 2}, {'name': 'Joe Molina', 'character': 'Extra (uncredited)', 'gender': 2}, {'name': 'Ernesto Molinari', 'character': 'Extra (uncredited)', 'gender': 2}, {'name': 'Ralph Moratz', 'character': 'Extra (uncredited)', 'gender': 1}, {'name': 'Clive Morgan', 'character': 'Extra (uncredited)', 'gender': 1}, {'name': 'Charles Morton', 'character': 'Extra (uncredited)', 'gender': 1}, {'name': 'Sol Murgi', 'character': 'Extra (uncredited)', 'gender': 2}, {'name': ' Frank Nagai', 'character': 'Extra (uncredited)', 'gender': 2}, {'name': 'George Nardelli', 'character': 'Extra (uncredited)', 'gender': 2}, {'name': 'Barry Norton', 'character': 'Extra (uncredited)', 'gender': 1}, {'name': 'Faye Michael Nuell', 'character': 'Extra (uncredited)', 'gender': 2}, {'name': 'Daniel Nunez', 'character': 'Extra (uncredited)', 'gender': 2}, {'name': "William H. O'Brien", 'character': 'Extra (uncredited)', 'gender': 1}, {'name': "Monty O'Grady", 'character': 'Extra (uncredited)', 'gender': 1}, {'name': "Lillian O'Malley", 'character': 'Extra (uncredited)', 'gender': 2}, {'name': 'Kent Odell', 'character': 'Extra (uncredited)', 'gender': 2}, {'name': 'Yasuzô Ogawa', 'character': 'Extra (uncredited)', 'gender': 1}, {'name': 'Bob Okazaki', 'character': 'Extra (uncredited)', 'gender': 2}, {'name': 'Victor Paul', 'character': 'Extra (uncredited)', 'gender': 1}, {'name': 'Charles Pendelton', 'character': 'Extra (uncredited)', 'gender': 2}, {'name': 'Elsa Peterson', 'character': 'Extra (uncredited)', 'gender': 2}, {'name': 'Joe Ploski', 'character': 'Extra (uncredited)', 'gender': 2}, {'name': 'Byron Poindexter', 'character': 'Extra (uncredited)', 'gender': 2}, {'name': 'Lucille Porcett', 'character': 'Extra (uncredited)', 'gender': 2}, {'name': 'Don Pulford', 'character': 'Extra (uncredited)', 'gender': 2}, {'name': 'Bill Raisch', 'character': 'Extra (uncredited)', 'gender': 2}, {'name': 'Fred Rapport', 'character': 'Extra (uncredited)', 'gender': 2}, {'name': 'Bob Reeves', 'character': 'Extra (uncredited)', 'gender': 1}, {'name': 'Max Reid', 'character': 'Extra (uncredited)', 'gender': 2}, {'name': 'Waclaw Rekwart', 'character': 'Extra (uncredited)', 'gender': 2}, {'name': 'Peggy Remington', 'character': 'Extra (uncredited)', 'gender': 2}, {'name': 'David Richardson', 'character': 'Extra (uncredited)', 'gender': 2}, {'name': 'Buddy Roosevelt', 'character': 'Extra (uncredited)', 'gender': 2}, {'name': 'George Ross', 'character': 'Extra (uncredited)', 'gender': 2}, {'name': 'Joe E. Ross', 'character': 'Extra (uncredited)', 'gender': 2}, {'name': 'Marion Ross', 'character': 'Extra (uncredited)', 'gender': 0}, {'name': 'Tony Roux', 'character': 'Extra (uncredited)', 'gender': 1}, {'name': 'John Roy', 'character': 'Extra (uncredited)', 'gender': 2}, {'name': 'Dick Ryan', 'character': 'Extra (uncredited)', 'gender': 1}, {'name': 'Loulette Sablon', 'character': 'Extra (uncredited)', 'gender': 2}, {'name': 'Danny Sands', 'character': 'Extra (uncredited)', 'gender': 2}, {'name': 'Cosmo Sardo', 'character': 'Extra (uncredited)', 'gender': 2}, {'name': 'Sam Savitsky', 'character': 'Extra (uncredited)', 'gender': 2}, {'name': 'Jeffrey Sayre', 'character': 'Extra (uncredited)', 'gender': 1}, {'name': 'Charles Schaeffer', 'character': 'Extra (uncredited)', 'gender': 2}, {'name': 'Fred Scheiwiller', 'character': 'Extra (uncredited)', 'gender': 2}, {'name': 'Eddie Searles', 'character': 'Extra (uncredited)', 'gender': 1}, {'name': 'Scott Seaton', 'character': 'Extra (uncredited)', 'gender': 1}, {'name': 'Bernard Sell', 'character': 'Extra (uncredited)', 'gender': 1}, {'name': 'Sammy Shack', 'character': 'Extra (uncredited)', 'gender': 2}, {'name': 'Leslie Sketchley', 'character': 'Extra (uncredited)', 'gender': 2}, {'name': 'June Smaney', 'character': 'Extra (uncredited)', 'gender': 0}, {'name': 'Mabel Smaney', 'character': 'Extra (uncredited)', 'gender': 0}, {'name': 'J. Lewis Smith', 'character': 'Extra (uncredited)', 'gender': 2}, {'name': 'Charles Soldani', 'character': 'Extra (uncredited)', 'gender': 2}, {'name': 'Stephen Soldi', 'character': 'Extra (uncredited)', 'gender': 2}, {'name': 'Kalu K. Sonkur', 'character': 'Extra (uncredited)', 'gender': 2}, {'name': 'Hayward Soo Hoo', 'character': 'Extra (uncredited)', 'gender': 2}, {'name': 'Walter Soo Hoo', 'character': 'Extra (uncredited)', 'gender': 2}, {'name': 'Ray Spiker', 'character': 'Extra (uncredited)', 'gender': 2}, {'name': 'Robert St. Angelo', 'character': 'Extra (uncredited)', 'gender': 2}, {'name': 'John Starck', 'character': 'Extra (uncredited)', 'gender': 2}, {'name': 'Norman Stevans', 'character': 'Extra (uncredited)', 'gender': 2}, {'name': 'Bert Stevens', 'character': 'Extra (uncredited)', 'gender': 1}, {'name': 'Jack Stoney', 'character': 'Extra (uncredited)', 'gender': 1}, {'name': 'Ramón Sánchez', 'character': 'Extra (uncredited)', 'gender': 2}, {'name': 'Mike Tellegen', 'character': 'Extra (uncredited)', 'gender': 1}, {'name': 'Lillian Ten Eyck', 'character': 'Extra (uncredited)', 'gender': 2}, {'name': 'Jack Tornek', 'character': 'Extra (uncredited)', 'gender': 2}, {'name': 'Arthur Tovey', 'character': 'Extra (uncredited)', 'gender': 1}, {'name': 'Michael Trubshawe', 'character': 'Extra (uncredited)', 'gender': 1}, {'name': 'Tony Urchel', 'character': 'Extra (uncredited)', 'gender': 2}, {'name': 'Sailor Vincent', 'character': 'Extra (uncredited)', 'gender': 1}, {'name': 'Sally Yarnell', 'character': 'Extra (uncredited)', 'gender': 0}, {'name': 'William Yokota', 'character': 'Extra (uncredited)', 'gender': 2}, {'name': 'Mary Young', 'character': 'Extra (uncredited)', 'gender': 0}, {'name': 'Patrick Cargill', 'character': 'Minor Role (uncredited)', 'gender': 1}, {'name': 'Richard Aherne', 'character': 'Minor Role (uncredited)', 'gender': 1}, {'name': 'Frank Baker', 'character': 'Minor Role (uncredited)', 'gender': 1}, {'name': 'John Benson', 'character': 'Minor Role (uncredited)', 'gender': 2}, {'name': 'Albert Cavens', 'character': 'Minor Role (uncredited)', 'gender': 2}, {'name': 'Fred Cavens', 'character': 'Minor Role (uncredited)', 'gender': 2}, {'name': 'Cecil Combs', 'character': 'Minor Role (uncredited)', 'gender': 2}, {'name': 'Ashley Cowan', 'character': 'Minor Role (uncredited)', 'gender': 2}, {'name': 'Roy Darmour', 'character': 'Minor Role (uncredited)', 'gender': 2}, {'name': 'Leslie Denison', 'character': 'Minor Role (uncredited)', 'gender': 1}, {'name': 'Carli Elinor', 'character': 'Minor Role (uncredited)', 'gender': 2}, {'name': 'Raoul Freeman', 'character': 'Minor Role (uncredited)', 'gender': 2}, {'name': 'Rudy Germane', 'character': 'Minor Role (uncredited)', 'gender': 2}, {'name': 'Arthur Gould-Porter', 'character': 'Minor Role (uncredited)', 'gender': 2}, {'name': 'Cameron Hall', 'character': 'Minor Role (uncredited)', 'gender': 1}, {'name': 'Chuck Hamilton', 'character': 'Minor Role (uncredited)', 'gender': 1}, {'name': 'Roddy Hughes', 'character': 'Minor Role (uncredited)', 'gender': 2}, {'name': 'Joanna Marie Jones', 'character': 'Minor Role (uncredited)', 'gender': 0}, {'name': 'Weaver Levy', 'character': 'Minor Role (uncredited)', 'gender': 1}, {'name': 'Fateh Lohani', 'character': 'Minor Role (uncredited)', 'gender': 2}, {'name': 'Joan Lora', 'character': 'Minor Role (uncredited)', 'gender': 2}, {'name': 'Manuel López', 'character': 'Minor Role (uncredited)', 'gender': 2}, {'name': 'William Meader', 'character': 'Minor Role (uncredited)', 'gender': 1}, {'name': 'Jack Mulhall', 'character': 'Minor Role (uncredited)', 'gender': 1}, {'name': 'Satini Pualoa', 'character': 'Minor Role (uncredited)', 'gender': 2}, {'name': 'Bill Shine', 'character': 'Minor Role (uncredited)', 'gender': 1}, {'name': 'Bhogwan Singh', 'character': 'Minor Role (uncredited)', 'gender': 2}, {'name': 'Cap Somers', 'character': 'Minor Role (uncredited)', 'gender': 1}, {'name': 'Philip Van Zandt', 'character': 'Minor Role (uncredited)', 'gender': 1}, {'name': 'Carol White', 'character': 'Minor Role (uncredited)', 'gender': 0}, {'name': 'Bob Whitney', 'character': 'Minor Role (uncredited)', 'gender': 1}, {'name': 'Al Bain', 'character': 'Saloon Patron (uncredited)', 'gender': 2}, {'name': 'Noble Chissell', 'character': 'Saloon Patron (uncredited)', 'gender': 1}, {'name': 'Alphonso DuBois', 'character': 'Saloon Patron (uncredited)', 'gender': 2}, {'name': 'Philip Ahn', 'character': 'Hong Kong Citizen (uncredited)', 'gender': 1}, {'name': 'Richard Loo', 'character': 'Hong Kong Saloon Manager (uncredited)', 'gender': 1}, {'name': 'Chet Brandenburg', 'character': 'Seaman (uncredited)', 'gender': 1}, {'name': 'Martin Cichy', 'character': 'Bartender (uncredited)', 'gender': 2}, {'name': 'Campbell Cotts', 'character': 'Reform Club Member (uncredited)', 'gender': 1}, {'name': 'Felix Felton', 'character': 'Reform Club Member (uncredited)', 'gender': 1}, {'name': 'Frederick Leister', 'character': 'Reform Club Member (uncredited)', 'gender': 1}, {'name': 'Norman Macowan', 'character': 'Reform Club Member (uncredited)', 'gender': 2}, {'name': 'Adolph Faylauer', 'character': 'Parade Onlooker (uncredited)', 'gender': 2}, {'name': 'Russell Meeker', 'character': 'Parade Onlooker (uncredited)', 'gender': 2}, {'name': 'Duke Fishman', 'character': 'Marcher in Parade (uncredited)', 'gender': 1}, {'name': 'John George', 'character': 'Marcher in Parade (uncredited)', 'gender': 1}, {'name': 'Al Haskell', 'character': 'Fireman in Parade (uncredited)', 'gender': 2}, {'name': 'Tex Holden', 'character': 'One-Legged Sailor (uncredited)', 'gender': 2}, {'name': 'Charles Horvath', 'character': 'Stoker (uncredited)', 'gender': 1}, {'name': "Boyd 'Red' Morgan", 'character': 'Stoker (uncredited)', 'gender': 2}, {'name': 'Shep Houghton', 'character': 'Specialty Dancer (uncredited)', 'gender': 2}, {'name': 'Walter Kingsford', 'character': "Captain of the 'Mongolia' (uncredited)", 'gender': 1}, {'name': 'Keye Luke', 'character': 'Old Man at Yokohama Travel Office (uncredited)', 'gender': 1}, {'name': 'Frank Mills', 'character': 'Barfly (uncredited)', 'gender': 2}, {'name': 'Manuel París', 'character': 'Spanish Townsman (uncredited)', 'gender': 1}, {'name': 'Jack Perrin', 'character': 'Man on Sidewalk (uncredited)', 'gender': 1}, {'name': 'Robert Robinson', 'character': 'Political Backer (uncredited)', 'gender': 2}, {'name': 'Edwin Rochelle', 'character': 'Waiter (uncredited)', 'gender': 2}, {'name': 'Phil Schumacher', 'character': 'Horseman in Parade (uncredited)', 'gender': 2}, {'name': 'Dick Wessel', 'character': 'Train Engineer (uncredited)', 'gender': 1}, {'name': 'Zamba The Lion', 'character': 'Lion (uncredited)', 'gender': 2}, {'name': 'Suzanne Ridgeway', 'character': 'Saloon Girl (uncredited)', 'gender': 0}, {'name': 'Winona Smith', 'character': 'Extra', 'gender': 0}]]

In [ ]:
df.loc[df.cast.isnull(), "cast"] =(df_total.loc[df_total.cast.isnull(), "train_bool"]*missing_cast).loc[df[df.cast.isnull()].index]
df_test.loc[df_test.cast.isnull(), "cast"] =(~df_total.loc[df_total.cast.isnull(), "train_bool"]*missing_cast).loc[df_test[df_test.cast.isnull()].index]
df_total.loc[df_total.cast.isnull(), "cast"] = np.array(missing_cast)

*  0 male, 1 female, 2 unknown gender

In [ ]:
df.loc[:, "cast_names"] = df.cast.apply(lambda x: [var["name"] for var in ast.literal_eval(str(x))])
df_test.loc[:, "cast_names"] = df_test.cast.apply(lambda x: [var["name"] for var in ast.literal_eval(str(x))])
df_total.loc[:, "cast_names"] = df_total.cast.apply(lambda x: [var["name"] for var in ast.literal_eval(str(x))])

df.loc[:, "cast_male"] = df.cast.apply(lambda x: sum([1 for var in ast.literal_eval(str(x)) if "gender" in var and var["gender"] == 0]))
df_test.loc[:, "cast_male"] = df_test.cast.apply(lambda x: sum([1 for var in ast.literal_eval(str(x)) if "gender" in var and var["gender"] == 0]))
df_total.loc[:, "cast_male"] = df_total.cast.apply(lambda x: sum([1 for var in ast.literal_eval(str(x)) if "gender" in var and var["gender"] == 0]))

df.loc[:, "cast_female"] = df.cast.apply(lambda x: sum([1 for var in ast.literal_eval(str(x)) if "gender" in var and var["gender"] == 1]))
df_test.loc[:, "cast_female"] = df_test.cast.apply(lambda x: sum([1 for var in ast.literal_eval(str(x)) if "gender" in var and var["gender"] == 1]))
df_total.loc[:, "cast_female"] = df_total.cast.apply(lambda x: sum([1 for var in ast.literal_eval(str(x)) if "gender" in var and var["gender"] == 1]))

df.loc[:, "cast_unknown_gender"] = df.cast.apply(lambda x: sum([1 for var in ast.literal_eval(str(x)) if "gender" in var and var["gender"] == 2]))
df_test.loc[:, "cast_unknown_gender"] = df_test.cast.apply(lambda x: sum([1 for var in ast.literal_eval(str(x)) if "gender" in var and var["gender"] == 2]))
df_total.loc[:, "cast_unknown_gender"] = df_total.cast.apply(lambda x: sum([1 for var in ast.literal_eval(str(x)) if "gender" in var and var["gender"] == 2]))

df.loc[:, "cast_male_ratio"] = df[["cast_male", "cast_female"]].apply(lambda x: x["cast_male"]*100/(x["cast_female"]+x["cast_male"]), axis=1)
df_test.loc[:, "cast_male_ratio"] = df_test[["cast_male", "cast_female"]].apply(lambda x: x["cast_male"]*100/(x["cast_female"]+x["cast_male"]), axis=1)
df_total.loc[:, "cast_male_ratio"] = df_total[["cast_male", "cast_female"]].apply(lambda x: x["cast_male"]*100/(x["cast_female"]+x["cast_male"]), axis=1)


In [ ]:
castMost = [x[0] for x in Counter("|".join(df["cast_names"].apply(lambda x:"|".join(x)).values).split("|")).most_common(30)]
Counter("|".join(df["cast_names"].apply(lambda x:"|".join(x)).values).split("|")).most_common(30)

In [ ]:
for cast in castMost:
    df[cast] = df.cast_names.apply(lambda x:1 if cast in x else 0)
    df_test[cast] = df_test.cast_names.apply(lambda x:1 if cast in x else 0)
    df_total[cast] = df_total.cast_names.apply(lambda x:1 if cast in x else 0)

In [ ]:
(100-df_total.groupby("year").mean().cast_male_ratio).plot(kind="bar", figsize=(20,10), color="pink");
(df_total.groupby("year").mean().cast_male_ratio).plot(kind="bar", figsize=(20,10), color="blue", bottom=(100-df_total.groupby("year").mean().cast_male_ratio));

 <font size="5">Crew</font><br/> 

* Fill missing values from tmdb webpage.

In [ ]:
missing_crew = [[{'department': 'Directing', 'name': 'Gary Rogers', 'job': 'Director', 'gender': 2}], [{'department': 'Art', 'name': 'Aline Bonetto', 'job': 'Production Design', 'gender': 2}, {'department': 'Camera', 'name': 'Matthew Jensen', 'job': 'Director of Photography', 'gender': 1}, {'department': 'Costume & Make-Up', 'name': 'Lindy Hemming', 'job': 'Costume Design', 'gender': 0}, {'department': 'Crew', 'name': 'William Moulton Marston', 'job': 'Creator', 'gender': 1}, {'department': 'Directing', 'name': 'Patty Jenkins', 'job': 'Director', 'gender': 0}, {'department': 'Editing', 'name': 'Martin Walsh', 'job': 'Editor', 'gender': 1}, {'department': 'Lighting', 'name': 'Wayne King', 'job': 'Gaffer', 'gender': 2}, {'department': 'Production', 'name': 'Charles Roven', 'job': 'Producer', 'gender': 2}, {'department': 'Sound', 'name': 'Peter Burgis', 'job': 'Foley', 'gender': 2}, {'department': 'Visual Effects', 'name': 'Aaron Reznick', 'job': '3D Coordinator', 'gender': 2}, {'department': 'Writing', 'name': 'Jay Oliva', 'job': 'Storyboard', 'gender': 1}], [{'department': 'Art', 'name': 'Paul Hotte', 'job': 'Set Decoration', 'gender': 1}, {'department': 'Camera', 'name': 'Ueli Steiger', 'job': 'Director of Photography', 'gender': 1}, {'department': 'Costume & Make-Up', 'name': 'Renée April', 'job': 'Costume Design', 'gender': 0}, {'department': 'Crew', 'name': 'Neil Corbould', 'job': 'Special Effects', 'gender': 2}, {'department': 'Directing', 'name': 'Roland Emmerich', 'job': 'Director', 'gender': 1}, {'department': 'Editing', 'name': 'David Brenner', 'job': 'Editor', 'gender': 1}, {'department': 'Lighting', 'name': 'Logan Leabo', 'job': 'Gaffer', 'gender': 2}, {'department': 'Production', 'name': 'Roland Emmerich', 'job': 'Producer', 'gender': 1}, {'department': 'Sound', 'name': 'Harald Kloser', 'job': 'Original Music Composer', 'gender': 1}, {'department': 'Visual Effects', 'name': 'Kristen Branan', 'job': 'Visual Effects', 'gender': 2}, {'department': 'Writing', 'name': 'Roland Emmerich', 'job': 'Screenplay', 'gender': 1}], [{'department': 'Art', 'name': 'Nathan Crowley', 'job': 'Production Design', 'gender': 1}, {'department': 'Camera', 'name': 'Wally Pfister', 'job': 'Director of Photography', 'gender': 1}, {'department': 'Costume & Make-Up', 'name': 'Lindy Hemming', 'job': 'Costume Design', 'gender': 0}, {'department': 'Crew', 'name': 'Aaron Vexler', 'job': 'Stunts', 'gender': 2}, {'department': 'Directing', 'name': 'Christopher Nolan', 'job': 'Director', 'gender': 1}, {'department': 'Editing', 'name': 'Lee Smith', 'job': 'Editor', 'gender': 1}, {'department': 'Lighting', 'name': "Brian 'Buzz' Buzzelli", 'job': 'Rigging Grip', 'gender': 2}, {'department': 'Production', 'name': 'Christopher Nolan', 'job': 'Producer', 'gender': 1}, {'department': 'Sound', 'name': 'Hans Zimmer', 'job': 'Original Music Composer', 'gender': 1}, {'department': 'Visual Effects', 'name': 'Mike Chambers', 'job': 'Visual Effects Producer', 'gender': 2}, {'department': 'Writing', 'name': 'Bob Kane', 'job': 'Characters', 'gender': 1}], [{'department': 'Art', 'name': 'Kevin Kavanaugh', 'job': 'Production Design', 'gender': 2}, {'department': 'Camera', 'name': 'Gregor Tavenner', 'job': 'Camera Operator', 'gender': 2}, {'department': 'Costume & Make-Up', 'name': 'Luca Mosca', 'job': 'Costume Design', 'gender': 1}, {'department': 'Crew', 'name': 'Eddie Grisco', 'job': 'Armorer', 'gender': 2}, {'department': 'Directing', 'name': 'Chad Stahelski', 'job': 'Director', 'gender': 1}, {'department': 'Editing', 'name': 'Evan Schiff', 'job': 'Editor', 'gender': 2}, {'department': 'Lighting', 'name': 'Bill Almeida', 'job': 'Gaffer', 'gender': 2}, {'department': 'Production', 'name': 'Jessica Kelly', 'job': 'Casting', 'gender': 0}, {'department': 'Sound', 'name': 'Richard Henderson', 'job': 'Music Editor', 'gender': 2}, {'department': 'Visual Effects', 'name': 'Chris Ebeling', 'job': 'Animation', 'gender': 2}, {'department': 'Writing', 'name': 'Derek Kolstad', 'job': 'Characters', 'gender': 2}], [{'department': 'Art', 'name': 'Lionel Banks', 'job': 'Art Direction', 'gender': 1}, {'department': 'Camera', 'name': 'Joseph Walker', 'job': 'Director of Photography', 'gender': 1}, {'department': 'Costume & Make-Up', 'name': 'Robert Kalloch', 'job': 'Costume Design', 'gender': 1}, {'department': 'Crew', 'name': 'Myles Connolly', 'job': 'Additional Dialogue', 'gender': 1}, {'department': 'Directing', 'name': 'Frank Capra', 'job': 'Director', 'gender': 1}, {'department': 'Editing', 'name': 'Gene Havlick', 'job': 'Editor', 'gender': 1}, {'department': 'Production', 'name': 'Frank Capra', 'job': 'Producer', 'gender': 1}, {'department': 'Sound', 'name': 'Dimitri Tiomkin', 'job': 'Original Music Composer', 'gender': 1}, {'department': 'Visual Effects', 'name': 'Slavko Vorkapich', 'job': 'Visual Effects', 'gender': 1}, {'department': 'Writing', 'name': 'Sidney Buchman', 'job': 'Screenplay', 'gender': 1}], [{'department': 'Art', 'name': 'Scott Cobb', 'job': 'Assistant Art Director', 'gender': 2}, {'department': 'Camera', 'name': 'Patrick Loungway', 'job': 'Camera Operator', 'gender': 1}, {'department': 'Costume & Make-Up', 'name': 'Louis Visco', 'job': 'Set Dressing Artist', 'gender': 2}, {'department': 'Crew', 'name': 'Michael Toji', 'job': 'Post Production Supervisor', 'gender': 2}, {'department': 'Directing', 'name': 'Renee Rosenfeld', 'job': 'Script Supervisor', 'gender': 2}, {'department': 'Editing', 'name': 'Kurt Mueller', 'job': 'Archival Footage Coordinator', 'gender': 2}, {'department': 'Lighting', 'name': 'Dayton Nietert', 'job': 'Gaffer', 'gender': 2}, {'department': 'Production', 'name': 'Debra Grieco', 'job': 'Unit Production Manager', 'gender': 0}, {'department': 'Sound', 'name': 'Anna MacKenzie', 'job': 'Supervising Sound Editor', 'gender': 2}, {'department': 'Visual Effects', 'name': 'Calvin Roberts', 'job': '24 Frame Playback', 'gender': 2}, {'department': 'Writing', 'name': 'Roel Robles', 'job': 'Storyboard', 'gender': 2}], [{'department': 'Art', 'name': 'François Audouy', 'job': 'Production Design', 'gender': 2}, {'department': 'Camera', 'name': 'John Mathieson', 'job': 'Director of Photography', 'gender': 1}, {'department': 'Costume & Make-Up', 'name': 'Daniel Orlandi', 'job': 'Costume Design', 'gender': 1}, {'department': 'Crew', 'name': 'Rocky Abou-Sakher', 'job': 'Stunts', 'gender': 2}, {'department': 'Directing', 'name': 'James Mangold', 'job': 'Director', 'gender': 1}, {'department': 'Editing', 'name': 'Michael McCusker', 'job': 'Editor', 'gender': 1}, {'department': 'Lighting', 'name': "Jaim O'Neil", 'job': 'Gaffer', 'gender': 2}, {'department': 'Production', 'name': 'Lauren Shuler Donner', 'job': 'Producer', 'gender': 0}, {'department': 'Sound', 'name': 'Laura Graham', 'job': 'ADR Editor', 'gender': 2}, {'department': 'Visual Effects', 'name': 'Preethi Venkateswaran', 'job': '3D Modeller', 'gender': 2}, {'department': 'Writing', 'name': 'David James Kelly', 'job': 'Story', 'gender': 2}], [{'department': 'Art', 'name': 'Chris Shriver', 'job': 'Art Direction', 'gender': 2}, {'department': 'Camera', 'name': 'Rodrigo Prieto', 'job': 'Director of Photography', 'gender': 1}, {'department': 'Costume & Make-Up', 'name': 'Sandy Powell', 'job': 'Costume Design', 'gender': 0}, {'department': 'Crew', 'name': 'Chris Bertholf', 'job': 'Scenic Artist', 'gender': 2}, {'department': 'Directing', 'name': 'Martin Scorsese', 'job': 'Director', 'gender': 1}, {'department': 'Editing', 'name': 'Thelma Schoonmaker', 'job': 'Editor', 'gender': 0}, {'department': 'Lighting', 'name': 'Kevin Flynn', 'job': 'Rigging Grip', 'gender': 1}, {'department': 'Production', 'name': 'Riza Aziz', 'job': 'Producer', 'gender': 2}, {'department': 'Sound', 'name': 'Randall Poster', 'job': 'Music Supervisor', 'gender': 1}, {'department': 'Visual Effects', 'name': 'Drew Jiritano', 'job': 'Special Effects Supervisor', 'gender': 2}, {'department': 'Writing', 'name': 'Terence Winter', 'job': 'Screenplay', 'gender': 1}], [{'department': 'Directing', 'name': 'Murray Jordan', 'job': 'Director', 'gender': 2}, {'department': 'Writing', 'name': 'Donald Arthur', 'job': 'Writer', 'gender': 2}], [{'department': 'Directing', 'name': 'Stephen Johnson', 'job': 'Director', 'gender': 2}], [{'department': 'Art', 'name': 'Mayne Berke', 'job': 'Production Design', 'gender': 1}, {'department': 'Camera', 'name': 'Jean-Yves Escoffier', 'job': 'Director of Photography', 'gender': 1}, {'department': 'Costume & Make-Up', 'name': 'April Ferry', 'job': 'Costume Design', 'gender': 0}, {'department': 'Crew', 'name': 'Nancy Young', 'job': 'Stunts', 'gender': 0}, {'department': 'Directing', 'name': 'John Herzfeld', 'job': 'Director', 'gender': 1}, {'department': 'Editing', 'name': 'Steven Cohen', 'job': 'Editor', 'gender': 1}, {'department': 'Lighting', 'name': 'Michael Bauman', 'job': 'Lighting Technician', 'gender': 1}, {'department': 'Production', 'name': 'Keith Addis', 'job': 'Producer', 'gender': 1}, {'department': 'Sound', 'name': 'Anthony Marinelli', 'job': 'Original Music Composer', 'gender': 1}, {'department': 'Visual Effects', 'name': 'Paul J. Lombardi', 'job': 'Special Effects Supervisor', 'gender': 1}, {'department': 'Writing', 'name': 'John Herzfeld', 'job': 'Writer', 'gender': 1}], [{'department': 'Art', 'name': 'Perry Andelin Blake', 'job': 'Production Design', 'gender': 1}, {'department': 'Camera', 'name': 'Michael Barrett', 'job': 'Director of Photography', 'gender': 1}, {'department': 'Costume & Make-Up', 'name': 'Ellen Lutter', 'job': 'Costume Design', 'gender': 0}, {'department': 'Crew', 'name': 'Patsy Bouge', 'job': 'Post Production Supervisor', 'gender': 2}, {'department': 'Directing', 'name': 'Dennis Dugan', 'job': 'Director', 'gender': 1}, {'department': 'Editing', 'name': 'Tom Costain', 'job': 'Editor', 'gender': 1}, {'department': 'Production', 'name': 'Adam Sandler', 'job': 'Producer', 'gender': 1}, {'department': 'Sound', 'name': 'Rupert Gregson-Williams', 'job': 'Original Music Composer', 'gender': 1}, {'department': 'Writing', 'name': 'Adam Sandler', 'job': 'Screenplay', 'gender': 1}], [{'department': 'Art', 'name': 'Hugues Tissandier', 'job': 'Production Design', 'gender': 1}, {'department': 'Camera', 'name': 'Philippe Brunel', 'job': 'Camera Operator', 'gender': 2}, {'department': 'Costume & Make-Up', 'name': 'Olivier Bériot', 'job': 'Costume Design', 'gender': 1}, {'department': 'Crew', 'name': 'Jean-Claude Mézières', 'job': 'Graphic Novel Illustrator', 'gender': 1}, {'department': 'Directing', 'name': 'Luc Besson', 'job': 'Director', 'gender': 1}, {'department': 'Editing', 'name': 'Julien Rey', 'job': 'Editor', 'gender': 2}, {'department': 'Lighting', 'name': 'Brice Bailly', 'job': 'Electrician', 'gender': 2}, {'department': 'Production', 'name': 'Luc Besson', 'job': 'Producer', 'gender': 1}, {'department': 'Sound', 'name': 'Alexandre Desplat', 'job': 'Music', 'gender': 1}, {'department': 'Visual Effects', 'name': 'Philippe Hubin', 'job': 'Special Effects Supervisor', 'gender': 2}, {'department': 'Writing', 'name': 'Luc Besson', 'job': 'Writer', 'gender': 1}], [{'department': 'Art', 'name': 'Mark S. Freeborn', 'job': 'Production Design', 'gender': 1}, {'department': 'Camera', 'name': 'Bill Roe', 'job': 'Director of Photography', 'gender': 1}, {'department': 'Costume & Make-Up', 'name': 'Lisa Tomczeszyn', 'job': 'Costume Design', 'gender': 1}, {'department': 'Crew', 'name': 'Julio Pallares', 'job': 'Makeup Effects', 'gender': 2}, {'department': 'Directing', 'name': 'Chris Carter', 'job': 'Director', 'gender': 1}, {'department': 'Editing', 'name': 'Richard A. Harris', 'job': 'Editor', 'gender': 1}, {'department': 'Lighting', 'name': 'Ryan Bailey', 'job': 'Gaffer', 'gender': 2}, {'department': 'Production', 'name': 'Chris Carter', 'job': 'Producer', 'gender': 1}, {'department': 'Sound', 'name': 'Mark Snow', 'job': 'Original Music Composer', 'gender': 1}, {'department': 'Visual Effects', 'name': 'Steve Au', 'job': 'CG Painter', 'gender': 2}, {'department': 'Writing', 'name': 'Chris Carter', 'job': 'Screenplay', 'gender': 1}], [{'department': 'Art', 'name': 'Eve Stewart', 'job': 'Production Design', 'gender': 0}, {'department': 'Camera', 'name': 'Danny Cohen', 'job': 'Director of Photography', 'gender': 1}, {'department': 'Costume & Make-Up', 'name': 'Paco Delgado', 'job': 'Costume Design', 'gender': 2}, {'department': 'Crew', 'name': 'Dean Moran', 'job': 'Transportation Coordinator', 'gender': 2}, {'department': 'Directing', 'name': 'Tom Hooper', 'job': 'Director', 'gender': 1}, {'department': 'Editing', 'name': 'Melanie Oliver', 'job': 'Editor', 'gender': 0}, {'department': 'Production', 'name': 'Tim Bevan', 'job': 'Producer', 'gender': 1}, {'department': 'Sound', 'name': 'Claude-Michel Schönberg', 'job': 'Original Music Composer', 'gender': 1}, {'department': 'Visual Effects', 'name': 'Richard Bain', 'job': 'Visual Effects Supervisor', 'gender': 2}, {'department': 'Writing', 'name': 'Victor Hugo', 'job': 'Novel', 'gender': 1}], [], [], [{'department': 'Camera', 'name': 'Bojan Bazelli', 'job': 'Director of Photography', 'gender': 1}, {'department': 'Costume & Make-Up', 'name': 'Camille Friend', 'job': 'Hair Department Head', 'gender': 0}, {'department': 'Crew', 'name': 'Dhumal Sagar', 'job': 'Compositors', 'gender': 2}, {'department': 'Directing', 'name': 'Adam Shankman', 'job': 'Director', 'gender': 1}, {'department': 'Editing', 'name': 'Emma E. Hickox', 'job': 'Editor', 'gender': 0}, {'department': 'Production', 'name': 'Jennifer Gibgot', 'job': 'Producer', 'gender': 0}, {'department': 'Sound', 'name': 'Adam Anders', 'job': 'Music Director', 'gender': 1}, {'department': 'Writing', 'name': 'Justin Theroux', 'job': 'Screenplay', 'gender': 1}], [], [{'department': 'Art', 'name': 'Ed Verreaux', 'job': 'Production Design', 'gender': 1}, {'department': 'Camera', 'name': 'Sam Hicks', 'job': 'Grip', 'gender': 2}, {'department': 'Costume & Make-Up', 'name': 'April Ferry', 'job': 'Costume Design', 'gender': 0}, {'department': 'Crew', 'name': 'Brian N. Bentley', 'job': 'Compositors', 'gender': 2}, {'department': 'Directing', 'name': 'Colin Trevorrow', 'job': 'Director', 'gender': 1}, {'department': 'Editing', 'name': 'Kevin Stitt', 'job': 'Editor', 'gender': 1}, {'department': 'Lighting', 'name': 'Ai Saimoto', 'job': 'Lighting Technician', 'gender': 2}, {'department': 'Production', 'name': 'Steven Spielberg', 'job': 'Executive Producer', 'gender': 1}, {'department': 'Sound', 'name': 'Christopher Boyes', 'job': 'Sound Re-Recording Mixer', 'gender': 2}, {'department': 'Visual Effects', 'name': 'Mike Dharney', 'job': 'Animation', 'gender': 1}, {'department': 'Writing', 'name': 'Michael Crichton', 'job': 'Characters', 'gender': 1}], [{'department': 'Directing', 'name': 'Shital Shah', 'job': 'Director', 'gender': 2}, {'department': 'Production', 'name': 'Tajdar Amrohi', 'job': 'Producer', 'gender': 2}, {'department': 'Writing', 'name': 'Ankit Joshipura', 'job': 'Writer', 'gender': 2}], [], [{'department': 'Art', 'name': 'Brooke Dibble', 'job': 'Sculptor', 'gender': 2}, {'department': 'Camera', 'name': 'Philippe Rousselot', 'job': 'Director of Photography', 'gender': 1}, {'department': 'Costume & Make-Up', 'name': 'Colleen Atwood', 'job': 'Costume Design', 'gender': 2}, {'department': 'Crew', 'name': 'Richard Cheal', 'job': 'Prop Maker', 'gender': 2}, {'department': 'Directing', 'name': 'David Yates', 'job': 'Director', 'gender': 1}, {'department': 'Editing', 'name': 'Mark Day', 'job': 'Editor', 'gender': 1}, {'department': 'Lighting', 'name': 'Chuck Finch', 'job': 'Gaffer', 'gender': 1}, {'department': 'Production', 'name': 'J.K. Rowling', 'job': 'Producer', 'gender': 0}, {'department': 'Sound', 'name': 'James Newton Howard', 'job': 'Original Music Composer', 'gender': 1}, {'department': 'Visual Effects', 'name': 'Chris Lentz', 'job': 'Animation Supervisor', 'gender': 2}, {'department': 'Writing', 'name': 'J.K. Rowling', 'job': 'Screenplay', 'gender': 0}], [], [], [{'department': 'Art', 'name': 'Philip Harrison', 'job': 'Production Design', 'gender': 1}, {'department': 'Camera', 'name': 'John Lindley', 'job': 'Director of Photography', 'gender': 2}, {'department': 'Costume & Make-Up', 'name': 'Daniel J. Lester', 'job': 'Costume Design', 'gender': 1}, {'department': 'Crew', 'name': 'Bob Bottieri', 'job': 'Scenic Artist', 'gender': 2}, {'department': 'Directing', 'name': 'Jon Amiel', 'job': 'Director', 'gender': 1}, {'department': 'Editing', 'name': 'Terry Rawlings', 'job': 'Editor', 'gender': 1}, {'department': 'Lighting', 'name': 'Leonardo Beltrame', 'job': 'Gaffer', 'gender': 2}, {'department': 'Production', 'name': 'Sean Bailey', 'job': 'Producer', 'gender': 1}, {'department': 'Sound', 'name': 'Christopher Young', 'job': 'Original Music Composer', 'gender': 1}, {'department': 'Visual Effects', 'name': 'Victoria Alonso', 'job': 'Visual Effects Producer', 'gender': 2}, {'department': 'Writing', 'name': 'Cooper Layne', 'job': 'Writer', 'gender': 2}], [{'department': 'Directing', 'name': 'Markku Pölönen', 'job': 'Director', 'gender': 2}, {'department': 'Writing', 'name': 'Markku Pölönen', 'job': 'Writer', 'gender': 2}], [{'department': 'Art', 'name': 'Holger Gross', 'job': 'Production Design', 'gender': 1}, {'department': 'Camera', 'name': 'Hugh Johnson', 'job': 'Director of Photography', 'gender': 1}, {'department': 'Costume & Make-Up', 'name': 'Ellen Mirojnick', 'job': 'Costume Design', 'gender': 0}, {'department': 'Crew', 'name': 'Ve Neill', 'job': 'Makeup Effects', 'gender': 0}, {'department': 'Directing', 'name': 'David Twohy', 'job': 'Director', 'gender': 1}, {'department': 'Editing', 'name': 'Martin Hunter', 'job': 'Editor', 'gender': 1}, {'department': 'Lighting', 'name': 'Drew Davidson', 'job': 'Gaffer', 'gender': 2}, {'department': 'Production', 'name': 'Vin Diesel', 'job': 'Producer', 'gender': 1}, {'department': 'Sound', 'name': 'Scott Martin Gershin', 'job': 'Sound Designer', 'gender': 1}, {'department': 'Visual Effects', 'name': 'Peter Chiang', 'job': 'Visual Effects Supervisor', 'gender': 2}, {'department': 'Writing', 'name': 'Jim Wheat', 'job': 'Characters', 'gender': 1}], [], [{'department': 'Art', 'name': 'Nathan Crowley', 'job': 'Production Design', 'gender': 1}, {'department': 'Camera', 'name': 'Hoyte van Hoytema', 'job': 'Director of Photography', 'gender': 1}, {'department': 'Costume & Make-Up', 'name': 'Jeffrey Kurland', 'job': 'Costume Design', 'gender': 1}, {'department': 'Crew', 'name': 'Scott R. Fisher', 'job': 'Special Effects Coordinator', 'gender': 2}, {'department': 'Directing', 'name': 'Christopher Nolan', 'job': 'Director', 'gender': 1}, {'department': 'Editing', 'name': 'Lee Smith', 'job': 'Editor', 'gender': 1}, {'department': 'Lighting', 'name': 'R. Adam Chambers', 'job': 'Gaffer', 'gender': 2}, {'department': 'Production', 'name': 'Christopher Nolan', 'job': 'Producer', 'gender': 1}, {'department': 'Sound', 'name': 'Richard King', 'job': 'Supervising Sound Editor', 'gender': 1}, {'department': 'Visual Effects', 'name': 'Paul Corbould', 'job': 'Special Effects Supervisor', 'gender': 2}, {'department': 'Writing', 'name': 'Christopher Nolan', 'job': 'Writer', 'gender': 1}], [{'department': 'Art', 'name': 'Harley Jessup', 'job': 'Production Design', 'gender': 2}, {'department': 'Camera', 'name': 'Perrin Cutting', 'job': 'Camera Department Manager', 'gender': 2}, {'department': 'Costume & Make-Up', 'name': 'Sophie Vincelette', 'job': 'Set Dressing Supervisor', 'gender': 2}, {'department': 'Crew', 'name': 'Thomas Porter', 'job': 'Supervising Technical Director', 'gender': 2}, {'department': 'Directing', 'name': 'Pete Docter', 'job': 'Director', 'gender': 1}, {'department': 'Editing', 'name': 'Jim Stewart', 'job': 'Editor', 'gender': 1}, {'department': 'Lighting', 'name': 'Jean-Claude Kalache', 'job': 'Lighting Supervisor', 'gender': 2}, {'department': 'Production', 'name': 'Darla K. Anderson', 'job': 'Producer', 'gender': 2}, {'department': 'Sound', 'name': 'Gary Rydstrom', 'job': 'Sound Designer', 'gender': 1}, {'department': 'Visual Effects', 'name': 'Rick Sayre', 'job': 'Shading', 'gender': 2}, {'department': 'Writing', 'name': 'Jill Culton', 'job': 'Original Story', 'gender': 0}], [{'department': 'Art', 'name': 'Tom Meyer', 'job': 'Production Design', 'gender': 1}, {'department': 'Camera', 'name': 'Mauro Fiore', 'job': 'Director of Photography', 'gender': 1}, {'department': 'Costume & Make-Up', 'name': 'Marlene Stewart', 'job': 'Costume Design', 'gender': 0}, {'department': 'Crew', 'name': 'Damion Poitier', 'job': 'Utility Stunts', 'gender': 1}, {'department': 'Directing', 'name': 'Shawn Levy', 'job': 'Director', 'gender': 1}, {'department': 'Editing', 'name': 'Dean Zimmerman', 'job': 'Editor', 'gender': 1}, {'department': 'Lighting', 'name': 'Chris McLeod', 'job': 'Additional Lighting Technician', 'gender': 2}, {'department': 'Production', 'name': 'Shawn Levy', 'job': 'Producer', 'gender': 1}, {'department': 'Sound', 'name': 'Danny Elfman', 'job': 'Original Music Composer', 'gender': 1}, {'department': 'Visual Effects', 'name': 'Erik Baker', 'job': 'Animation', 'gender': 2}, {'department': 'Writing', 'name': 'Leslie Bohem', 'job': 'Screenplay', 'gender': 1}], [{'department': 'Art', 'name': 'Christopher Glass', 'job': 'Production Design', 'gender': 2}, {'department': 'Camera', 'name': 'Rasmus Videbæk', 'job': 'Director of Photography', 'gender': 2}, {'department': 'Costume & Make-Up', 'name': 'Trish Summerville', 'job': 'Costume Design', 'gender': 2}, {'department': 'Crew', 'name': 'Burt Dalton', 'job': 'Special Effects Coordinator', 'gender': 2}, {'department': 'Directing', 'name': 'Nikolaj Arcel', 'job': 'Director', 'gender': 1}, {'department': 'Editing', 'name': 'Alan Edward Bell', 'job': 'Editor', 'gender': 1}, {'department': 'Lighting', 'name': 'Paul Avery', 'job': 'Best Boy Electric', 'gender': 2}, {'department': 'Production', 'name': 'Ron Howard', 'job': 'Producer', 'gender': 1}, {'department': 'Sound', 'name': 'Junkie XL', 'job': 'Original Music Composer', 'gender': 1}, {'department': 'Visual Effects', 'name': 'Sachin Mathew', 'job': 'Animation', 'gender': 2}, {'department': 'Writing', 'name': 'Nikolaj Arcel', 'job': 'Screenplay', 'gender': 1}], [{'department': 'Art', 'name': 'Paul Kirby', 'job': 'Production Design', 'gender': 1}, {'department': 'Camera', 'name': 'Barry Ackroyd', 'job': 'Director of Photography', 'gender': 1}, {'department': 'Costume & Make-Up', 'name': 'Mark Bridges', 'job': 'Costume Design', 'gender': 1}, {'department': 'Crew', 'name': 'Simon Atherton', 'job': 'Armorer', 'gender': 2}, {'department': 'Directing', 'name': 'Paul Greengrass', 'job': 'Director', 'gender': 1}, {'department': 'Editing', 'name': 'Christopher Rouse', 'job': 'Editor', 'gender': 2}, {'department': 'Lighting', 'name': 'Michael Ambrose', 'job': 'Gaffer', 'gender': 2}, {'department': 'Production', 'name': 'Matt Damon', 'job': 'Producer', 'gender': 1}, {'department': 'Sound', 'name': 'John Powell', 'job': 'Original Music Composer', 'gender': 1}, {'department': 'Visual Effects', 'name': 'Scott Fritts', 'job': 'Animation', 'gender': 2}, {'department': 'Writing', 'name': 'Robert Ludlum', 'job': 'Characters', 'gender': 1}], [{'department': 'Directing', 'name': 'Adam Wood', 'job': 'Director', 'gender': 1}, {'department': 'Writing', 'name': 'Elise Allen', 'job': 'Screenplay', 'gender': 0}], [{'department': 'Art', 'name': 'Ken Adam', 'job': 'Production Design', 'gender': 1}, {'department': 'Camera', 'name': 'Lionel Lindon', 'job': 'Director of Photography', 'gender': 1}, {'department': 'Costume & Make-Up', 'name': 'Edith Keon', 'job': 'Hairstylist', 'gender': 2}, {'department': 'Crew', 'name': 'Max Kleven', 'job': 'Stunts', 'gender': 1}, {'department': 'Directing', 'name': 'Michael Anderson', 'job': 'Director', 'gender': 1}, {'department': 'Editing', 'name': 'Howard Epstein', 'job': 'Editor', 'gender': 1}, {'department': 'Production', 'name': 'Kevin McClory', 'job': 'Associate Producer', 'gender': 1}, {'department': 'Sound', 'name': 'Victor Young', 'job': 'Original Music Composer', 'gender': 1}, {'department': 'Visual Effects', 'name': 'Fred Sersen', 'job': 'Visual Effects Supervisor', 'gender': 2}, {'department': 'Writing', 'name': 'Jules Verne', 'job': 'Novel', 'gender': 1}], []]

In [ ]:
df.loc[df.crew.isnull(), "crew"] =(df_total.loc[df_total.crew.isnull(), "train_bool"]*missing_crew).loc[df[df.crew.isnull()].index]
df_test.loc[df_test.crew.isnull(), "crew"] =(~df_total.loc[df_total.crew.isnull(), "train_bool"]*missing_crew).loc[df_test[df_test.crew.isnull()].index]
df_total.loc[df_total.crew.isnull(), "crew"] = np.array(missing_crew)

In [ ]:
df.loc[:, "crew_names"] = df.crew.apply(lambda x: [var["name"] for var in ast.literal_eval(str(x))])
df_test.loc[:, "crew_names"] = df_test.crew.apply(lambda x: [var["name"] for var in ast.literal_eval(str(x))])
df_total.loc[:, "crew_names"] = df_total.crew.apply(lambda x: [var["name"] for var in ast.literal_eval(str(x))])

df.loc[:, "crew_male"] = df.crew.apply(lambda x: sum([1 for var in ast.literal_eval(str(x)) if "gender" in var and var["gender"] == 0]))
df_test.loc[:, "crew_male"] = df_test.crew.apply(lambda x: sum([1 for var in ast.literal_eval(str(x)) if "gender" in var and var["gender"] == 0]))
df_total.loc[:, "crew_male"] = df_total.crew.apply(lambda x: sum([1 for var in ast.literal_eval(str(x)) if "gender" in var and var["gender"] == 0]))

df.loc[:, "crew_female"] = df.crew.apply(lambda x: sum([1 for var in ast.literal_eval(str(x)) if "gender" in var and var["gender"] == 1]))
df_test.loc[:, "crew_female"] = df_test.crew.apply(lambda x: sum([1 for var in ast.literal_eval(str(x)) if "gender" in var and var["gender"] == 1]))
df_total.loc[:, "crew_female"] = df_total.crew.apply(lambda x: sum([1 for var in ast.literal_eval(str(x)) if "gender" in var and var["gender"] == 1]))

df.loc[:, "crew_unknown_gender"] = df.crew.apply(lambda x: sum([1 for var in ast.literal_eval(str(x)) if "gender" in var and var["gender"] == 2]))
df_test.loc[:, "crew_unknown_gender"] = df_test.crew.apply(lambda x: sum([1 for var in ast.literal_eval(str(x)) if "gender" in var and var["gender"] == 2]))
df_total.loc[:, "crew_unknown_gender"] = df_total.crew.apply(lambda x: sum([1 for var in ast.literal_eval(str(x)) if "gender" in var and var["gender"] == 2]))

df.loc[:, "crew_male_ratio"] = df[["crew_male", "crew_female"]].apply(lambda x: x["crew_male"]*100/(x["crew_female"]+x["crew_male"]), axis=1)
df_test.loc[:, "crew_male_ratio"] = df_test[["crew_male", "crew_female"]].apply(lambda x: x["crew_male"]*100/(x["crew_female"]+x["crew_male"]), axis=1)
df_total.loc[:, "crew_male_ratio"] = df_total[["crew_male", "crew_female"]].apply(lambda x: x["crew_male"]*100/(x["crew_female"]+x["crew_male"]), axis=1)

df.loc[:, "crew_size"] = df.crew.apply(lambda x:  len(ast.literal_eval(str(x))))
df_test.loc[:, "crew_size"] = df_test.crew.apply(lambda x:  len(ast.literal_eval(str(x))))
df_total.loc[:, "crew_size"] = df_total.crew.apply(lambda x: len(ast.literal_eval(str(x))))

In [ ]:
crewMost = [x[0] for x in Counter("|".join(df["crew_names"].apply(lambda x:"|".join(x)).values).split("|")).most_common(30)]
Counter("|".join(df["crew_names"].apply(lambda x:"|".join(x)).values).split("|")).most_common(30)

In [ ]:
for crew in crewMost:
    df[crew] = df.crew_names.apply(lambda x:1 if crew in x else 0)
    df_test[crew] = df_test.crew_names.apply(lambda x:1 if crew in x else 0)
    df_total[crew] = df_total.crew_names.apply(lambda x:1 if crew in x else 0)

In [ ]:
df.loc[:, "dept_names"] = df.crew.apply(lambda x: ",".join([var["department"].strip() for var in ast.literal_eval(str(x))]))
df_test.loc[:, "dept_names"] = df_test.crew.apply(lambda x: ",".join([var["department"].strip() for var in ast.literal_eval(str(x))]))
df_total.loc[:, "dept_names"] = df_total.crew.apply(lambda x: ",".join([var["department"].strip() for var in ast.literal_eval(str(x))]))

In [ ]:
department_list = list(set(",".join(list(set(df.dept_names.values))).split(",")))
for i, dept in enumerate(department_list):
    df[dept] = df.dept_names.str.count(dept)
    df_test[dept] = df_test.dept_names.str.count(dept)
    df_total[dept] = df_total.dept_names.str.count(dept)
    df[dept+"_crew_ratio"] = (df.dept_names.str.count(dept)/df.crew_size)*100
    df_test[dept+"_crew_ratio"] = (df_test.dept_names.str.count(dept)/df_test.crew_size)*100
    df_total[dept+"_crew_ratio"] = (df_total.dept_names.str.count(dept)/df_total.crew_size)*100

In [ ]:
df.groupby("year")["crew_size"].mean().plot(kind="bar", figsize=(20,10), stacked=True)
plt.title("Average Crew Size", fontsize=25)
plt.ylabel("Crew Size", fontsize=15);
plt.xticks(fontsize=13);

In [ ]:
df[[x+"_crew_ratio" for x in department_list]].mean().sort_values(ascending=True).plot(kind="barh", figsize=(20,10))
plt.title("Overall Crew Department Ratio");
plt.yticks(range(len(department_list)),df[[x+"_crew_ratio" for x in department_list]].mean().sort_values(ascending=True).index,fontsize=15);

In [ ]:
df[department_list].mean().sort_values(ascending=True).plot(kind="barh", figsize=(20,10))
plt.title("Overall Average Crew Department Size");
plt.yticks(fontsize=15);


In [ ]:
df.groupby("year")[department_list[:len(department_list)//2]].mean().plot(kind="bar", figsize=(20,10), stacked=True)
plt.title("Average Crew Department Size First Half", fontsize=25)
plt.ylabel("Crew Size", fontsize=15);
plt.xticks(fontsize=13);

In [ ]:
df.groupby("year")[department_list[len(department_list)//2:]].mean().plot(kind="bar", figsize=(20,10), stacked=True)
plt.title("Average Crew  Department Size Second Half", fontsize=25)
plt.ylabel("Crew Size", fontsize=15);
plt.xticks(fontsize=13);

In [ ]:
df.head()

In [ ]:
keep =list(df.describe().columns)

In [ ]:
train = df[keep]
keep.remove("revenue")
keep.remove("revenue_log")
test = df_test[keep]

In [ ]:
drop = train.corr().revenue[abs(train.corr().revenue) < 0.005].index

In [ ]:
cols = train.corr().nlargest(20,"revenue")["revenue"].index
plt.figure(figsize=(20,10))
sns.heatmap(train.corr().loc[cols,cols],annot=True,fmt=".2f");

In [ ]:
train.drop("revenue", inplace=True, axis=1)

In [ ]:
X = train.drop("revenue_log", axis=1)
y = train["revenue_log"]

In [ ]:
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [ ]:
model = XGBRegressor(n_estimators=3000)
model.fit(X_train, y_train,early_stopping_rounds=10,eval_set=[(X_test, y_test)], verbose=False)

In [ ]:
result = pd.DataFrame(model.predict(test.loc[:, X_train.columns]), index=df_test.index, columns=["revenue"])

In [ ]:
result["revenue"] = np.e**result.revenue

In [ ]:
result.reset_index(inplace=True)
result.columns = ["Id","revenue"]

In [ ]:
result.to_csv("result.csv",index=False)